In [2]:
pip install pytorch_lightning

  Using cached pytorch_lightning-1.7.0-py3-none-any.whl (700 kB)
  Using cached pyDeprecate-0.3.2-py3-none-any.whl (10 kB)
  Using cached tensorboard-2.9.1-py3-none-any.whl (5.8 MB)
  Using cached torchmetrics-0.9.3-py3-none-any.whl (419 kB)
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.6.0
    Uninstalling tensorboard-2.6.0:
      Successfully uninstalled tensorboard-2.6.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.6.2 requires tensorboard<2.7,>=2.6.0, but you have tensorboard 2.9.1 which is incompatible.
tensorflow 2.6.2 requires typing-extensions~=3.7.4, but you have typing-extensions 4.1.0 which is incompatible.
Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install disent==0.4.0

  Using cached disent-0.4.0-py3-none-any.whl (340 kB)
  Using cached torch_optimizer-0.3.0-py3-none-any.whl (61 kB)
  Using cached pytorch_ranger-0.1.1-py3-none-any.whl (14 kB)
Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install matplotlib

Note: you may need to restart the kernel to use updated packages.


In [5]:
pip install numpy

Note: you may need to restart the kernel to use updated packages.


In [8]:
pip install wandb

Note: you may need to restart the kernel to use updated packages.


In [1]:
import pytorch_lightning as pl
from pytorch_lightning.loggers.wandb import WandbLogger
from torch.utils.data import DataLoader
from disent.dataset import DisentDataset
from disent.dataset.data import XYObjectData
from disent.dataset.sampling import GroundTruthPairOrigSampler
from disent.frameworks.ae import Ae
from disent.frameworks.vae import BetaVae, AdaVae
from disent.model import AutoEncoder
from disent.model.ae import DecoderConv64, EncoderConv64
from disent.dataset.transform import ToImgTensorF32
from disent.util import is_test_run  # you can ignore and remove this
from disent.metrics import metric_dci, metric_mig, metric_sap 
import torch
import matplotlib.pyplot as plt
import numpy as np
import wandb

wandb.init()
# prepare the data
data = XYObjectData()
dataset = DisentDataset(data, GroundTruthPairOrigSampler(), transform=ToImgTensorF32())
dataloader = DataLoader(dataset=dataset, batch_size=128, shuffle=True)

# create the pytorch lightning system
module: pl.LightningModule = AdaVae(
    model=AutoEncoder(
        encoder=EncoderConv64(x_shape=data.x_shape, z_size=6, z_multiplier=2),
        decoder=DecoderConv64(x_shape=data.x_shape, z_size=6),
    ),
    cfg=AdaVae.cfg(
        optimizer='adam', optimizer_kwargs=dict(lr=1e-3),
        loss_reduction='mean_sum', beta=2, ada_average_mode='gvae', ada_thresh_mode='kl',
    )
)
wandb_logger = WandbLogger(project="validation.ipynb")
# train the model
trainer = pl.Trainer(logger=wandb_logger, max_epochs = 10, fast_dev_run=is_test_run())
trainer.fit(module, dataloader)

module.eval()

get_repr = lambda x: module.encode(x.to(module.device))

a_results = {
        **metric_dci(dataset, get_repr, num_train=10 if is_test_run() else 1000, num_test=5 if is_test_run() else 500, boost_mode='sklearn'),
        **metric_mig(dataset, get_repr, num_train=20 if is_test_run() else 2000),
        **metric_sap(dataset, get_repr),
    }
print(a_results)
# AdaGVAE beta = 2

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: iceburg. Use `wandb login --relogin` to force relogin


/opt/conda/lib/python3.9/site-packages/pytorch_lightning/loggers/wandb.py:351: UserWarning: There is a wandb run already in progress and newly created instances of `WandbLogger` will reuse this run. If this is not desired, call `wandb.finish()` before instantiating `WandbLogger`.
  rank_zero_warn(
GPU available: True (cuda), used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/opt/conda/lib/python3.9/site-packages/pytorch_lightning/trainer/trainer.py:1763: PossibleUserWarning: GPU available but not used. Set `accelerator` and `devices` using `Trainer(accelerator='gpu', devices=1)`.
  rank_zero_warn(
/opt/conda/lib/python3.9/site-packages/pytorch_lightning/trainer/configuration_validator.py:117: PossibleUserWarning: You defined a `validation_step` but have no `val_dataloader`. Skipping val loop.
  rank_zero_warn(

  | Name                          | Type                | Params
-------------------------------------

Training: 0it [00:00, ?it/s]

/opt/conda/lib/python3.9/site-packages/pytorch_lightning/callbacks/progress/base.py:247: UserWarning: The progress bar already tracks a metric with the name(s) 'loss' and `self.log('loss', ..., prog_bar=True)` will overwrite this value.  If this is undesired, change the name or override `get_metrics()` in the progress bar callback.
  rank_zero_warn(
`Trainer.fit` stopped: `max_epochs=10` reached.


{'dci.informativeness_train': 0.95875, 'dci.informativeness_test': 0.1855, 'dci.disentanglement': 0.009197158716181066, 'dci.completeness': 0.00750449279484891, 'mig.discrete_score': 0.008437209629986486, 'sap.score': 0.0}


In [2]:
import pytorch_lightning as pl
from pytorch_lightning.loggers.wandb import WandbLogger
from torch.utils.data import DataLoader
from disent.dataset import DisentDataset
from disent.dataset.data import XYObjectData
from disent.dataset.sampling import GroundTruthPairOrigSampler
from disent.frameworks.ae import Ae
from disent.frameworks.vae import BetaVae, AdaVae
from disent.model import AutoEncoder
from disent.model.ae import DecoderConv64, EncoderConv64
from disent.dataset.transform import ToImgTensorF32
from disent.util import is_test_run  # you can ignore and remove this
from disent.metrics import metric_dci, metric_mig, metric_sap 
import torch
import matplotlib.pyplot as plt
import numpy as np
import wandb

wandb.init()
# prepare the data
data = XYObjectData()
dataset = DisentDataset(data, GroundTruthPairOrigSampler(), transform=ToImgTensorF32())
dataloader = DataLoader(dataset=dataset, batch_size=128, shuffle=True)

# create the pytorch lightning system
module: pl.LightningModule = AdaVae(
    model=AutoEncoder(
        encoder=EncoderConv64(x_shape=data.x_shape, z_size=6, z_multiplier=2),
        decoder=DecoderConv64(x_shape=data.x_shape, z_size=6),
    ),
    cfg=AdaVae.cfg(
        optimizer='adam', optimizer_kwargs=dict(lr=1e-3),
        loss_reduction='mean_sum', beta=1, ada_average_mode='gvae', ada_thresh_mode='kl',
    )
)
wandb_logger = WandbLogger(project="validation.ipynb")
# train the model
trainer = pl.Trainer(logger=wandb_logger, max_epochs = 10, fast_dev_run=is_test_run())
trainer.fit(module, dataloader)

module.eval()

get_repr = lambda x: module.encode(x.to(module.device))

a_results = {
        **metric_dci(dataset, get_repr, num_train=10 if is_test_run() else 1000, num_test=5 if is_test_run() else 500, boost_mode='sklearn'),
        **metric_mig(dataset, get_repr, num_train=20 if is_test_run() else 2000),
        **metric_sap(dataset, get_repr),
    }
print(a_results)
# AdaGVAE beta = 1

aug_loss,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch,▁▁▁▁▂▂▂▂▃▃▃▃▃▃▃▃▄▄▄▄▅▅▅▅▆▆▆▆▆▆▆▆▇▇▇▇████
kl_loss,▅▁▁▁█▂▂▂▂▄▂▁▁▁▂▁▁▁▁▁▂▁▁▂▁▁█▃▂▅▁▃▂▁▁▂▁▁▁▁
kl_reg_loss,▅▁▁▁█▂▂▂▂▄▂▁▁▁▂▁▁▁▁▁▂▁▁▂▁▁█▃▂▅▁▃▂▁▁▂▁▁▁▁
loss,▄▅▂▄▅▃▄▅▅▁▄▄▂▃▇▅▄▅▅▃▆▆▃▃▃▃▅▃▃▅▃█▅▅▄▅▅▃▄▄
pixel_loss,▄▅▂▄▅▃▄▅▅▁▄▄▂▃▇▅▄▅▅▃▆▆▃▃▃▃▅▃▃▅▃█▅▅▄▅▅▃▄▄
recon_loss,▄▅▂▄▅▃▄▅▅▁▄▄▂▃▇▅▄▅▅▃▆▆▃▃▃▃▅▃▃▅▃█▅▅▄▅▅▃▄▄
reg_loss,▅▁▁▁█▂▂▂▂▄▂▁▁▁▂▁▁▁▁▁▂▁▁▂▁▁█▃▂▅▁▃▂▁▁▂▁▁▁▁
shared,▄▅▅▅▄▄▅▄▅▅▅▄▄▆▂▇▅▆▄▃▄▅▁▂▄▂▇▃▇██▁▅▅▅▄▅▅▅▄
trainer/global_step,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
aug_loss,0.0


/opt/conda/lib/python3.9/site-packages/pytorch_lightning/loggers/wandb.py:351: UserWarning: There is a wandb run already in progress and newly created instances of `WandbLogger` will reuse this run. If this is not desired, call `wandb.finish()` before instantiating `WandbLogger`.
  rank_zero_warn(
GPU available: True (cuda), used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/opt/conda/lib/python3.9/site-packages/pytorch_lightning/trainer/trainer.py:1763: PossibleUserWarning: GPU available but not used. Set `accelerator` and `devices` using `Trainer(accelerator='gpu', devices=1)`.
  rank_zero_warn(
/opt/conda/lib/python3.9/site-packages/pytorch_lightning/trainer/configuration_validator.py:117: PossibleUserWarning: You defined a `validation_step` but have no `val_dataloader`. Skipping val loop.
  rank_zero_warn(

  | Name                          | Type                | Params
-------------------------------------

Training: 0it [00:00, ?it/s]

/opt/conda/lib/python3.9/site-packages/pytorch_lightning/callbacks/progress/base.py:247: UserWarning: The progress bar already tracks a metric with the name(s) 'loss' and `self.log('loss', ..., prog_bar=True)` will overwrite this value.  If this is undesired, change the name or override `get_metrics()` in the progress bar callback.
  rank_zero_warn(
`Trainer.fit` stopped: `max_epochs=10` reached.


{'dci.informativeness_train': 0.9715, 'dci.informativeness_test': 0.729, 'dci.disentanglement': 0.4678957277868262, 'dci.completeness': 0.476518398132732, 'mig.discrete_score': 0.3479721196722052, 'sap.score': 0.010299999999999997}


In [1]:
import pytorch_lightning as pl
from pytorch_lightning.loggers.wandb import WandbLogger
from torch.utils.data import DataLoader
from disent.dataset import DisentDataset
from disent.dataset.data import XYObjectData
from disent.dataset.sampling import GroundTruthPairOrigSampler
from disent.frameworks.ae import Ae
from disent.frameworks.vae import BetaVae, AdaVae
from disent.model import AutoEncoder
from disent.model.ae import DecoderConv64, EncoderConv64
from disent.dataset.transform import ToImgTensorF32
from disent.util import is_test_run  # you can ignore and remove this
from disent.metrics import metric_dci, metric_mig, metric_sap 
import torch
import matplotlib.pyplot as plt
import numpy as np
import wandb

wandb.init()
# prepare the data
data = XYObjectData()
dataset = DisentDataset(data, GroundTruthPairOrigSampler(), transform=ToImgTensorF32())
dataloader = DataLoader(dataset=dataset, batch_size=128, shuffle=True)

# create the pytorch lightning system
module: pl.LightningModule = AdaVae(
    model=AutoEncoder(
        encoder=EncoderConv64(x_shape=data.x_shape, z_size=6, z_multiplier=2),
        decoder=DecoderConv64(x_shape=data.x_shape, z_size=6),
    ),
    cfg=AdaVae.cfg(
        optimizer='adam', optimizer_kwargs=dict(lr=1e-3),
        loss_reduction='mean_sum', beta=4, ada_average_mode='gvae', ada_thresh_mode='kl',
    )
)
wandb_logger = WandbLogger(project="validation.ipynb")
# train the model
trainer = pl.Trainer(logger=wandb_logger, max_epochs = 10, fast_dev_run=is_test_run())
trainer.fit(module, dataloader)

module.eval()

get_repr = lambda x: module.encode(x.to(module.device))

a_results = {
        **metric_dci(dataset, get_repr, num_train=10 if is_test_run() else 1000, num_test=5 if is_test_run() else 500, boost_mode='sklearn'),
        **metric_mig(dataset, get_repr, num_train=20 if is_test_run() else 2000),
        **metric_sap(dataset, get_repr),
    }
print(a_results)
# AdaGVAE beta = 4

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: iceburg. Use `wandb login --relogin` to force relogin


/opt/conda/lib/python3.9/site-packages/pytorch_lightning/loggers/wandb.py:351: UserWarning: There is a wandb run already in progress and newly created instances of `WandbLogger` will reuse this run. If this is not desired, call `wandb.finish()` before instantiating `WandbLogger`.
  rank_zero_warn(
GPU available: True (cuda), used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/opt/conda/lib/python3.9/site-packages/pytorch_lightning/trainer/trainer.py:1763: PossibleUserWarning: GPU available but not used. Set `accelerator` and `devices` using `Trainer(accelerator='gpu', devices=1)`.
  rank_zero_warn(
/opt/conda/lib/python3.9/site-packages/pytorch_lightning/trainer/configuration_validator.py:117: PossibleUserWarning: You defined a `validation_step` but have no `val_dataloader`. Skipping val loop.
  rank_zero_warn(

  | Name                          | Type                | Params
-------------------------------------

Training: 0it [00:00, ?it/s]

/opt/conda/lib/python3.9/site-packages/pytorch_lightning/callbacks/progress/base.py:247: UserWarning: The progress bar already tracks a metric with the name(s) 'loss' and `self.log('loss', ..., prog_bar=True)` will overwrite this value.  If this is undesired, change the name or override `get_metrics()` in the progress bar callback.
  rank_zero_warn(
`Trainer.fit` stopped: `max_epochs=10` reached.


{'dci.informativeness_train': 0.962, 'dci.informativeness_test': 0.217, 'dci.disentanglement': 0.0038403137500412575, 'dci.completeness': 0.004091004740570697, 'mig.discrete_score': 0.0033430075155543855, 'sap.score': 9.99999999999994e-05}


In [3]:
import pytorch_lightning as pl
from pytorch_lightning.loggers.wandb import WandbLogger
from torch.utils.data import DataLoader
from disent.dataset import DisentDataset
from disent.dataset.data import XYObjectData
from disent.dataset.sampling import GroundTruthPairOrigSampler
from disent.frameworks.ae import Ae
from disent.frameworks.vae import BetaVae, AdaVae
from disent.model import AutoEncoder
from disent.model.ae import DecoderConv64, EncoderConv64
from disent.dataset.transform import ToImgTensorF32
from disent.util import is_test_run  # you can ignore and remove this
from disent.metrics import metric_dci, metric_mig, metric_sap 
import torch
import matplotlib.pyplot as plt
import numpy as np
import wandb

wandb.init()
# prepare the data
data = XYObjectData()
dataset = DisentDataset(data, GroundTruthPairOrigSampler(), transform=ToImgTensorF32())
dataloader = DataLoader(dataset=dataset, batch_size=128, shuffle=True)

# create the pytorch lightning system
module: pl.LightningModule = AdaVae(
    model=AutoEncoder(
        encoder=EncoderConv64(x_shape=data.x_shape, z_size=6, z_multiplier=2),
        decoder=DecoderConv64(x_shape=data.x_shape, z_size=6),
    ),
    cfg=AdaVae.cfg(
        optimizer='adam', optimizer_kwargs=dict(lr=1e-3),
        loss_reduction='mean_sum', beta=3, ada_average_mode='gvae', ada_thresh_mode='kl',
    )
)
wandb_logger = WandbLogger(project="validation.ipynb")
# train the model
trainer = pl.Trainer(logger=wandb_logger, max_epochs = 10, fast_dev_run=is_test_run())
trainer.fit(module, dataloader)

module.eval()

get_repr = lambda x: module.encode(x.to(module.device))

a_results = {
        **metric_dci(dataset, get_repr, num_train=10 if is_test_run() else 1000, num_test=5 if is_test_run() else 500, boost_mode='sklearn'),
        **metric_mig(dataset, get_repr, num_train=20 if is_test_run() else 2000),
        **metric_sap(dataset, get_repr),
    }
print(a_results)

# AdaGVAE beta = 3

aug_loss,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch,▁▁▁▁▁▁▁▁▁▁▃▃▃▃▃▃▃▃▃▃▅▅▅▅▅▅▅▅▅▅▆▆▆▆▆▆▆▆▆█
kl_loss,▁▁▁▁▁▁▁▁▅▅▅▅▆▅▅▅▆▅▆▆▆▅▆▆▆▆▆▆▇▇▇▇▇▇██████
kl_reg_loss,▁▁▁▁▁▁▁▁▅▅▅▅▆▅▅▅▆▅▆▆▆▅▆▆▆▆▆▆▇▇▇▇▇▇██████
loss,▇▆█▇▇▆▇█▆▅▄▅▄▄▄▃▅▄▅▄▄▄▄▄▄▄▃▃▃▃▃▃▂▂▁▁▁▁▁▁
pixel_loss,▇▆█▇▇▇▇█▅▄▄▄▄▄▃▃▄▄▄▄▄▄▄▄▄▄▃▃▃▃▃▂▂▂▁▁▁▁▁▁
recon_loss,▇▆█▇▇▇▇█▅▄▄▄▄▄▃▃▄▄▄▄▄▄▄▄▄▄▃▃▃▃▃▂▂▂▁▁▁▁▁▁
reg_loss,▁▁▁▁▁▁▁▁▅▅▅▅▆▅▅▅▆▅▆▆▆▅▆▆▆▆▆▆▇▇▇▇▇▇██████
trainer/global_step,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
aug_loss,0.0
epoch,4


/opt/conda/lib/python3.9/site-packages/pytorch_lightning/loggers/wandb.py:351: UserWarning: There is a wandb run already in progress and newly created instances of `WandbLogger` will reuse this run. If this is not desired, call `wandb.finish()` before instantiating `WandbLogger`.
  rank_zero_warn(
GPU available: True (cuda), used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/opt/conda/lib/python3.9/site-packages/pytorch_lightning/trainer/trainer.py:1763: PossibleUserWarning: GPU available but not used. Set `accelerator` and `devices` using `Trainer(accelerator='gpu', devices=1)`.
  rank_zero_warn(
/opt/conda/lib/python3.9/site-packages/pytorch_lightning/trainer/configuration_validator.py:117: PossibleUserWarning: You defined a `validation_step` but have no `val_dataloader`. Skipping val loop.
  rank_zero_warn(

  | Name                          | Type                | Params
-------------------------------------

Training: 0it [00:00, ?it/s]

/opt/conda/lib/python3.9/site-packages/pytorch_lightning/callbacks/progress/base.py:247: UserWarning: The progress bar already tracks a metric with the name(s) 'loss' and `self.log('loss', ..., prog_bar=True)` will overwrite this value.  If this is undesired, change the name or override `get_metrics()` in the progress bar callback.
  rank_zero_warn(
`Trainer.fit` stopped: `max_epochs=10` reached.


{'dci.informativeness_train': 0.96375, 'dci.informativeness_test': 0.2235, 'dci.disentanglement': 0.010871655247839495, 'dci.completeness': 0.009935935145965802, 'mig.discrete_score': 0.006160505930339412, 'sap.score': 9.99999999999994e-05}


In [3]:
import pytorch_lightning as pl
from pytorch_lightning.loggers.wandb import WandbLogger
from torch.utils.data import DataLoader
from disent.dataset import DisentDataset
from disent.dataset.data import XYObjectData
from disent.dataset.sampling import GroundTruthPairOrigSampler
from disent.frameworks.ae import Ae
from disent.frameworks.vae import BetaVae, AdaVae
from disent.model import AutoEncoder
from disent.model.ae import DecoderConv64, EncoderConv64
from disent.dataset.transform import ToImgTensorF32
from disent.util import is_test_run  # you can ignore and remove this
from disent.metrics import metric_dci, metric_mig, metric_sap 
import torch
import matplotlib.pyplot as plt
import numpy as np
import wandb

wandb.init()
# prepare the data
data = XYObjectData()
dataset = DisentDataset(data, transform=ToImgTensorF32())
dataloader = DataLoader(dataset=dataset, batch_size=128, shuffle=True)

# create the pytorch lightning system
# create the pytorch lightning system
module: pl.LightningModule = BetaVae(
    model=AutoEncoder(
        encoder=EncoderConv64(x_shape=data.x_shape, z_size=6, z_multiplier=2),
        decoder=DecoderConv64(x_shape=data.x_shape, z_size=6),
    ),
    cfg=BetaVae.cfg(optimizer='adam', optimizer_kwargs=dict(lr=1e-3), loss_reduction='mean_sum', beta=4)
)
wandb_logger = WandbLogger(project="validation.ipynb")
# train the model
trainer = pl.Trainer(logger=wandb_logger, max_epochs = 10, fast_dev_run=is_test_run())
trainer.fit(module, dataloader)

module.eval()

get_repr = lambda x: module.encode(x.to(module.device))

a_results = {
        **metric_dci(dataset, get_repr, num_train=10 if is_test_run() else 1000, num_test=5 if is_test_run() else 500, boost_mode='sklearn'),
        **metric_mig(dataset, get_repr, num_train=20 if is_test_run() else 2000),
        **metric_sap(dataset, get_repr),
    }
print(a_results)
# BetaVAE beta = 4

/opt/conda/lib/python3.9/site-packages/pytorch_lightning/loggers/wandb.py:351: UserWarning: There is a wandb run already in progress and newly created instances of `WandbLogger` will reuse this run. If this is not desired, call `wandb.finish()` before instantiating `WandbLogger`.
  rank_zero_warn(
GPU available: True (cuda), used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/opt/conda/lib/python3.9/site-packages/pytorch_lightning/trainer/trainer.py:1763: PossibleUserWarning: GPU available but not used. Set `accelerator` and `devices` using `Trainer(accelerator='gpu', devices=1)`.
  rank_zero_warn(
/opt/conda/lib/python3.9/site-packages/pytorch_lightning/trainer/configuration_validator.py:117: PossibleUserWarning: You defined a `validation_step` but have no `val_dataloader`. Skipping val loop.
  rank_zero_warn(

  | Name                          | Type                | Params
-------------------------------------

Training: 0it [00:00, ?it/s]

/opt/conda/lib/python3.9/site-packages/pytorch_lightning/callbacks/progress/base.py:247: UserWarning: The progress bar already tracks a metric with the name(s) 'loss' and `self.log('loss', ..., prog_bar=True)` will overwrite this value.  If this is undesired, change the name or override `get_metrics()` in the progress bar callback.
  rank_zero_warn(
`Trainer.fit` stopped: `max_epochs=10` reached.


{'dci.informativeness_train': 0.9635, 'dci.informativeness_test': 0.236, 'dci.disentanglement': 0.005969988782628775, 'dci.completeness': 0.005825293230425216, 'mig.discrete_score': 0.006639468823705195, 'sap.score': 0.0}


In [4]:
import pytorch_lightning as pl
from pytorch_lightning.loggers.wandb import WandbLogger
from torch.utils.data import DataLoader
from disent.dataset import DisentDataset
from disent.dataset.data import XYObjectData
from disent.dataset.sampling import GroundTruthPairOrigSampler
from disent.frameworks.ae import Ae
from disent.frameworks.vae import BetaVae, AdaVae
from disent.model import AutoEncoder
from disent.model.ae import DecoderConv64, EncoderConv64
from disent.dataset.transform import ToImgTensorF32
from disent.util import is_test_run  # you can ignore and remove this
from disent.metrics import metric_dci, metric_mig, metric_sap 
import torch
import matplotlib.pyplot as plt
import numpy as np
import wandb

wandb.init()
# prepare the data
data = XYObjectData()
dataset = DisentDataset(data, transform=ToImgTensorF32())
dataloader = DataLoader(dataset=dataset, batch_size=128, shuffle=True)

# create the pytorch lightning system
# create the pytorch lightning system
module: pl.LightningModule = BetaVae(
    model=AutoEncoder(
        encoder=EncoderConv64(x_shape=data.x_shape, z_size=6, z_multiplier=2),
        decoder=DecoderConv64(x_shape=data.x_shape, z_size=6),
    ),
    cfg=BetaVae.cfg(optimizer='adam', optimizer_kwargs=dict(lr=1e-3), loss_reduction='mean_sum', beta=3)
)
wandb_logger = WandbLogger(project="validation.ipynb")
# train the model
trainer = pl.Trainer(logger=wandb_logger, max_epochs = 10, fast_dev_run=is_test_run())
trainer.fit(module, dataloader)

module.eval()

get_repr = lambda x: module.encode(x.to(module.device))

a_results = {
        **metric_dci(dataset, get_repr, num_train=10 if is_test_run() else 1000, num_test=5 if is_test_run() else 500, boost_mode='sklearn'),
        **metric_mig(dataset, get_repr, num_train=20 if is_test_run() else 2000),
        **metric_sap(dataset, get_repr),
    }
print(a_results)
# BetaVAE beta = 3

aug_loss,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch,▁▁▁▁▂▂▂▂▃▃▃▃▃▃▃▃▄▄▄▄▅▅▅▅▆▆▆▆▆▆▆▆▇▇▇▇████
kl_loss,▅█▃▃▃▃▂▄▄▄▄▄▂▂▂▃▁▂▁▁▄▂▁▂▁▁▃▁▁▂▂▁▁▁▁▁▁▁▂▁
kl_reg_loss,▅█▃▃▃▃▂▄▄▄▄▄▂▂▂▃▁▂▁▁▄▂▁▂▁▁▃▁▁▂▂▁▁▁▁▁▁▁▂▁
loss,▃▃▅▄▆▃▄█▄▃▅▆▆▃▄▅▂▅▅██▃▆▄▄▃▆▃▅▁▅▅▃▆▄▅▄▅▂▂
pixel_loss,▃▃▅▄▆▃▄█▄▃▅▆▆▃▄▅▂▅▅██▃▆▄▄▃▆▃▅▁▅▅▃▆▄▅▄▅▂▂
recon_loss,▃▃▅▄▆▃▄█▄▃▅▆▆▃▄▅▂▅▅██▃▆▄▄▃▆▃▅▁▅▅▃▆▄▅▄▅▂▂
reg_loss,▅█▃▃▃▃▂▄▄▄▄▄▂▂▂▃▁▂▁▁▄▂▁▂▁▁▃▁▁▂▂▁▁▁▁▁▁▁▂▁
trainer/global_step,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
aug_loss,0.0
epoch,9


/opt/conda/lib/python3.9/site-packages/pytorch_lightning/loggers/wandb.py:351: UserWarning: There is a wandb run already in progress and newly created instances of `WandbLogger` will reuse this run. If this is not desired, call `wandb.finish()` before instantiating `WandbLogger`.
  rank_zero_warn(
GPU available: True (cuda), used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/opt/conda/lib/python3.9/site-packages/pytorch_lightning/trainer/trainer.py:1763: PossibleUserWarning: GPU available but not used. Set `accelerator` and `devices` using `Trainer(accelerator='gpu', devices=1)`.
  rank_zero_warn(
/opt/conda/lib/python3.9/site-packages/pytorch_lightning/trainer/configuration_validator.py:117: PossibleUserWarning: You defined a `validation_step` but have no `val_dataloader`. Skipping val loop.
  rank_zero_warn(

  | Name                          | Type                | Params
-------------------------------------

Training: 0it [00:00, ?it/s]

/opt/conda/lib/python3.9/site-packages/pytorch_lightning/callbacks/progress/base.py:247: UserWarning: The progress bar already tracks a metric with the name(s) 'loss' and `self.log('loss', ..., prog_bar=True)` will overwrite this value.  If this is undesired, change the name or override `get_metrics()` in the progress bar callback.
  rank_zero_warn(
`Trainer.fit` stopped: `max_epochs=10` reached.


{'dci.informativeness_train': 0.9572499999999999, 'dci.informativeness_test': 0.219, 'dci.disentanglement': 0.006172441969339984, 'dci.completeness': 0.006268037868535997, 'mig.discrete_score': 0.011178546678891104, 'sap.score': 0.007000000000000003}


In [5]:
import pytorch_lightning as pl
from pytorch_lightning.loggers.wandb import WandbLogger
from torch.utils.data import DataLoader
from disent.dataset import DisentDataset
from disent.dataset.data import XYObjectData
from disent.dataset.sampling import GroundTruthPairOrigSampler
from disent.frameworks.ae import Ae
from disent.frameworks.vae import BetaVae, AdaVae
from disent.model import AutoEncoder
from disent.model.ae import DecoderConv64, EncoderConv64
from disent.dataset.transform import ToImgTensorF32
from disent.util import is_test_run  # you can ignore and remove this
from disent.metrics import metric_dci, metric_mig, metric_sap 
import torch
import matplotlib.pyplot as plt
import numpy as np
import wandb

wandb.init()
# prepare the data
data = XYObjectData()
dataset = DisentDataset(data, transform=ToImgTensorF32())
dataloader = DataLoader(dataset=dataset, batch_size=128, shuffle=True)

# create the pytorch lightning system
# create the pytorch lightning system
module: pl.LightningModule = BetaVae(
    model=AutoEncoder(
        encoder=EncoderConv64(x_shape=data.x_shape, z_size=6, z_multiplier=2),
        decoder=DecoderConv64(x_shape=data.x_shape, z_size=6),
    ),
    cfg=BetaVae.cfg(optimizer='adam', optimizer_kwargs=dict(lr=1e-3), loss_reduction='mean_sum', beta=2)
)
wandb_logger = WandbLogger(project="validation.ipynb")
# train the model
trainer = pl.Trainer(logger=wandb_logger, max_epochs = 10, fast_dev_run=is_test_run())
trainer.fit(module, dataloader)

module.eval()

get_repr = lambda x: module.encode(x.to(module.device))

a_results = {
        **metric_dci(dataset, get_repr, num_train=10 if is_test_run() else 1000, num_test=5 if is_test_run() else 500, boost_mode='sklearn'),
        **metric_mig(dataset, get_repr, num_train=20 if is_test_run() else 2000),
        **metric_sap(dataset, get_repr),
    }
print(a_results)
# BetaVAE beta = 2

aug_loss,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch,▁▁▁▁▂▂▂▂▃▃▃▃▃▃▃▃▄▄▄▄▅▅▅▅▆▆▆▆▆▆▆▆▇▇▇▇████
kl_loss,█▂▃▃▂▂▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▃▁▁▁▂▁▁▁▁▁▁▁▁▁▁▁
kl_reg_loss,█▂▃▃▂▂▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▃▁▁▁▂▁▁▁▁▁▁▁▁▁▁▁
loss,▆▄▇▅▅▁▆▄▃▃▇▄▆▅▄▇▃▄▃▅▅▄▅█▇▅▂▃▆▄▆▅▆▅▄▇▃▂▆▂
pixel_loss,▆▄▇▅▅▁▆▄▃▃▇▄▆▅▄▇▃▄▃▅▅▄▅█▇▅▂▃▆▄▆▅▆▅▄▇▃▂▆▂
recon_loss,▆▄▇▅▅▁▆▄▃▃▇▄▆▅▄▇▃▄▃▅▅▄▅█▇▅▂▃▆▄▆▅▆▅▄▇▃▂▆▂
reg_loss,█▂▃▃▂▂▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▃▁▁▁▂▁▁▁▁▁▁▁▁▁▁▁
trainer/global_step,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
aug_loss,0.0
epoch,9


/opt/conda/lib/python3.9/site-packages/pytorch_lightning/loggers/wandb.py:351: UserWarning: There is a wandb run already in progress and newly created instances of `WandbLogger` will reuse this run. If this is not desired, call `wandb.finish()` before instantiating `WandbLogger`.
  rank_zero_warn(
GPU available: True (cuda), used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/opt/conda/lib/python3.9/site-packages/pytorch_lightning/trainer/trainer.py:1763: PossibleUserWarning: GPU available but not used. Set `accelerator` and `devices` using `Trainer(accelerator='gpu', devices=1)`.
  rank_zero_warn(
/opt/conda/lib/python3.9/site-packages/pytorch_lightning/trainer/configuration_validator.py:117: PossibleUserWarning: You defined a `validation_step` but have no `val_dataloader`. Skipping val loop.
  rank_zero_warn(

  | Name                          | Type                | Params
-------------------------------------

Training: 0it [00:00, ?it/s]

/opt/conda/lib/python3.9/site-packages/pytorch_lightning/callbacks/progress/base.py:247: UserWarning: The progress bar already tracks a metric with the name(s) 'loss' and `self.log('loss', ..., prog_bar=True)` will overwrite this value.  If this is undesired, change the name or override `get_metrics()` in the progress bar callback.
  rank_zero_warn(
`Trainer.fit` stopped: `max_epochs=10` reached.


{'dci.informativeness_train': 0.9509999999999998, 'dci.informativeness_test': 0.2585, 'dci.disentanglement': 0.010707287774888604, 'dci.completeness': 0.017592249995094258, 'mig.discrete_score': 0.020320297942774707, 'sap.score': 0.013299999999999996}


In [1]:
import pytorch_lightning as pl
from pytorch_lightning.loggers.wandb import WandbLogger
from torch.utils.data import DataLoader
from disent.dataset import DisentDataset
from disent.dataset.data import XYObjectData
from disent.dataset.sampling import GroundTruthPairOrigSampler
from disent.frameworks.ae import Ae
from disent.frameworks.vae import BetaVae, AdaVae
from disent.model import AutoEncoder
from disent.model.ae import DecoderConv64, EncoderConv64
from disent.dataset.transform import ToImgTensorF32
from disent.util import is_test_run  # you can ignore and remove this
from disent.metrics import metric_dci, metric_mig, metric_sap 
import torch
import matplotlib.pyplot as plt
import numpy as np
import wandb

wandb.init()
# prepare the data
data = XYObjectData()
dataset = DisentDataset(data, transform=ToImgTensorF32())
dataloader = DataLoader(dataset=dataset, batch_size=128, shuffle=True)

# create the pytorch lightning system
# create the pytorch lightning system
module: pl.LightningModule = BetaVae(
    model=AutoEncoder(
        encoder=EncoderConv64(x_shape=data.x_shape, z_size=6, z_multiplier=2),
        decoder=DecoderConv64(x_shape=data.x_shape, z_size=6),
    ),
    cfg=BetaVae.cfg(optimizer='adam', optimizer_kwargs=dict(lr=1e-3), loss_reduction='mean_sum', beta=1)
)
wandb_logger = WandbLogger(project="validation.ipynb")
# train the model
trainer = pl.Trainer(logger=wandb_logger, max_epochs = 10, fast_dev_run=is_test_run())
trainer.fit(module, dataloader)

module.eval()

get_repr = lambda x: module.encode(x.to(module.device))

a_results = {
        **metric_dci(dataset, get_repr, num_train=10 if is_test_run() else 1000, num_test=5 if is_test_run() else 500, boost_mode='sklearn'),
        **metric_mig(dataset, get_repr, num_train=20 if is_test_run() else 2000),
        **metric_sap(dataset, get_repr),
    }
print(a_results)
# BetaVAE beta = 1

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: iceburg. Use `wandb login --relogin` to force relogin


/opt/conda/lib/python3.9/site-packages/pytorch_lightning/loggers/wandb.py:351: UserWarning: There is a wandb run already in progress and newly created instances of `WandbLogger` will reuse this run. If this is not desired, call `wandb.finish()` before instantiating `WandbLogger`.
  rank_zero_warn(
GPU available: True (cuda), used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/opt/conda/lib/python3.9/site-packages/pytorch_lightning/trainer/trainer.py:1763: PossibleUserWarning: GPU available but not used. Set `accelerator` and `devices` using `Trainer(accelerator='gpu', devices=1)`.
  rank_zero_warn(
/opt/conda/lib/python3.9/site-packages/pytorch_lightning/trainer/configuration_validator.py:117: PossibleUserWarning: You defined a `validation_step` but have no `val_dataloader`. Skipping val loop.
  rank_zero_warn(

  | Name                          | Type                | Params
-------------------------------------

Training: 0it [00:00, ?it/s]

/opt/conda/lib/python3.9/site-packages/pytorch_lightning/callbacks/progress/base.py:247: UserWarning: The progress bar already tracks a metric with the name(s) 'loss' and `self.log('loss', ..., prog_bar=True)` will overwrite this value.  If this is undesired, change the name or override `get_metrics()` in the progress bar callback.
  rank_zero_warn(
`Trainer.fit` stopped: `max_epochs=10` reached.


{'dci.informativeness_train': 0.9632499999999999, 'dci.informativeness_test': 0.309, 'dci.disentanglement': 0.01996698214528427, 'dci.completeness': 0.027402376355425695, 'mig.discrete_score': 0.018147187608936906, 'sap.score': 0.004399999999999994}


In [1]:
import pytorch_lightning as pl
from pytorch_lightning.loggers.wandb import WandbLogger
from torch.utils.data import DataLoader
from disent.dataset import DisentDataset
from disent.dataset.data import XYObjectData
from disent.dataset.sampling import GroundTruthPairOrigSampler
from disent.frameworks.ae import Ae
from disent.frameworks.vae import BetaVae, AdaVae
from disent.model import AutoEncoder
from disent.model.ae import DecoderConv64, EncoderConv64
from disent.dataset.transform import ToImgTensorF32
from disent.util import is_test_run  # you can ignore and remove this
from disent.metrics import metric_dci, metric_mig, metric_sap 
import torch
import matplotlib.pyplot as plt
import numpy as np
import wandb
import callback as tc

wandb.init()
# prepare the data
data = XYObjectData()
dataset = DisentDataset(data, transform=ToImgTensorF32())
dataloader = DataLoader(dataset=dataset, batch_size=128, shuffle=True)

# create the pytorch lightning system
# create the pytorch lightning system
module: pl.LightningModule = BetaVae(
    model=AutoEncoder(
        encoder=EncoderConv64(x_shape=data.x_shape, z_size=6, z_multiplier=2),
        decoder=DecoderConv64(x_shape=data.x_shape, z_size=6),
    ),
    cfg=BetaVae.cfg(optimizer='adam', optimizer_kwargs=dict(lr=1e-3), loss_reduction='mean_sum', beta=1)
)
wandb_logger = WandbLogger(project="validation.ipynb")
# train the model
trainer = pl.Trainer(logger=wandb_logger, callbacks=[tc.betaControlCallback()], max_epochs = 10, fast_dev_run=is_test_run())
trainer.fit(module, dataloader)

module.eval()

get_repr = lambda x: module.encode(x.to(module.device))

a_results = {
        **metric_dci(dataset, get_repr, num_train=10 if is_test_run() else 1000, num_test=5 if is_test_run() else 500, boost_mode='sklearn'),
        **metric_mig(dataset, get_repr, num_train=20 if is_test_run() else 2000),
        **metric_sap(dataset, get_repr),
    }
print(a_results)
# BetaVAE beta = 1, at e=5 beta = 4

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: iceburg. Use `wandb login --relogin` to force relogin


/opt/conda/lib/python3.9/site-packages/pytorch_lightning/loggers/wandb.py:351: UserWarning: There is a wandb run already in progress and newly created instances of `WandbLogger` will reuse this run. If this is not desired, call `wandb.finish()` before instantiating `WandbLogger`.
  rank_zero_warn(
GPU available: True (cuda), used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/opt/conda/lib/python3.9/site-packages/pytorch_lightning/trainer/trainer.py:1763: PossibleUserWarning: GPU available but not used. Set `accelerator` and `devices` using `Trainer(accelerator='gpu', devices=1)`.
  rank_zero_warn(
/opt/conda/lib/python3.9/site-packages/pytorch_lightning/trainer/configuration_validator.py:117: PossibleUserWarning: You defined a `validation_step` but have no `val_dataloader`. Skipping val loop.
  rank_zero_warn(

  | Name                          | Type                | Params
-------------------------------------

Training: 0it [00:00, ?it/s]

/opt/conda/lib/python3.9/site-packages/pytorch_lightning/callbacks/progress/base.py:247: UserWarning: The progress bar already tracks a metric with the name(s) 'loss' and `self.log('loss', ..., prog_bar=True)` will overwrite this value.  If this is undesired, change the name or override `get_metrics()` in the progress bar callback.
  rank_zero_warn(


trainer.current_epoch is now 5
beta was, 1
beta is now, 4


`Trainer.fit` stopped: `max_epochs=10` reached.



 Train end and beta NOW =  4
{'dci.informativeness_train': 0.9525, 'dci.informativeness_test': 0.248, 'dci.disentanglement': 0.0064043845508086296, 'dci.completeness': 0.009494685276765336, 'mig.discrete_score': 0.006008339990720148, 'sap.score': 0.007999999999999998}


In [1]:
import pytorch_lightning as pl
from pytorch_lightning.loggers.wandb import WandbLogger
from torch.utils.data import DataLoader
from disent.dataset import DisentDataset
from disent.dataset.data import XYObjectData
from disent.dataset.sampling import GroundTruthPairOrigSampler
from disent.frameworks.ae import Ae
from disent.frameworks.vae import BetaVae, AdaVae
from disent.model import AutoEncoder
from disent.model.ae import DecoderConv64, EncoderConv64
from disent.dataset.transform import ToImgTensorF32
from disent.util import is_test_run  # you can ignore and remove this
from disent.metrics import metric_dci, metric_mig, metric_sap 
import torch
import matplotlib.pyplot as plt
import numpy as np
import wandb
import callback as tc

wandb.init()
# prepare the data
data = XYObjectData()
dataset = DisentDataset(data, transform=ToImgTensorF32())
dataloader = DataLoader(dataset=dataset, batch_size=128, shuffle=True)

# create the pytorch lightning system
# create the pytorch lightning system
module: pl.LightningModule = BetaVae(
    model=AutoEncoder(
        encoder=EncoderConv64(x_shape=data.x_shape, z_size=6, z_multiplier=2),
        decoder=DecoderConv64(x_shape=data.x_shape, z_size=6),
    ),
    cfg=BetaVae.cfg(optimizer='adam', optimizer_kwargs=dict(lr=1e-3), loss_reduction='mean_sum', beta=1)
)
wandb_logger = WandbLogger(project="validation.ipynb")
# train the model
trainer = pl.Trainer(logger=wandb_logger, callbacks=[tc.betaControlCallback()], max_epochs = 10, fast_dev_run=is_test_run())
trainer.fit(module, dataloader)

module.eval()

get_repr = lambda x: module.encode(x.to(module.device))

a_results = {
        **metric_dci(dataset, get_repr, num_train=10 if is_test_run() else 1000, num_test=5 if is_test_run() else 500, boost_mode='sklearn'),
        **metric_mig(dataset, get_repr, num_train=20 if is_test_run() else 2000),
        **metric_sap(dataset, get_repr),
    }
print(a_results)
# BetaVAE beta = 1, at e=3 beta = 4

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: iceburg. Use `wandb login --relogin` to force relogin


/opt/conda/lib/python3.9/site-packages/pytorch_lightning/loggers/wandb.py:351: UserWarning: There is a wandb run already in progress and newly created instances of `WandbLogger` will reuse this run. If this is not desired, call `wandb.finish()` before instantiating `WandbLogger`.
  rank_zero_warn(
GPU available: True (cuda), used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/opt/conda/lib/python3.9/site-packages/pytorch_lightning/trainer/trainer.py:1763: PossibleUserWarning: GPU available but not used. Set `accelerator` and `devices` using `Trainer(accelerator='gpu', devices=1)`.
  rank_zero_warn(
/opt/conda/lib/python3.9/site-packages/pytorch_lightning/trainer/configuration_validator.py:117: PossibleUserWarning: You defined a `validation_step` but have no `val_dataloader`. Skipping val loop.
  rank_zero_warn(

  | Name                          | Type                | Params
-------------------------------------

Training: 0it [00:00, ?it/s]

/opt/conda/lib/python3.9/site-packages/pytorch_lightning/callbacks/progress/base.py:247: UserWarning: The progress bar already tracks a metric with the name(s) 'loss' and `self.log('loss', ..., prog_bar=True)` will overwrite this value.  If this is undesired, change the name or override `get_metrics()` in the progress bar callback.
  rank_zero_warn(


trainer.current_epoch is now 3
beta was, 1
beta is now, 4


`Trainer.fit` stopped: `max_epochs=10` reached.



 Train end and beta NOW =  4
{'dci.informativeness_train': 0.9545, 'dci.informativeness_test': 0.2595, 'dci.disentanglement': 0.018864963501065223, 'dci.completeness': 0.026612815216192937, 'mig.discrete_score': 0.030318262422222825, 'sap.score': 0.007100000000000009}


In [1]:
import pytorch_lightning as pl
from pytorch_lightning.loggers.wandb import WandbLogger
from torch.utils.data import DataLoader
from disent.dataset import DisentDataset
from disent.dataset.data import XYObjectData
from disent.dataset.sampling import GroundTruthPairOrigSampler
from disent.frameworks.ae import Ae
from disent.frameworks.vae import BetaVae, AdaVae
from disent.model import AutoEncoder
from disent.model.ae import DecoderConv64, EncoderConv64
from disent.dataset.transform import ToImgTensorF32
from disent.util import is_test_run  # you can ignore and remove this
from disent.metrics import metric_dci, metric_mig, metric_sap 
import torch
import matplotlib.pyplot as plt
import numpy as np
import wandb
import callback as tc

wandb.init()
# prepare the data
data = XYObjectData()
dataset = DisentDataset(data, transform=ToImgTensorF32())
dataloader = DataLoader(dataset=dataset, batch_size=128, shuffle=True)

# create the pytorch lightning system
# create the pytorch lightning system
module: pl.LightningModule = BetaVae(
    model=AutoEncoder(
        encoder=EncoderConv64(x_shape=data.x_shape, z_size=6, z_multiplier=2),
        decoder=DecoderConv64(x_shape=data.x_shape, z_size=6),
    ),
    cfg=BetaVae.cfg(optimizer='adam', optimizer_kwargs=dict(lr=1e-3), loss_reduction='mean_sum', beta=1)
)
wandb_logger = WandbLogger(project="validation.ipynb")
# train the model
trainer = pl.Trainer(logger=wandb_logger, callbacks=[tc.betaControlCallback()], max_epochs = 10, fast_dev_run=is_test_run())
trainer.fit(module, dataloader)

module.eval()

get_repr = lambda x: module.encode(x.to(module.device))

a_results = {
        **metric_dci(dataset, get_repr, num_train=10 if is_test_run() else 1000, num_test=5 if is_test_run() else 500, boost_mode='sklearn'),
        **metric_mig(dataset, get_repr, num_train=20 if is_test_run() else 2000),
        **metric_sap(dataset, get_repr),
    }
print(a_results)
# BetaVAE beta = 1, at e=7 beta = 4

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: iceburg. Use `wandb login --relogin` to force relogin


/opt/conda/lib/python3.9/site-packages/pytorch_lightning/loggers/wandb.py:351: UserWarning: There is a wandb run already in progress and newly created instances of `WandbLogger` will reuse this run. If this is not desired, call `wandb.finish()` before instantiating `WandbLogger`.
  rank_zero_warn(
GPU available: True (cuda), used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/opt/conda/lib/python3.9/site-packages/pytorch_lightning/trainer/trainer.py:1763: PossibleUserWarning: GPU available but not used. Set `accelerator` and `devices` using `Trainer(accelerator='gpu', devices=1)`.
  rank_zero_warn(
/opt/conda/lib/python3.9/site-packages/pytorch_lightning/trainer/configuration_validator.py:117: PossibleUserWarning: You defined a `validation_step` but have no `val_dataloader`. Skipping val loop.
  rank_zero_warn(

  | Name                          | Type                | Params
-------------------------------------

Training: 0it [00:00, ?it/s]

/opt/conda/lib/python3.9/site-packages/pytorch_lightning/callbacks/progress/base.py:247: UserWarning: The progress bar already tracks a metric with the name(s) 'loss' and `self.log('loss', ..., prog_bar=True)` will overwrite this value.  If this is undesired, change the name or override `get_metrics()` in the progress bar callback.
  rank_zero_warn(


trainer.current_epoch is now 7
beta was, 1
beta is now, 4


`Trainer.fit` stopped: `max_epochs=10` reached.



 Train end and beta NOW =  4
{'dci.informativeness_train': 0.95775, 'dci.informativeness_test': 0.2405, 'dci.disentanglement': 0.016056858295094255, 'dci.completeness': 0.016623338785882374, 'mig.discrete_score': 0.03253441363947755, 'sap.score': 0.005849999999999999}


In [1]:
import pytorch_lightning as pl
from pytorch_lightning.loggers.wandb import WandbLogger
from torch.utils.data import DataLoader
from disent.dataset import DisentDataset
from disent.dataset.data import XYObjectData
from disent.dataset.sampling import GroundTruthPairOrigSampler
from disent.frameworks.ae import Ae
from disent.frameworks.vae import BetaVae, AdaVae
from disent.model import AutoEncoder
from disent.model.ae import DecoderConv64, EncoderConv64
from disent.dataset.transform import ToImgTensorF32
from disent.util import is_test_run  # you can ignore and remove this
from disent.metrics import metric_dci, metric_mig, metric_sap 
import torch
import matplotlib.pyplot as plt
import numpy as np
import wandb
import callback as tc

wandb.init()
# prepare the data
data = XYObjectData()
dataset = DisentDataset(data, transform=ToImgTensorF32())
dataloader = DataLoader(dataset=dataset, batch_size=128, shuffle=True)

# create the pytorch lightning system
# create the pytorch lightning system
module: pl.LightningModule = BetaVae(
    model=AutoEncoder(
        encoder=EncoderConv64(x_shape=data.x_shape, z_size=6, z_multiplier=2),
        decoder=DecoderConv64(x_shape=data.x_shape, z_size=6),
    ),
    cfg=BetaVae.cfg(optimizer='adam', optimizer_kwargs=dict(lr=1e-3), loss_reduction='mean_sum', beta=1)
)
wandb_logger = WandbLogger(project="validation.ipynb")
# train the model
trainer = pl.Trainer(logger=wandb_logger, callbacks=[tc.betaControlCallback()], max_epochs = 10, fast_dev_run=is_test_run())
trainer.fit(module, dataloader)

module.eval()

get_repr = lambda x: module.encode(x.to(module.device))

a_results = {
        **metric_dci(dataset, get_repr, num_train=10 if is_test_run() else 1000, num_test=5 if is_test_run() else 500, boost_mode='sklearn'),
        **metric_mig(dataset, get_repr, num_train=20 if is_test_run() else 2000),
        **metric_sap(dataset, get_repr),
    }
print(a_results)
# BetaVAE beta = 1, at e=1 beta = 4

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: iceburg. Use `wandb login --relogin` to force relogin


/opt/conda/lib/python3.9/site-packages/pytorch_lightning/loggers/wandb.py:351: UserWarning: There is a wandb run already in progress and newly created instances of `WandbLogger` will reuse this run. If this is not desired, call `wandb.finish()` before instantiating `WandbLogger`.
  rank_zero_warn(
GPU available: True (cuda), used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/opt/conda/lib/python3.9/site-packages/pytorch_lightning/trainer/trainer.py:1763: PossibleUserWarning: GPU available but not used. Set `accelerator` and `devices` using `Trainer(accelerator='gpu', devices=1)`.
  rank_zero_warn(
/opt/conda/lib/python3.9/site-packages/pytorch_lightning/trainer/configuration_validator.py:117: PossibleUserWarning: You defined a `validation_step` but have no `val_dataloader`. Skipping val loop.
  rank_zero_warn(

  | Name                          | Type                | Params
-------------------------------------

Training: 0it [00:00, ?it/s]

/opt/conda/lib/python3.9/site-packages/pytorch_lightning/callbacks/progress/base.py:247: UserWarning: The progress bar already tracks a metric with the name(s) 'loss' and `self.log('loss', ..., prog_bar=True)` will overwrite this value.  If this is undesired, change the name or override `get_metrics()` in the progress bar callback.
  rank_zero_warn(


trainer.current_epoch is now 1
beta was, 1
beta is now, 4


`Trainer.fit` stopped: `max_epochs=10` reached.



 Train end and beta NOW =  4
{'dci.informativeness_train': 0.90625, 'dci.informativeness_test': 0.22999999999999998, 'dci.disentanglement': 0.008335863746349221, 'dci.completeness': 0.00849464688215959, 'mig.discrete_score': 0.013138901224010152, 'sap.score': 0.0039000000000000163}


In [1]:
import pytorch_lightning as pl
from pytorch_lightning.loggers.wandb import WandbLogger
from torch.utils.data import DataLoader
from disent.dataset import DisentDataset
from disent.dataset.data import XYObjectData
from disent.dataset.sampling import GroundTruthPairOrigSampler
from disent.frameworks.ae import Ae
from disent.frameworks.vae import BetaVae, AdaVae
from disent.model import AutoEncoder
from disent.model.ae import DecoderConv64, EncoderConv64
from disent.dataset.transform import ToImgTensorF32
from disent.util import is_test_run  # you can ignore and remove this
from disent.metrics import metric_dci, metric_mig, metric_sap 
import torch
import matplotlib.pyplot as plt
import numpy as np
import wandb
import callback as tc

wandb.init()
# prepare the data
data = XYObjectData()
dataset = DisentDataset(data, transform=ToImgTensorF32())
dataloader = DataLoader(dataset=dataset, batch_size=128, shuffle=True)

# create the pytorch lightning system
# create the pytorch lightning system
module: pl.LightningModule = BetaVae(
    model=AutoEncoder(
        encoder=EncoderConv64(x_shape=data.x_shape, z_size=6, z_multiplier=2),
        decoder=DecoderConv64(x_shape=data.x_shape, z_size=6),
    ),
    cfg=BetaVae.cfg(optimizer='adam', optimizer_kwargs=dict(lr=1e-3), loss_reduction='mean_sum', beta=1)
)
wandb_logger = WandbLogger(project="validation.ipynb")
# train the model
trainer = pl.Trainer(logger=wandb_logger, callbacks=[tc.betaControlCallback()], max_epochs = 10, fast_dev_run=is_test_run())
trainer.fit(module, dataloader)

module.eval()

get_repr = lambda x: module.encode(x.to(module.device))

a_results = {
        **metric_dci(dataset, get_repr, num_train=10 if is_test_run() else 1000, num_test=5 if is_test_run() else 500, boost_mode='sklearn'),
        **metric_mig(dataset, get_repr, num_train=20 if is_test_run() else 2000),
        **metric_sap(dataset, get_repr),
    }
print(a_results)
# BetaVAE beta = 1, at e=9 beta = 4

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: iceburg. Use `wandb login --relogin` to force relogin


/opt/conda/lib/python3.9/site-packages/pytorch_lightning/loggers/wandb.py:351: UserWarning: There is a wandb run already in progress and newly created instances of `WandbLogger` will reuse this run. If this is not desired, call `wandb.finish()` before instantiating `WandbLogger`.
  rank_zero_warn(
GPU available: True (cuda), used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/opt/conda/lib/python3.9/site-packages/pytorch_lightning/trainer/trainer.py:1763: PossibleUserWarning: GPU available but not used. Set `accelerator` and `devices` using `Trainer(accelerator='gpu', devices=1)`.
  rank_zero_warn(
/opt/conda/lib/python3.9/site-packages/pytorch_lightning/trainer/configuration_validator.py:117: PossibleUserWarning: You defined a `validation_step` but have no `val_dataloader`. Skipping val loop.
  rank_zero_warn(

  | Name                          | Type                | Params
-------------------------------------

Training: 0it [00:00, ?it/s]

/opt/conda/lib/python3.9/site-packages/pytorch_lightning/callbacks/progress/base.py:247: UserWarning: The progress bar already tracks a metric with the name(s) 'loss' and `self.log('loss', ..., prog_bar=True)` will overwrite this value.  If this is undesired, change the name or override `get_metrics()` in the progress bar callback.
  rank_zero_warn(


trainer.current_epoch is now 9
beta was, 1
beta is now, 4


`Trainer.fit` stopped: `max_epochs=10` reached.



 Train end and beta NOW =  4
{'dci.informativeness_train': 0.96625, 'dci.informativeness_test': 0.258, 'dci.disentanglement': 0.034566779414661006, 'dci.completeness': 0.036886596314951836, 'mig.discrete_score': 0.04563616416504548, 'sap.score': 0.014199999999999999}


In [1]:
import pytorch_lightning as pl
from pytorch_lightning.loggers.wandb import WandbLogger
from torch.utils.data import DataLoader
from disent.dataset import DisentDataset
from disent.dataset.data import XYObjectData
from disent.dataset.sampling import GroundTruthPairOrigSampler
from disent.frameworks.ae import Ae
from disent.frameworks.vae import BetaVae, AdaVae
from disent.model import AutoEncoder
from disent.model.ae import DecoderConv64, EncoderConv64
from disent.dataset.transform import ToImgTensorF32
from disent.util import is_test_run  # you can ignore and remove this
from disent.metrics import metric_dci, metric_mig, metric_sap 
import torch
import matplotlib.pyplot as plt
import numpy as np
import wandb
import callback as tc

wandb.init()
# prepare the data
data = XYObjectData()
dataset = DisentDataset(data, transform=ToImgTensorF32())
dataloader = DataLoader(dataset=dataset, batch_size=128, shuffle=True)

# create the pytorch lightning system
# create the pytorch lightning system
module: pl.LightningModule = BetaVae(
    model=AutoEncoder(
        encoder=EncoderConv64(x_shape=data.x_shape, z_size=6, z_multiplier=2),
        decoder=DecoderConv64(x_shape=data.x_shape, z_size=6),
    ),
    cfg=BetaVae.cfg(optimizer='adam', optimizer_kwargs=dict(lr=1e-3), loss_reduction='mean_sum', beta=4)
)
wandb_logger = WandbLogger(project="validation.ipynb")
# train the model
trainer = pl.Trainer(logger=wandb_logger, callbacks=[tc.betaControlCallback()], max_epochs = 10, fast_dev_run=is_test_run())
trainer.fit(module, dataloader)

module.eval()

get_repr = lambda x: module.encode(x.to(module.device))

a_results = {
        **metric_dci(dataset, get_repr, num_train=10 if is_test_run() else 1000, num_test=5 if is_test_run() else 500, boost_mode='sklearn'),
        **metric_mig(dataset, get_repr, num_train=20 if is_test_run() else 2000),
        **metric_sap(dataset, get_repr),
    }
print(a_results)
# BetaVAE beta = 4, at e=1 beta = 1

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: iceburg. Use `wandb login --relogin` to force relogin


/opt/conda/lib/python3.9/site-packages/pytorch_lightning/loggers/wandb.py:351: UserWarning: There is a wandb run already in progress and newly created instances of `WandbLogger` will reuse this run. If this is not desired, call `wandb.finish()` before instantiating `WandbLogger`.
  rank_zero_warn(
GPU available: True (cuda), used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/opt/conda/lib/python3.9/site-packages/pytorch_lightning/trainer/trainer.py:1763: PossibleUserWarning: GPU available but not used. Set `accelerator` and `devices` using `Trainer(accelerator='gpu', devices=1)`.
  rank_zero_warn(
/opt/conda/lib/python3.9/site-packages/pytorch_lightning/trainer/configuration_validator.py:117: PossibleUserWarning: You defined a `validation_step` but have no `val_dataloader`. Skipping val loop.
  rank_zero_warn(

  | Name                          | Type                | Params
-------------------------------------

Training: 0it [00:00, ?it/s]

/opt/conda/lib/python3.9/site-packages/pytorch_lightning/callbacks/progress/base.py:247: UserWarning: The progress bar already tracks a metric with the name(s) 'loss' and `self.log('loss', ..., prog_bar=True)` will overwrite this value.  If this is undesired, change the name or override `get_metrics()` in the progress bar callback.
  rank_zero_warn(


trainer.current_epoch is now 1
beta was, 4
beta is now, 1


`Trainer.fit` stopped: `max_epochs=10` reached.



 Train end and beta NOW =  1
{'dci.informativeness_train': 0.9572499999999999, 'dci.informativeness_test': 0.329, 'dci.disentanglement': 0.044190248253106336, 'dci.completeness': 0.046239238086364556, 'mig.discrete_score': 0.06821623848035582, 'sap.score': 0.013300000000000003}


In [1]:
import pytorch_lightning as pl
from pytorch_lightning.loggers.wandb import WandbLogger
from torch.utils.data import DataLoader
from disent.dataset import DisentDataset
from disent.dataset.data import XYObjectData
from disent.dataset.sampling import GroundTruthPairOrigSampler
from disent.frameworks.ae import Ae
from disent.frameworks.vae import BetaVae, AdaVae
from disent.model import AutoEncoder
from disent.model.ae import DecoderConv64, EncoderConv64
from disent.dataset.transform import ToImgTensorF32
from disent.util import is_test_run  # you can ignore and remove this
from disent.metrics import metric_dci, metric_mig, metric_sap 
import torch
import matplotlib.pyplot as plt
import numpy as np
import wandb
import callback as tc

wandb.init()
# prepare the data
data = XYObjectData()
dataset = DisentDataset(data, transform=ToImgTensorF32())
dataloader = DataLoader(dataset=dataset, batch_size=128, shuffle=True)

# create the pytorch lightning system
# create the pytorch lightning system
module: pl.LightningModule = BetaVae(
    model=AutoEncoder(
        encoder=EncoderConv64(x_shape=data.x_shape, z_size=6, z_multiplier=2),
        decoder=DecoderConv64(x_shape=data.x_shape, z_size=6),
    ),
    cfg=BetaVae.cfg(optimizer='adam', optimizer_kwargs=dict(lr=1e-3), loss_reduction='mean_sum', beta=4)
)
wandb_logger = WandbLogger(project="validation.ipynb")
# train the model
trainer = pl.Trainer(logger=wandb_logger, callbacks=[tc.betaControlCallback()], max_epochs = 10, fast_dev_run=is_test_run())
trainer.fit(module, dataloader)

module.eval()

get_repr = lambda x: module.encode(x.to(module.device))

a_results = {
        **metric_dci(dataset, get_repr, num_train=10 if is_test_run() else 1000, num_test=5 if is_test_run() else 500, boost_mode='sklearn'),
        **metric_mig(dataset, get_repr, num_train=20 if is_test_run() else 2000),
        **metric_sap(dataset, get_repr),
    }
print(a_results)
# BetaVAE beta = 4, at e=3 beta = 1

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: iceburg. Use `wandb login --relogin` to force relogin


/opt/conda/lib/python3.9/site-packages/pytorch_lightning/loggers/wandb.py:351: UserWarning: There is a wandb run already in progress and newly created instances of `WandbLogger` will reuse this run. If this is not desired, call `wandb.finish()` before instantiating `WandbLogger`.
  rank_zero_warn(
GPU available: True (cuda), used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/opt/conda/lib/python3.9/site-packages/pytorch_lightning/trainer/trainer.py:1763: PossibleUserWarning: GPU available but not used. Set `accelerator` and `devices` using `Trainer(accelerator='gpu', devices=1)`.
  rank_zero_warn(
/opt/conda/lib/python3.9/site-packages/pytorch_lightning/trainer/configuration_validator.py:117: PossibleUserWarning: You defined a `validation_step` but have no `val_dataloader`. Skipping val loop.
  rank_zero_warn(

  | Name                          | Type                | Params
-------------------------------------

Training: 0it [00:00, ?it/s]

/opt/conda/lib/python3.9/site-packages/pytorch_lightning/callbacks/progress/base.py:247: UserWarning: The progress bar already tracks a metric with the name(s) 'loss' and `self.log('loss', ..., prog_bar=True)` will overwrite this value.  If this is undesired, change the name or override `get_metrics()` in the progress bar callback.
  rank_zero_warn(


trainer.current_epoch is now 3
beta was, 4
beta is now, 1


`Trainer.fit` stopped: `max_epochs=10` reached.



 Train end and beta NOW =  1
{'dci.informativeness_train': 0.9632499999999999, 'dci.informativeness_test': 0.28200000000000003, 'dci.disentanglement': 0.019761623350170232, 'dci.completeness': 0.02609588040215255, 'mig.discrete_score': 0.013057781691484492, 'sap.score': 0.010449999999999997}


In [1]:
import pytorch_lightning as pl
from pytorch_lightning.loggers.wandb import WandbLogger
from torch.utils.data import DataLoader
from disent.dataset import DisentDataset
from disent.dataset.data import XYObjectData
from disent.dataset.sampling import GroundTruthPairOrigSampler
from disent.frameworks.ae import Ae
from disent.frameworks.vae import BetaVae, AdaVae
from disent.model import AutoEncoder
from disent.model.ae import DecoderConv64, EncoderConv64
from disent.dataset.transform import ToImgTensorF32
from disent.util import is_test_run  # you can ignore and remove this
from disent.metrics import metric_dci, metric_mig, metric_sap 
import torch
import matplotlib.pyplot as plt
import numpy as np
import wandb
import callback as tc

wandb.init()
# prepare the data
data = XYObjectData()
dataset = DisentDataset(data, transform=ToImgTensorF32())
dataloader = DataLoader(dataset=dataset, batch_size=128, shuffle=True)

# create the pytorch lightning system
# create the pytorch lightning system
module: pl.LightningModule = BetaVae(
    model=AutoEncoder(
        encoder=EncoderConv64(x_shape=data.x_shape, z_size=6, z_multiplier=2),
        decoder=DecoderConv64(x_shape=data.x_shape, z_size=6),
    ),
    cfg=BetaVae.cfg(optimizer='adam', optimizer_kwargs=dict(lr=1e-3), loss_reduction='mean_sum', beta=4)
)
wandb_logger = WandbLogger(project="validation.ipynb")
# train the model
trainer = pl.Trainer(logger=wandb_logger, callbacks=[tc.betaControlCallback()], max_epochs = 10, fast_dev_run=is_test_run())
trainer.fit(module, dataloader)

module.eval()

get_repr = lambda x: module.encode(x.to(module.device))

a_results = {
        **metric_dci(dataset, get_repr, num_train=10 if is_test_run() else 1000, num_test=5 if is_test_run() else 500, boost_mode='sklearn'),
        **metric_mig(dataset, get_repr, num_train=20 if is_test_run() else 2000),
        **metric_sap(dataset, get_repr),
    }
print(a_results)
# BetaVAE beta = 4, at e=5 beta = 1

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: iceburg. Use `wandb login --relogin` to force relogin


/opt/conda/lib/python3.9/site-packages/pytorch_lightning/loggers/wandb.py:351: UserWarning: There is a wandb run already in progress and newly created instances of `WandbLogger` will reuse this run. If this is not desired, call `wandb.finish()` before instantiating `WandbLogger`.
  rank_zero_warn(
GPU available: True (cuda), used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/opt/conda/lib/python3.9/site-packages/pytorch_lightning/trainer/trainer.py:1763: PossibleUserWarning: GPU available but not used. Set `accelerator` and `devices` using `Trainer(accelerator='gpu', devices=1)`.
  rank_zero_warn(
/opt/conda/lib/python3.9/site-packages/pytorch_lightning/trainer/configuration_validator.py:117: PossibleUserWarning: You defined a `validation_step` but have no `val_dataloader`. Skipping val loop.
  rank_zero_warn(

  | Name                          | Type                | Params
-------------------------------------

Training: 0it [00:00, ?it/s]

/opt/conda/lib/python3.9/site-packages/pytorch_lightning/callbacks/progress/base.py:247: UserWarning: The progress bar already tracks a metric with the name(s) 'loss' and `self.log('loss', ..., prog_bar=True)` will overwrite this value.  If this is undesired, change the name or override `get_metrics()` in the progress bar callback.
  rank_zero_warn(


trainer.current_epoch is now 5
beta was, 4
beta is now, 1


`Trainer.fit` stopped: `max_epochs=10` reached.



 Train end and beta NOW =  1
{'dci.informativeness_train': 0.959, 'dci.informativeness_test': 0.2955, 'dci.disentanglement': 0.02402374064629238, 'dci.completeness': 0.033005766301051676, 'mig.discrete_score': 0.02367765810076227, 'sap.score': 0.028550000000000002}


In [1]:
import pytorch_lightning as pl
from pytorch_lightning.loggers.wandb import WandbLogger
from torch.utils.data import DataLoader
from disent.dataset import DisentDataset
from disent.dataset.data import XYObjectData
from disent.dataset.sampling import GroundTruthPairOrigSampler
from disent.frameworks.ae import Ae
from disent.frameworks.vae import BetaVae, AdaVae
from disent.model import AutoEncoder
from disent.model.ae import DecoderConv64, EncoderConv64
from disent.dataset.transform import ToImgTensorF32
from disent.util import is_test_run  # you can ignore and remove this
from disent.metrics import metric_dci, metric_mig, metric_sap 
import torch
import matplotlib.pyplot as plt
import numpy as np
import wandb
import callback as tc

wandb.init()
# prepare the data
data = XYObjectData()
dataset = DisentDataset(data, transform=ToImgTensorF32())
dataloader = DataLoader(dataset=dataset, batch_size=128, shuffle=True)

# create the pytorch lightning system
# create the pytorch lightning system
module: pl.LightningModule = BetaVae(
    model=AutoEncoder(
        encoder=EncoderConv64(x_shape=data.x_shape, z_size=6, z_multiplier=2),
        decoder=DecoderConv64(x_shape=data.x_shape, z_size=6),
    ),
    cfg=BetaVae.cfg(optimizer='adam', optimizer_kwargs=dict(lr=1e-3), loss_reduction='mean_sum', beta=4)
)
wandb_logger = WandbLogger(project="validation.ipynb")
# train the model
trainer = pl.Trainer(logger=wandb_logger, callbacks=[tc.betaControlCallback()], max_epochs = 10, fast_dev_run=is_test_run())
trainer.fit(module, dataloader)

module.eval()

get_repr = lambda x: module.encode(x.to(module.device))

a_results = {
        **metric_dci(dataset, get_repr, num_train=10 if is_test_run() else 1000, num_test=5 if is_test_run() else 500, boost_mode='sklearn'),
        **metric_mig(dataset, get_repr, num_train=20 if is_test_run() else 2000),
        **metric_sap(dataset, get_repr),
    }
print(a_results)
# BetaVAE beta = 4, at e=7 beta = 1

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: iceburg. Use `wandb login --relogin` to force relogin


/opt/conda/lib/python3.9/site-packages/pytorch_lightning/loggers/wandb.py:351: UserWarning: There is a wandb run already in progress and newly created instances of `WandbLogger` will reuse this run. If this is not desired, call `wandb.finish()` before instantiating `WandbLogger`.
  rank_zero_warn(
GPU available: True (cuda), used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/opt/conda/lib/python3.9/site-packages/pytorch_lightning/trainer/trainer.py:1763: PossibleUserWarning: GPU available but not used. Set `accelerator` and `devices` using `Trainer(accelerator='gpu', devices=1)`.
  rank_zero_warn(
/opt/conda/lib/python3.9/site-packages/pytorch_lightning/trainer/configuration_validator.py:117: PossibleUserWarning: You defined a `validation_step` but have no `val_dataloader`. Skipping val loop.
  rank_zero_warn(

  | Name                          | Type                | Params
-------------------------------------

Training: 0it [00:00, ?it/s]

/opt/conda/lib/python3.9/site-packages/pytorch_lightning/callbacks/progress/base.py:247: UserWarning: The progress bar already tracks a metric with the name(s) 'loss' and `self.log('loss', ..., prog_bar=True)` will overwrite this value.  If this is undesired, change the name or override `get_metrics()` in the progress bar callback.
  rank_zero_warn(


trainer.current_epoch is now 7
beta was, 4
beta is now, 1


`Trainer.fit` stopped: `max_epochs=10` reached.



 Train end and beta NOW =  1
{'dci.informativeness_train': 0.9782500000000001, 'dci.informativeness_test': 0.32799999999999996, 'dci.disentanglement': 0.01821267033167238, 'dci.completeness': 0.024363697373460363, 'mig.discrete_score': 0.018021741062646687, 'sap.score': 0.014599999999999998}


In [1]:
import pytorch_lightning as pl
from pytorch_lightning.loggers.wandb import WandbLogger
from torch.utils.data import DataLoader
from disent.dataset import DisentDataset
from disent.dataset.data import XYObjectData
from disent.dataset.sampling import GroundTruthPairOrigSampler
from disent.frameworks.ae import Ae
from disent.frameworks.vae import BetaVae, AdaVae
from disent.model import AutoEncoder
from disent.model.ae import DecoderConv64, EncoderConv64
from disent.dataset.transform import ToImgTensorF32
from disent.util import is_test_run  # you can ignore and remove this
from disent.metrics import metric_dci, metric_mig, metric_sap 
import torch
import matplotlib.pyplot as plt
import numpy as np
import wandb
import callback as tc

wandb.init()
# prepare the data
data = XYObjectData()
dataset = DisentDataset(data, transform=ToImgTensorF32())
dataloader = DataLoader(dataset=dataset, batch_size=128, shuffle=True)

# create the pytorch lightning system
# create the pytorch lightning system
module: pl.LightningModule = BetaVae(
    model=AutoEncoder(
        encoder=EncoderConv64(x_shape=data.x_shape, z_size=6, z_multiplier=2),
        decoder=DecoderConv64(x_shape=data.x_shape, z_size=6),
    ),
    cfg=BetaVae.cfg(optimizer='adam', optimizer_kwargs=dict(lr=1e-3), loss_reduction='mean_sum', beta=4)
)
wandb_logger = WandbLogger(project="validation.ipynb")
# train the model
trainer = pl.Trainer(logger=wandb_logger, callbacks=[tc.betaControlCallback()], max_epochs = 10, fast_dev_run=is_test_run())
trainer.fit(module, dataloader)

module.eval()

get_repr = lambda x: module.encode(x.to(module.device))

a_results = {
        **metric_dci(dataset, get_repr, num_train=10 if is_test_run() else 1000, num_test=5 if is_test_run() else 500, boost_mode='sklearn'),
        **metric_mig(dataset, get_repr, num_train=20 if is_test_run() else 2000),
        **metric_sap(dataset, get_repr),
    }
print(a_results)
# BetaVAE beta = 4, at e=9 beta = 1

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: iceburg. Use `wandb login --relogin` to force relogin


/opt/conda/lib/python3.9/site-packages/pytorch_lightning/loggers/wandb.py:351: UserWarning: There is a wandb run already in progress and newly created instances of `WandbLogger` will reuse this run. If this is not desired, call `wandb.finish()` before instantiating `WandbLogger`.
  rank_zero_warn(
GPU available: True (cuda), used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/opt/conda/lib/python3.9/site-packages/pytorch_lightning/trainer/trainer.py:1763: PossibleUserWarning: GPU available but not used. Set `accelerator` and `devices` using `Trainer(accelerator='gpu', devices=1)`.
  rank_zero_warn(
/opt/conda/lib/python3.9/site-packages/pytorch_lightning/trainer/configuration_validator.py:117: PossibleUserWarning: You defined a `validation_step` but have no `val_dataloader`. Skipping val loop.
  rank_zero_warn(

  | Name                          | Type                | Params
-------------------------------------

Training: 0it [00:00, ?it/s]

/opt/conda/lib/python3.9/site-packages/pytorch_lightning/callbacks/progress/base.py:247: UserWarning: The progress bar already tracks a metric with the name(s) 'loss' and `self.log('loss', ..., prog_bar=True)` will overwrite this value.  If this is undesired, change the name or override `get_metrics()` in the progress bar callback.
  rank_zero_warn(


trainer.current_epoch is now 9
beta was, 4
beta is now, 1


`Trainer.fit` stopped: `max_epochs=10` reached.



 Train end and beta NOW =  1
{'dci.informativeness_train': 0.80675, 'dci.informativeness_test': 0.1915, 'dci.disentanglement': 0.00587335907786125, 'dci.completeness': 0.008619202870956855, 'mig.discrete_score': 0.005614116556037368, 'sap.score': 0.0012500000000000011}


In [1]:
import pytorch_lightning as pl
from pytorch_lightning.loggers.wandb import WandbLogger
from torch.utils.data import DataLoader
from disent.dataset import DisentDataset
from disent.dataset.data import XYObjectData
from disent.dataset.sampling import GroundTruthPairOrigSampler
from disent.frameworks.ae import Ae
from disent.frameworks.vae import BetaVae, AdaVae
from disent.model import AutoEncoder
from disent.model.ae import DecoderConv64, EncoderConv64
from disent.dataset.transform import ToImgTensorF32
from disent.util import is_test_run  # you can ignore and remove this
from disent.metrics import metric_dci, metric_mig, metric_sap 
import torch
import matplotlib.pyplot as plt
import numpy as np
import wandb
import callback as tc

wandb.init()
# prepare the data
data = XYObjectData()
dataset = DisentDataset(data, transform=ToImgTensorF32())
dataloader = DataLoader(dataset=dataset, batch_size=128, shuffle=True)

# create the pytorch lightning system
# create the pytorch lightning system
module: pl.LightningModule = BetaVae(
    model=AutoEncoder(
        encoder=EncoderConv64(x_shape=data.x_shape, z_size=6, z_multiplier=2),
        decoder=DecoderConv64(x_shape=data.x_shape, z_size=6),
    ),
    cfg=BetaVae.cfg(optimizer='adam', optimizer_kwargs=dict(lr=1e-3), loss_reduction='mean_sum', beta=2)
)
wandb_logger = WandbLogger(project="validation.ipynb")
# train the model
trainer = pl.Trainer(logger=wandb_logger, callbacks=[tc.betaControlCallback()], max_epochs = 10, fast_dev_run=is_test_run())
trainer.fit(module, dataloader)

module.eval()

get_repr = lambda x: module.encode(x.to(module.device))

a_results = {
        **metric_dci(dataset, get_repr, num_train=10 if is_test_run() else 1000, num_test=5 if is_test_run() else 500, boost_mode='sklearn'),
        **metric_mig(dataset, get_repr, num_train=20 if is_test_run() else 2000),
        **metric_sap(dataset, get_repr),
    }
print(a_results)
# BetaVAE beta = 2, at e=1 beta = 1

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: iceburg. Use `wandb login --relogin` to force relogin


/opt/conda/lib/python3.9/site-packages/pytorch_lightning/loggers/wandb.py:351: UserWarning: There is a wandb run already in progress and newly created instances of `WandbLogger` will reuse this run. If this is not desired, call `wandb.finish()` before instantiating `WandbLogger`.
  rank_zero_warn(
GPU available: True (cuda), used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/opt/conda/lib/python3.9/site-packages/pytorch_lightning/trainer/trainer.py:1763: PossibleUserWarning: GPU available but not used. Set `accelerator` and `devices` using `Trainer(accelerator='gpu', devices=1)`.
  rank_zero_warn(
/opt/conda/lib/python3.9/site-packages/pytorch_lightning/trainer/configuration_validator.py:117: PossibleUserWarning: You defined a `validation_step` but have no `val_dataloader`. Skipping val loop.
  rank_zero_warn(

  | Name                          | Type                | Params
-------------------------------------

Training: 0it [00:00, ?it/s]

/opt/conda/lib/python3.9/site-packages/pytorch_lightning/callbacks/progress/base.py:247: UserWarning: The progress bar already tracks a metric with the name(s) 'loss' and `self.log('loss', ..., prog_bar=True)` will overwrite this value.  If this is undesired, change the name or override `get_metrics()` in the progress bar callback.
  rank_zero_warn(


trainer.current_epoch is now 1
beta was, 2
beta is now, 1


`Trainer.fit` stopped: `max_epochs=10` reached.



 Train end and beta NOW =  1
{'dci.informativeness_train': 0.95975, 'dci.informativeness_test': 0.257, 'dci.disentanglement': 0.011188806756418852, 'dci.completeness': 0.02560644120213104, 'mig.discrete_score': 0.004978899937053942, 'sap.score': 0.012650000000000002}


In [1]:
import pytorch_lightning as pl
from pytorch_lightning.loggers.wandb import WandbLogger
from torch.utils.data import DataLoader
from disent.dataset import DisentDataset
from disent.dataset.data import XYObjectData
from disent.dataset.sampling import GroundTruthPairOrigSampler
from disent.frameworks.ae import Ae
from disent.frameworks.vae import BetaVae, AdaVae
from disent.model import AutoEncoder
from disent.model.ae import DecoderConv64, EncoderConv64
from disent.dataset.transform import ToImgTensorF32
from disent.util import is_test_run  # you can ignore and remove this
from disent.metrics import metric_dci, metric_mig, metric_sap 
import torch
import matplotlib.pyplot as plt
import numpy as np
import wandb
import callback as tc

wandb.init()
# prepare the data
data = XYObjectData()
dataset = DisentDataset(data, transform=ToImgTensorF32())
dataloader = DataLoader(dataset=dataset, batch_size=128, shuffle=True)

# create the pytorch lightning system
# create the pytorch lightning system
module: pl.LightningModule = BetaVae(
    model=AutoEncoder(
        encoder=EncoderConv64(x_shape=data.x_shape, z_size=6, z_multiplier=2),
        decoder=DecoderConv64(x_shape=data.x_shape, z_size=6),
    ),
    cfg=BetaVae.cfg(optimizer='adam', optimizer_kwargs=dict(lr=1e-3), loss_reduction='mean_sum', beta=2)
)
wandb_logger = WandbLogger(project="validation.ipynb")
# train the model
trainer = pl.Trainer(logger=wandb_logger, callbacks=[tc.betaControlCallback()], max_epochs = 10, fast_dev_run=is_test_run())
trainer.fit(module, dataloader)

module.eval()

get_repr = lambda x: module.encode(x.to(module.device))

a_results = {
        **metric_dci(dataset, get_repr, num_train=10 if is_test_run() else 1000, num_test=5 if is_test_run() else 500, boost_mode='sklearn'),
        **metric_mig(dataset, get_repr, num_train=20 if is_test_run() else 2000),
        **metric_sap(dataset, get_repr),
    }
print(a_results)
# BetaVAE beta = 2, at e=3 beta = 1

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: iceburg. Use `wandb login --relogin` to force relogin


/opt/conda/lib/python3.9/site-packages/pytorch_lightning/loggers/wandb.py:351: UserWarning: There is a wandb run already in progress and newly created instances of `WandbLogger` will reuse this run. If this is not desired, call `wandb.finish()` before instantiating `WandbLogger`.
  rank_zero_warn(
GPU available: True (cuda), used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/opt/conda/lib/python3.9/site-packages/pytorch_lightning/trainer/trainer.py:1763: PossibleUserWarning: GPU available but not used. Set `accelerator` and `devices` using `Trainer(accelerator='gpu', devices=1)`.
  rank_zero_warn(
/opt/conda/lib/python3.9/site-packages/pytorch_lightning/trainer/configuration_validator.py:117: PossibleUserWarning: You defined a `validation_step` but have no `val_dataloader`. Skipping val loop.
  rank_zero_warn(

  | Name                          | Type                | Params
-------------------------------------

Training: 0it [00:00, ?it/s]

/opt/conda/lib/python3.9/site-packages/pytorch_lightning/callbacks/progress/base.py:247: UserWarning: The progress bar already tracks a metric with the name(s) 'loss' and `self.log('loss', ..., prog_bar=True)` will overwrite this value.  If this is undesired, change the name or override `get_metrics()` in the progress bar callback.
  rank_zero_warn(


trainer.current_epoch is now 3
beta was, 2
beta is now, 1


`Trainer.fit` stopped: `max_epochs=10` reached.



 Train end and beta NOW =  1
{'dci.informativeness_train': 0.96475, 'dci.informativeness_test': 0.27149999999999996, 'dci.disentanglement': 0.01967200421929615, 'dci.completeness': 0.029848008727565517, 'mig.discrete_score': 0.030622950902403256, 'sap.score': 0.007550000000000001}


In [1]:
import pytorch_lightning as pl
from pytorch_lightning.loggers.wandb import WandbLogger
from torch.utils.data import DataLoader
from disent.dataset import DisentDataset
from disent.dataset.data import XYObjectData
from disent.dataset.sampling import GroundTruthPairOrigSampler
from disent.frameworks.ae import Ae
from disent.frameworks.vae import BetaVae, AdaVae
from disent.model import AutoEncoder
from disent.model.ae import DecoderConv64, EncoderConv64
from disent.dataset.transform import ToImgTensorF32
from disent.util import is_test_run  # you can ignore and remove this
from disent.metrics import metric_dci, metric_mig, metric_sap 
import torch
import matplotlib.pyplot as plt
import numpy as np
import wandb
import callback as tc

wandb.init()
# prepare the data
data = XYObjectData()
dataset = DisentDataset(data, transform=ToImgTensorF32())
dataloader = DataLoader(dataset=dataset, batch_size=128, shuffle=True)

# create the pytorch lightning system
# create the pytorch lightning system
module: pl.LightningModule = BetaVae(
    model=AutoEncoder(
        encoder=EncoderConv64(x_shape=data.x_shape, z_size=6, z_multiplier=2),
        decoder=DecoderConv64(x_shape=data.x_shape, z_size=6),
    ),
    cfg=BetaVae.cfg(optimizer='adam', optimizer_kwargs=dict(lr=1e-3), loss_reduction='mean_sum', beta=2)
)
wandb_logger = WandbLogger(project="validation.ipynb")
# train the model
trainer = pl.Trainer(logger=wandb_logger, callbacks=[tc.betaControlCallback()], max_epochs = 10, fast_dev_run=is_test_run())
trainer.fit(module, dataloader)

module.eval()

get_repr = lambda x: module.encode(x.to(module.device))

a_results = {
        **metric_dci(dataset, get_repr, num_train=10 if is_test_run() else 1000, num_test=5 if is_test_run() else 500, boost_mode='sklearn'),
        **metric_mig(dataset, get_repr, num_train=20 if is_test_run() else 2000),
        **metric_sap(dataset, get_repr),
    }
print(a_results)
# BetaVAE beta = 2, at e=5 beta = 1

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: iceburg. Use `wandb login --relogin` to force relogin


/opt/conda/lib/python3.9/site-packages/pytorch_lightning/loggers/wandb.py:351: UserWarning: There is a wandb run already in progress and newly created instances of `WandbLogger` will reuse this run. If this is not desired, call `wandb.finish()` before instantiating `WandbLogger`.
  rank_zero_warn(
GPU available: True (cuda), used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/opt/conda/lib/python3.9/site-packages/pytorch_lightning/trainer/trainer.py:1763: PossibleUserWarning: GPU available but not used. Set `accelerator` and `devices` using `Trainer(accelerator='gpu', devices=1)`.
  rank_zero_warn(
/opt/conda/lib/python3.9/site-packages/pytorch_lightning/trainer/configuration_validator.py:117: PossibleUserWarning: You defined a `validation_step` but have no `val_dataloader`. Skipping val loop.
  rank_zero_warn(

  | Name                          | Type                | Params
-------------------------------------

Training: 0it [00:00, ?it/s]

/opt/conda/lib/python3.9/site-packages/pytorch_lightning/callbacks/progress/base.py:247: UserWarning: The progress bar already tracks a metric with the name(s) 'loss' and `self.log('loss', ..., prog_bar=True)` will overwrite this value.  If this is undesired, change the name or override `get_metrics()` in the progress bar callback.
  rank_zero_warn(


trainer.current_epoch is now 5
beta was, 2
beta is now, 1


`Trainer.fit` stopped: `max_epochs=10` reached.



 Train end and beta NOW =  1
{'dci.informativeness_train': 0.96625, 'dci.informativeness_test': 0.302, 'dci.disentanglement': 0.013626891196798432, 'dci.completeness': 0.016013949347044504, 'mig.discrete_score': 0.007126057124956649, 'sap.score': 0.006499999999999999}


In [1]:
import pytorch_lightning as pl
from pytorch_lightning.loggers.wandb import WandbLogger
from torch.utils.data import DataLoader
from disent.dataset import DisentDataset
from disent.dataset.data import XYObjectData
from disent.dataset.sampling import GroundTruthPairOrigSampler
from disent.frameworks.ae import Ae
from disent.frameworks.vae import BetaVae, AdaVae
from disent.model import AutoEncoder
from disent.model.ae import DecoderConv64, EncoderConv64
from disent.dataset.transform import ToImgTensorF32
from disent.util import is_test_run  # you can ignore and remove this
from disent.metrics import metric_dci, metric_mig, metric_sap 
import torch
import matplotlib.pyplot as plt
import numpy as np
import wandb
import callback as tc

wandb.init()
# prepare the data
data = XYObjectData()
dataset = DisentDataset(data, transform=ToImgTensorF32())
dataloader = DataLoader(dataset=dataset, batch_size=128, shuffle=True)

# create the pytorch lightning system
# create the pytorch lightning system
module: pl.LightningModule = BetaVae(
    model=AutoEncoder(
        encoder=EncoderConv64(x_shape=data.x_shape, z_size=6, z_multiplier=2),
        decoder=DecoderConv64(x_shape=data.x_shape, z_size=6),
    ),
    cfg=BetaVae.cfg(optimizer='adam', optimizer_kwargs=dict(lr=1e-3), loss_reduction='mean_sum', beta=2)
)
wandb_logger = WandbLogger(project="validation.ipynb")
# train the model
trainer = pl.Trainer(logger=wandb_logger, callbacks=[tc.betaControlCallback()], max_epochs = 10, fast_dev_run=is_test_run())
trainer.fit(module, dataloader)

module.eval()

get_repr = lambda x: module.encode(x.to(module.device))

a_results = {
        **metric_dci(dataset, get_repr, num_train=10 if is_test_run() else 1000, num_test=5 if is_test_run() else 500, boost_mode='sklearn'),
        **metric_mig(dataset, get_repr, num_train=20 if is_test_run() else 2000),
        **metric_sap(dataset, get_repr),
    }
print(a_results)
# BetaVAE beta = 2, at e=7 beta = 1

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: iceburg. Use `wandb login --relogin` to force relogin


/opt/conda/lib/python3.9/site-packages/pytorch_lightning/loggers/wandb.py:351: UserWarning: There is a wandb run already in progress and newly created instances of `WandbLogger` will reuse this run. If this is not desired, call `wandb.finish()` before instantiating `WandbLogger`.
  rank_zero_warn(
GPU available: True (cuda), used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/opt/conda/lib/python3.9/site-packages/pytorch_lightning/trainer/trainer.py:1763: PossibleUserWarning: GPU available but not used. Set `accelerator` and `devices` using `Trainer(accelerator='gpu', devices=1)`.
  rank_zero_warn(
/opt/conda/lib/python3.9/site-packages/pytorch_lightning/trainer/configuration_validator.py:117: PossibleUserWarning: You defined a `validation_step` but have no `val_dataloader`. Skipping val loop.
  rank_zero_warn(

  | Name                          | Type                | Params
-------------------------------------

Training: 0it [00:00, ?it/s]

/opt/conda/lib/python3.9/site-packages/pytorch_lightning/callbacks/progress/base.py:247: UserWarning: The progress bar already tracks a metric with the name(s) 'loss' and `self.log('loss', ..., prog_bar=True)` will overwrite this value.  If this is undesired, change the name or override `get_metrics()` in the progress bar callback.
  rank_zero_warn(


trainer.current_epoch is now 7
beta was, 2
beta is now, 1


`Trainer.fit` stopped: `max_epochs=10` reached.



 Train end and beta NOW =  1
{'dci.informativeness_train': 0.956, 'dci.informativeness_test': 0.327, 'dci.disentanglement': 0.04470688129698834, 'dci.completeness': 0.04341613300730843, 'mig.discrete_score': 0.05734274339976101, 'sap.score': 0.012950000000000005}


In [1]:
import pytorch_lightning as pl
from pytorch_lightning.loggers.wandb import WandbLogger
from torch.utils.data import DataLoader
from disent.dataset import DisentDataset
from disent.dataset.data import XYObjectData
from disent.dataset.sampling import GroundTruthPairOrigSampler
from disent.frameworks.ae import Ae
from disent.frameworks.vae import BetaVae, AdaVae
from disent.model import AutoEncoder
from disent.model.ae import DecoderConv64, EncoderConv64
from disent.dataset.transform import ToImgTensorF32
from disent.util import is_test_run  # you can ignore and remove this
from disent.metrics import metric_dci, metric_mig, metric_sap 
import torch
import matplotlib.pyplot as plt
import numpy as np
import wandb
import callback as tc

wandb.init()
# prepare the data
data = XYObjectData()
dataset = DisentDataset(data, transform=ToImgTensorF32())
dataloader = DataLoader(dataset=dataset, batch_size=128, shuffle=True)

# create the pytorch lightning system
# create the pytorch lightning system
module: pl.LightningModule = BetaVae(
    model=AutoEncoder(
        encoder=EncoderConv64(x_shape=data.x_shape, z_size=6, z_multiplier=2),
        decoder=DecoderConv64(x_shape=data.x_shape, z_size=6),
    ),
    cfg=BetaVae.cfg(optimizer='adam', optimizer_kwargs=dict(lr=1e-3), loss_reduction='mean_sum', beta=2)
)
wandb_logger = WandbLogger(project="validation.ipynb")
# train the model
trainer = pl.Trainer(logger=wandb_logger, callbacks=[tc.betaControlCallback()], max_epochs = 10, fast_dev_run=is_test_run())
trainer.fit(module, dataloader)

module.eval()

get_repr = lambda x: module.encode(x.to(module.device))

a_results = {
        **metric_dci(dataset, get_repr, num_train=10 if is_test_run() else 1000, num_test=5 if is_test_run() else 500, boost_mode='sklearn'),
        **metric_mig(dataset, get_repr, num_train=20 if is_test_run() else 2000),
        **metric_sap(dataset, get_repr),
    }
print(a_results)
# BetaVAE beta = 2, at e=9 beta = 1

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: iceburg. Use `wandb login --relogin` to force relogin


/opt/conda/lib/python3.9/site-packages/pytorch_lightning/loggers/wandb.py:351: UserWarning: There is a wandb run already in progress and newly created instances of `WandbLogger` will reuse this run. If this is not desired, call `wandb.finish()` before instantiating `WandbLogger`.
  rank_zero_warn(
GPU available: True (cuda), used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/opt/conda/lib/python3.9/site-packages/pytorch_lightning/trainer/trainer.py:1763: PossibleUserWarning: GPU available but not used. Set `accelerator` and `devices` using `Trainer(accelerator='gpu', devices=1)`.
  rank_zero_warn(
/opt/conda/lib/python3.9/site-packages/pytorch_lightning/trainer/configuration_validator.py:117: PossibleUserWarning: You defined a `validation_step` but have no `val_dataloader`. Skipping val loop.
  rank_zero_warn(

  | Name                          | Type                | Params
-------------------------------------

Training: 0it [00:00, ?it/s]

/opt/conda/lib/python3.9/site-packages/pytorch_lightning/callbacks/progress/base.py:247: UserWarning: The progress bar already tracks a metric with the name(s) 'loss' and `self.log('loss', ..., prog_bar=True)` will overwrite this value.  If this is undesired, change the name or override `get_metrics()` in the progress bar callback.
  rank_zero_warn(


trainer.current_epoch is now 9
beta was, 2
beta is now, 1


`Trainer.fit` stopped: `max_epochs=10` reached.



 Train end and beta NOW =  1
{'dci.informativeness_train': 0.9655, 'dci.informativeness_test': 0.313, 'dci.disentanglement': 0.01029566115377221, 'dci.completeness': 0.010967841682280655, 'mig.discrete_score': 0.013032734380361847, 'sap.score': 0.015550000000000005}


In [1]:
import pytorch_lightning as pl
from pytorch_lightning.loggers.wandb import WandbLogger
from torch.utils.data import DataLoader
from disent.dataset import DisentDataset
from disent.dataset.data import XYObjectData
from disent.dataset.sampling import GroundTruthPairOrigSampler
from disent.frameworks.ae import Ae
from disent.frameworks.vae import BetaVae, AdaVae
from disent.model import AutoEncoder
from disent.model.ae import DecoderConv64, EncoderConv64
from disent.dataset.transform import ToImgTensorF32
from disent.util import is_test_run  # you can ignore and remove this
from disent.metrics import metric_dci, metric_mig, metric_sap 
import torch
import matplotlib.pyplot as plt
import numpy as np
import wandb
import callback as tc

wandb.init()
# prepare the data
data = XYObjectData()
dataset = DisentDataset(data, transform=ToImgTensorF32())
dataloader = DataLoader(dataset=dataset, batch_size=128, shuffle=True)

# create the pytorch lightning system
# create the pytorch lightning system
module: pl.LightningModule = BetaVae(
    model=AutoEncoder(
        encoder=EncoderConv64(x_shape=data.x_shape, z_size=6, z_multiplier=2),
        decoder=DecoderConv64(x_shape=data.x_shape, z_size=6),
    ),
    cfg=BetaVae.cfg(optimizer='adam', optimizer_kwargs=dict(lr=1e-3), loss_reduction='mean_sum', beta=1)
)
wandb_logger = WandbLogger(project="validation.ipynb")
# train the model
trainer = pl.Trainer(logger=wandb_logger, callbacks=[tc.betaControlCallback()], max_epochs = 10, fast_dev_run=is_test_run())
trainer.fit(module, dataloader)

module.eval()

get_repr = lambda x: module.encode(x.to(module.device))

a_results = {
        **metric_dci(dataset, get_repr, num_train=10 if is_test_run() else 1000, num_test=5 if is_test_run() else 500, boost_mode='sklearn'),
        **metric_mig(dataset, get_repr, num_train=20 if is_test_run() else 2000),
        **metric_sap(dataset, get_repr),
    }
print(a_results)
# BetaVAE beta = 1, at e=1 beta = 2

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: iceburg. Use `wandb login --relogin` to force relogin


/opt/conda/lib/python3.9/site-packages/pytorch_lightning/loggers/wandb.py:351: UserWarning: There is a wandb run already in progress and newly created instances of `WandbLogger` will reuse this run. If this is not desired, call `wandb.finish()` before instantiating `WandbLogger`.
  rank_zero_warn(
GPU available: True (cuda), used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/opt/conda/lib/python3.9/site-packages/pytorch_lightning/trainer/trainer.py:1763: PossibleUserWarning: GPU available but not used. Set `accelerator` and `devices` using `Trainer(accelerator='gpu', devices=1)`.
  rank_zero_warn(
/opt/conda/lib/python3.9/site-packages/pytorch_lightning/trainer/configuration_validator.py:117: PossibleUserWarning: You defined a `validation_step` but have no `val_dataloader`. Skipping val loop.
  rank_zero_warn(

  | Name                          | Type                | Params
-------------------------------------

Training: 0it [00:00, ?it/s]

/opt/conda/lib/python3.9/site-packages/pytorch_lightning/callbacks/progress/base.py:247: UserWarning: The progress bar already tracks a metric with the name(s) 'loss' and `self.log('loss', ..., prog_bar=True)` will overwrite this value.  If this is undesired, change the name or override `get_metrics()` in the progress bar callback.
  rank_zero_warn(


trainer.current_epoch is now 1
beta was, 1
beta is now, 2


`Trainer.fit` stopped: `max_epochs=10` reached.



 Train end and beta NOW =  2
{'dci.informativeness_train': 0.9655, 'dci.informativeness_test': 0.29000000000000004, 'dci.disentanglement': 0.014679437523383722, 'dci.completeness': 0.019805473732933315, 'mig.discrete_score': 0.020953557496252107, 'sap.score': 0.016100000000000007}


In [1]:
import pytorch_lightning as pl
from pytorch_lightning.loggers.wandb import WandbLogger
from torch.utils.data import DataLoader
from disent.dataset import DisentDataset
from disent.dataset.data import XYObjectData
from disent.dataset.sampling import GroundTruthPairOrigSampler
from disent.frameworks.ae import Ae
from disent.frameworks.vae import BetaVae, AdaVae
from disent.model import AutoEncoder
from disent.model.ae import DecoderConv64, EncoderConv64
from disent.dataset.transform import ToImgTensorF32
from disent.util import is_test_run  # you can ignore and remove this
from disent.metrics import metric_dci, metric_mig, metric_sap 
import torch
import matplotlib.pyplot as plt
import numpy as np
import wandb
import callback as tc

wandb.init()
# prepare the data
data = XYObjectData()
dataset = DisentDataset(data, transform=ToImgTensorF32())
dataloader = DataLoader(dataset=dataset,num_workers = 128, batch_size=128, shuffle=True)

# create the pytorch lightning system
# create the pytorch lightning system
module: pl.LightningModule = BetaVae(
    model=AutoEncoder(
        encoder=EncoderConv64(x_shape=data.x_shape, z_size=6, z_multiplier=2),
        decoder=DecoderConv64(x_shape=data.x_shape, z_size=6),
    ),
    cfg=BetaVae.cfg(optimizer='adam', optimizer_kwargs=dict(lr=1e-3), loss_reduction='mean_sum', beta=1)
)
wandb_logger = WandbLogger(project="validation.ipynb")
# train the model
trainer = pl.Trainer(logger=wandb_logger, callbacks=[tc.betaControlCallback()], max_epochs = 10, fast_dev_run=is_test_run())
trainer.fit(module, dataloader)

module.eval()

get_repr = lambda x: module.encode(x.to(module.device))

a_results = {
        **metric_dci(dataset, get_repr, num_train=10 if is_test_run() else 1000, num_test=5 if is_test_run() else 500, boost_mode='sklearn'),
        **metric_mig(dataset, get_repr, num_train=20 if is_test_run() else 2000),
        **metric_sap(dataset, get_repr),
    }
print(a_results)
# BetaVAE beta = 1, at e=3 beta = 2

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: iceburg. Use `wandb login --relogin` to force relogin


/opt/conda/lib/python3.9/site-packages/pytorch_lightning/loggers/wandb.py:351: UserWarning: There is a wandb run already in progress and newly created instances of `WandbLogger` will reuse this run. If this is not desired, call `wandb.finish()` before instantiating `WandbLogger`.
  rank_zero_warn(
GPU available: True (cuda), used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/opt/conda/lib/python3.9/site-packages/pytorch_lightning/trainer/trainer.py:1763: PossibleUserWarning: GPU available but not used. Set `accelerator` and `devices` using `Trainer(accelerator='gpu', devices=1)`.
  rank_zero_warn(
/opt/conda/lib/python3.9/site-packages/pytorch_lightning/trainer/configuration_validator.py:117: PossibleUserWarning: You defined a `validation_step` but have no `val_dataloader`. Skipping val loop.
  rank_zero_warn(

  | Name                          | Type                | Params
-------------------------------------

Training: 0it [00:00, ?it/s]

/opt/conda/lib/python3.9/site-packages/pytorch_lightning/callbacks/progress/base.py:247: UserWarning: The progress bar already tracks a metric with the name(s) 'loss' and `self.log('loss', ..., prog_bar=True)` will overwrite this value.  If this is undesired, change the name or override `get_metrics()` in the progress bar callback.
  rank_zero_warn(


trainer.current_epoch is now 3
beta was, 1
beta is now, 2


`Trainer.fit` stopped: `max_epochs=10` reached.



 Train end and beta NOW =  2
{'dci.informativeness_train': 0.977, 'dci.informativeness_test': 0.3035, 'dci.disentanglement': 0.04560209803053397, 'dci.completeness': 0.05231574464408498, 'mig.discrete_score': 0.07017369924526681, 'sap.score': 0.013899999999999992}


In [1]:
import pytorch_lightning as pl
from pytorch_lightning.loggers.wandb import WandbLogger
from torch.utils.data import DataLoader
from disent.dataset import DisentDataset
from disent.dataset.data import XYObjectData
from disent.dataset.sampling import GroundTruthPairOrigSampler
from disent.frameworks.ae import Ae
from disent.frameworks.vae import BetaVae, AdaVae
from disent.model import AutoEncoder
from disent.model.ae import DecoderConv64, EncoderConv64
from disent.dataset.transform import ToImgTensorF32
from disent.util import is_test_run  # you can ignore and remove this
from disent.metrics import metric_dci, metric_mig, metric_sap 
import torch
import matplotlib.pyplot as plt
import numpy as np
import wandb
import callback as tc

wandb.init()
# prepare the data
data = XYObjectData()
dataset = DisentDataset(data, transform=ToImgTensorF32())
dataloader = DataLoader(dataset=dataset,num_workers = 128, batch_size=128, shuffle=True)

# create the pytorch lightning system
# create the pytorch lightning system
module: pl.LightningModule = BetaVae(
    model=AutoEncoder(
        encoder=EncoderConv64(x_shape=data.x_shape, z_size=6, z_multiplier=2),
        decoder=DecoderConv64(x_shape=data.x_shape, z_size=6),
    ),
    cfg=BetaVae.cfg(optimizer='adam', optimizer_kwargs=dict(lr=1e-3), loss_reduction='mean_sum', beta=1)
)
wandb_logger = WandbLogger(project="validation.ipynb")
# train the model
trainer = pl.Trainer(logger=wandb_logger, callbacks=[tc.betaControlCallback()], max_epochs = 10, fast_dev_run=is_test_run())
trainer.fit(module, dataloader)

module.eval()

get_repr = lambda x: module.encode(x.to(module.device))

a_results = {
        **metric_dci(dataset, get_repr, num_train=10 if is_test_run() else 1000, num_test=5 if is_test_run() else 500, boost_mode='sklearn'),
        **metric_mig(dataset, get_repr, num_train=20 if is_test_run() else 2000),
        **metric_sap(dataset, get_repr),
    }
print(a_results)
# BetaVAE beta = 1, at e=5 beta = 2 一会重跑

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: iceburg. Use `wandb login --relogin` to force relogin


/opt/conda/lib/python3.9/site-packages/pytorch_lightning/loggers/wandb.py:351: UserWarning: There is a wandb run already in progress and newly created instances of `WandbLogger` will reuse this run. If this is not desired, call `wandb.finish()` before instantiating `WandbLogger`.
  rank_zero_warn(
GPU available: True (cuda), used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/opt/conda/lib/python3.9/site-packages/pytorch_lightning/trainer/trainer.py:1763: PossibleUserWarning: GPU available but not used. Set `accelerator` and `devices` using `Trainer(accelerator='gpu', devices=1)`.
  rank_zero_warn(
/opt/conda/lib/python3.9/site-packages/pytorch_lightning/trainer/configuration_validator.py:117: PossibleUserWarning: You defined a `validation_step` but have no `val_dataloader`. Skipping val loop.
  rank_zero_warn(

  | Name                          | Type                | Params
-------------------------------------

Training: 0it [00:00, ?it/s]

/opt/conda/lib/python3.9/site-packages/pytorch_lightning/callbacks/progress/base.py:247: UserWarning: The progress bar already tracks a metric with the name(s) 'loss' and `self.log('loss', ..., prog_bar=True)` will overwrite this value.  If this is undesired, change the name or override `get_metrics()` in the progress bar callback.
  rank_zero_warn(


trainer.current_epoch is now 5
beta was, 1
beta is now, 2


`Trainer.fit` stopped: `max_epochs=10` reached.



 Train end and beta NOW =  2
{'dci.informativeness_train': 0.9615, 'dci.informativeness_test': 0.26, 'dci.disentanglement': 0.005728503764652066, 'dci.completeness': 0.011398844617252045, 'mig.discrete_score': 0.021745446389124505, 'sap.score': 0.0027500000000000024}


In [1]:
import pytorch_lightning as pl
from pytorch_lightning.loggers.wandb import WandbLogger
from torch.utils.data import DataLoader
from disent.dataset import DisentDataset
from disent.dataset.data import XYObjectData
from disent.dataset.sampling import GroundTruthPairOrigSampler
from disent.frameworks.ae import Ae
from disent.frameworks.vae import BetaVae, AdaVae
from disent.model import AutoEncoder
from disent.model.ae import DecoderConv64, EncoderConv64
from disent.dataset.transform import ToImgTensorF32
from disent.util import is_test_run  # you can ignore and remove this
from disent.metrics import metric_dci, metric_mig, metric_sap 
import torch
import matplotlib.pyplot as plt
import numpy as np
import wandb
import callback as tc

wandb.init()
# prepare the data
data = XYObjectData()
dataset = DisentDataset(data, transform=ToImgTensorF32())
dataloader = DataLoader(dataset=dataset,num_workers = 128, batch_size=128, shuffle=True)

# create the pytorch lightning system
# create the pytorch lightning system
module: pl.LightningModule = BetaVae(
    model=AutoEncoder(
        encoder=EncoderConv64(x_shape=data.x_shape, z_size=6, z_multiplier=2),
        decoder=DecoderConv64(x_shape=data.x_shape, z_size=6),
    ),
    cfg=BetaVae.cfg(optimizer='adam', optimizer_kwargs=dict(lr=1e-3), loss_reduction='mean_sum', beta=1)
)
wandb_logger = WandbLogger(project="validation.ipynb")
# train the model
trainer = pl.Trainer(logger=wandb_logger, callbacks=[tc.betaControlCallback()], max_epochs = 10, fast_dev_run=is_test_run())
trainer.fit(module, dataloader)

module.eval()

get_repr = lambda x: module.encode(x.to(module.device))

a_results = {
        **metric_dci(dataset, get_repr, num_train=10 if is_test_run() else 1000, num_test=5 if is_test_run() else 500, boost_mode='sklearn'),
        **metric_mig(dataset, get_repr, num_train=20 if is_test_run() else 2000),
        **metric_sap(dataset, get_repr),
    }
print(a_results)
# BetaVAE beta = 1, at e=7 beta = 2

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: iceburg. Use `wandb login --relogin` to force relogin


/opt/conda/lib/python3.9/site-packages/pytorch_lightning/loggers/wandb.py:351: UserWarning: There is a wandb run already in progress and newly created instances of `WandbLogger` will reuse this run. If this is not desired, call `wandb.finish()` before instantiating `WandbLogger`.
  rank_zero_warn(
GPU available: True (cuda), used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/opt/conda/lib/python3.9/site-packages/pytorch_lightning/trainer/trainer.py:1763: PossibleUserWarning: GPU available but not used. Set `accelerator` and `devices` using `Trainer(accelerator='gpu', devices=1)`.
  rank_zero_warn(
/opt/conda/lib/python3.9/site-packages/pytorch_lightning/trainer/configuration_validator.py:117: PossibleUserWarning: You defined a `validation_step` but have no `val_dataloader`. Skipping val loop.
  rank_zero_warn(

  | Name                          | Type                | Params
-------------------------------------

Training: 0it [00:00, ?it/s]

/opt/conda/lib/python3.9/site-packages/pytorch_lightning/callbacks/progress/base.py:247: UserWarning: The progress bar already tracks a metric with the name(s) 'loss' and `self.log('loss', ..., prog_bar=True)` will overwrite this value.  If this is undesired, change the name or override `get_metrics()` in the progress bar callback.
  rank_zero_warn(


trainer.current_epoch is now 7
beta was, 1
beta is now, 2


`Trainer.fit` stopped: `max_epochs=10` reached.



 Train end and beta NOW =  2
{'dci.informativeness_train': 0.9652499999999999, 'dci.informativeness_test': 0.3185, 'dci.disentanglement': 0.0227221063486676, 'dci.completeness': 0.026959690287225863, 'mig.discrete_score': 0.04021348805593837, 'sap.score': 0.012900000000000005}


In [1]:
import pytorch_lightning as pl
from pytorch_lightning.loggers.wandb import WandbLogger
from torch.utils.data import DataLoader
from disent.dataset import DisentDataset
from disent.dataset.data import XYObjectData
from disent.dataset.sampling import GroundTruthPairOrigSampler
from disent.frameworks.ae import Ae
from disent.frameworks.vae import BetaVae, AdaVae
from disent.model import AutoEncoder
from disent.model.ae import DecoderConv64, EncoderConv64
from disent.dataset.transform import ToImgTensorF32
from disent.util import is_test_run  # you can ignore and remove this
from disent.metrics import metric_dci, metric_mig, metric_sap 
import torch
import matplotlib.pyplot as plt
import numpy as np
import wandb
import callback as tc

wandb.init()
# prepare the data
data = XYObjectData()
dataset = DisentDataset(data, transform=ToImgTensorF32())
dataloader = DataLoader(dataset=dataset,num_workers = 128, batch_size=128, shuffle=True)

# create the pytorch lightning system
# create the pytorch lightning system
module: pl.LightningModule = BetaVae(
    model=AutoEncoder(
        encoder=EncoderConv64(x_shape=data.x_shape, z_size=6, z_multiplier=2),
        decoder=DecoderConv64(x_shape=data.x_shape, z_size=6),
    ),
    cfg=BetaVae.cfg(optimizer='adam', optimizer_kwargs=dict(lr=1e-3), loss_reduction='mean_sum', beta=1)
)
wandb_logger = WandbLogger(project="validation.ipynb")
# train the model
trainer = pl.Trainer(logger=wandb_logger, callbacks=[tc.betaControlCallback()], max_epochs = 10, fast_dev_run=is_test_run())
trainer.fit(module, dataloader)

module.eval()

get_repr = lambda x: module.encode(x.to(module.device))

a_results = {
        **metric_dci(dataset, get_repr, num_train=10 if is_test_run() else 1000, num_test=5 if is_test_run() else 500, boost_mode='sklearn'),
        **metric_mig(dataset, get_repr, num_train=20 if is_test_run() else 2000),
        **metric_sap(dataset, get_repr),
    }
print(a_results)
# BetaVAE beta = 1, at e=9 beta = 2

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: iceburg. Use `wandb login --relogin` to force relogin


/opt/conda/lib/python3.9/site-packages/pytorch_lightning/loggers/wandb.py:351: UserWarning: There is a wandb run already in progress and newly created instances of `WandbLogger` will reuse this run. If this is not desired, call `wandb.finish()` before instantiating `WandbLogger`.
  rank_zero_warn(
GPU available: True (cuda), used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/opt/conda/lib/python3.9/site-packages/pytorch_lightning/trainer/trainer.py:1763: PossibleUserWarning: GPU available but not used. Set `accelerator` and `devices` using `Trainer(accelerator='gpu', devices=1)`.
  rank_zero_warn(
/opt/conda/lib/python3.9/site-packages/pytorch_lightning/trainer/configuration_validator.py:117: PossibleUserWarning: You defined a `validation_step` but have no `val_dataloader`. Skipping val loop.
  rank_zero_warn(

  | Name                          | Type                | Params
-------------------------------------

Training: 0it [00:00, ?it/s]

/opt/conda/lib/python3.9/site-packages/pytorch_lightning/callbacks/progress/base.py:247: UserWarning: The progress bar already tracks a metric with the name(s) 'loss' and `self.log('loss', ..., prog_bar=True)` will overwrite this value.  If this is undesired, change the name or override `get_metrics()` in the progress bar callback.
  rank_zero_warn(


trainer.current_epoch is now 9
beta was, 1
beta is now, 2


`Trainer.fit` stopped: `max_epochs=10` reached.



 Train end and beta NOW =  2
{'dci.informativeness_train': 0.96175, 'dci.informativeness_test': 0.2775, 'dci.disentanglement': 0.02827503692616434, 'dci.completeness': 0.031913190185128615, 'mig.discrete_score': 0.030319606993282327, 'sap.score': 0.007449999999999991}


In [7]:
#  ~=~=~=~=~=~=~=~=~=~=~=~=~=~=~=~=~=~=~=~=~=~=~=~=~=~=~=~=~=~=~=~=~=~=~=~=~=~=~
#  MIT License
#
#  Copyright (c) 2021 Nathan Juraj Michlo
#
#  Permission is hereby granted, free of charge, to any person obtaining a copy
#  of this software and associated documentation files (the "Software"), to deal
#  in the Software without restriction, including without limitation the rights
#  to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
#  copies of the Software, and to permit persons to whom the Software is
#  furnished to do so, subject to the following conditions:
#
#  The above copyright notice and this permission notice shall be included in
#  all copies or substantial portions of the Software.
#
#  THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
#  IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
#  FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
#  AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
#  LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
#  OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE
#  SOFTWARE.
#  ~=~=~=~=~=~=~=~=~=~=~=~=~=~=~=~=~=~=~=~=~=~=~=~=~=~=~=~=~=~=~=~=~=~=~=~=~=~=~

import logging

from disent.dataset.util.datafile import DataFileHashedDlH5
from disent.dataset.data._groundtruth import Hdf5GroundTruthData


# ========================================================================= #
# dataset_dsprites                                                          #
# ========================================================================= #


# TODO: this seems to have a memory leak compared to other datasets?
class DSpritesData(Hdf5GroundTruthData):
    """
    DSprites Dataset
    - beta-VAE: Learning Basic Visual Concepts with a Constrained Variational Framework
      (https://github.com/deepmind/dsprites-dataset)
    Files:
        - direct npz: https://raw.githubusercontent.com/deepmind/dsprites-dataset/master/dsprites_ndarray_co1sh3sc6or40x32y32_64x64.npz
                      approx 2.5 GB loaded into memory
        - direct hdf5: https://raw.githubusercontent.com/deepmind/dsprites-dataset/master/dsprites_ndarray_co1sh3sc6or40x32y32_64x64.hdf5
                       default chunk size is (23040, 2, 4), dataset is (737280, 64, 64) uint8.
    # reference implementation: https://github.com/google-research/disentanglement_lib/blob/master/disentanglement_lib/data/ground_truth/dsprites.py
    """

    name = 'dsprites'

    # TODO: reference implementation has colour variants
    factor_names = ('shape', 'scale', 'orientation', 'position_x', 'position_y')
    factor_sizes = (3, 6, 40, 32, 32)  # TOTAL: 737280
    img_shape = (64, 64, 1)

    datafile = DataFileHashedDlH5(
        # download file/link
        uri='https://raw.githubusercontent.com/deepmind/dsprites-dataset/master/dsprites_ndarray_co1sh3sc6or40x32y32_64x64.hdf5',
        uri_hash={'fast': 'd6ee1e43db715c2f0de3c41e38863347', 'full': 'b331c4447a651c44bf5e8ae09022e230'},
        # processed dataset file
        file_hash={'fast': '25013c85aebbf4b1023d72564f9413f0', 'full': '4611d1a03e709cd5d0f6fdcdc221ca0e'},
        # h5 re-save settings
        hdf5_dataset_name='imgs',
        hdf5_chunk_size=(1, 64, 64, 1),
        hdf5_dtype='uint8',
        hdf5_mutator=lambda x: (x * 255)[..., None],  # data is of shape (-1, 64, 64), so we add the channel dimension
        hdf5_obs_shape=(64, 64, 1),
    )


# ========================================================================= #
# END                                                                       #
# ========================================================================= #


if __name__ == '__main__':
    logging.basicConfig(level=logging.DEBUG)
    DSpritesData(prepare=True)

INFO:disent.util.inout.paths:created missing directories: /home/jovyan/data/dataset/dsprites
INFO:disent.dataset.data._groundtruth:dsprites: data_dir_share='/home/jovyan/data/dataset/dsprites'
DEBUG:disent.dataset.data._groundtruth:[preparing]: DataFileHashedDlH5(uri='https://raw.githubusercontent.com/deepmind/dsprites-dataset/master/dsprites_ndarray_co1sh3sc6or40x32y32_64x64.hdf5', uri_name='dsprites_ndarray_co1sh3sc6or40x32y32_64x64.hdf5', out_name='gen.dsprites_ndarray_co1sh3sc6or40x32y32_64x64.hdf5') into data dir: /home/jovyan/data/dataset/dsprites
INFO:disent.util.inout.cache:file is stale because it does not exist: '/home/jovyan/data/dataset/dsprites/gen.dsprites_ndarray_co1sh3sc6or40x32y32_64x64.hdf5'
DEBUG:disent.util.inout.cache:calling wrapped function: <function DataFileHashed.prepare.<locals>.wrapped at 0x7fd6bd6c6310> because the file is stale: '/home/jovyan/data/dataset/dsprites/gen.dsprites_ndarray_co1sh3sc6or40x32y32_64x64.hdf5'
INFO:disent.util.inout.cache:file is sta


[IN ] entry: [1, 64, 64]        (uint8   )   4.000 KiB chunk: [23040, 2, 4]      180.000 KiB chunks per entry: [1, 32, 16]         90.000 MiB (  512)  |  compression: 'gzip' compression lvl: 4
[OUT] entry: [1, 64, 64, 1]     (uint8   )   4.000 KiB chunk: [1, 64, 64, 1]       4.000 KiB chunks per entry: [1, 1, 1, 1]         4.000 KiB (    1)  |  compression: 'gzip' compression lvl: 4


100%|██████████| 737280/737280 [00:44<00:00, 16641.34it/s]
INFO:disent.util.inout.files:moved temporary file to final location: /home/jovyan/data/dataset/dsprites/.temp.8321fd09-384e-448b-a6a7-2502834e893e.gen.dsprites_ndarray_co1sh3sc6or40x32y32_64x64.hdf5 -> /home/jovyan/data/dataset/dsprites/gen.dsprites_ndarray_co1sh3sc6or40x32y32_64x64.hdf5
INFO:disent.dataset.util.hdf5:[FILE SIZES] IN:  26.726 MiB OUT: 101.710 MiB


In [14]:
import pytorch_lightning as pl
from pytorch_lightning.loggers.wandb import WandbLogger
from torch.utils.data import DataLoader, Subset
from disent.dataset import DisentDataset
from disent.dataset.data import XYObjectData, DSpritesData
from disent.dataset.sampling import GroundTruthPairOrigSampler
from disent.frameworks.ae import Ae
from disent.frameworks.vae import BetaVae, AdaVae
from disent.model import AutoEncoder
from disent.model.ae import DecoderConv64, EncoderConv64
from disent.dataset.transform import ToImgTensorF32
from disent.util import is_test_run  # you can ignore and remove this
from disent.metrics import metric_dci, metric_mig, metric_sap 
import torch
import matplotlib.pyplot as plt
import numpy as np
import wandb

#wandb.init()
# prepare the data
data = DSpritesData()
#dataset = DisentDataset(data, GroundTruthPairOrigSampler(), transform=ToImgTensorF32())
dataset1 = Subset(DisentDataset(data, GroundTruthPairOrigSampler(), transform=ToImgTensorF32()),range(1,40000))
dataloader = DataLoader(dataset=dataset1, batch_size=128, shuffle=True)

# create the pytorch lightning system
module: pl.LightningModule = AdaVae(
    model=AutoEncoder(
        encoder=EncoderConv64(x_shape=data.x_shape, z_size=6, z_multiplier=2),
        decoder=DecoderConv64(x_shape=data.x_shape, z_size=6),
    ),
    cfg=AdaVae.cfg(
        optimizer='adam', optimizer_kwargs=dict(lr=1e-3),
        loss_reduction='mean_sum', beta=3, ada_average_mode='gvae', ada_thresh_mode='kl',
    )
)
#wandb_logger = WandbLogger(project="validation.ipynb")
# train the model
trainer = pl.Trainer(logger=wandb_logger, max_epochs = 0, fast_dev_run=is_test_run())
trainer.fit(module, dataloader)

module.eval()

get_repr = lambda x: module.encode(x.to(module.device))

a_results = {
        **metric_dci(dataset, get_repr, num_train=10 if is_test_run() else 1000, num_test=5 if is_test_run() else 500, boost_mode='sklearn'),
        **metric_mig(dataset, get_repr, num_train=20 if is_test_run() else 2000),
        **metric_sap(dataset, get_repr),
    }
print(a_results)

# AdaGVAE beta = 3

INFO:disent.dataset.data._groundtruth:dsprites: data_dir_share='/home/jovyan/data/dataset/dsprites'
GPU available: True (cuda), used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/opt/conda/lib/python3.9/site-packages/pytorch_lightning/trainer/trainer.py:1763: PossibleUserWarning: GPU available but not used. Set `accelerator` and `devices` using `Trainer(accelerator='gpu', devices=1)`.
  rank_zero_warn(
/opt/conda/lib/python3.9/site-packages/pytorch_lightning/trainer/configuration_validator.py:117: PossibleUserWarning: You defined a `validation_step` but have no `val_dataloader`. Skipping val loop.
  rank_zero_warn(

  | Name                          | Type                | Params
----------------------------------------------------------------------
0 | _model                        | AutoEncoder         | 908 K 
1 | _AeAndVaeMixin__recon_handler | ReconLossHandlerMse | 0     
-----------------------------------

{'dci.informativeness_train': 0.9404, 'dci.informativeness_test': 0.18760000000000002, 'dci.disentanglement': 0.009106485591500891, 'dci.completeness': 0.01019433648783741, 'mig.discrete_score': 0.01900471147250521, 'sap.score': 0.00276}


wandb: ERROR Dropped streaming file chunk (see wandb/debug-internal.log)
wandb: ERROR Dropped streaming file chunk (see wandb/debug-internal.log)
wandb: ERROR Dropped streaming file chunk (see wandb/debug-internal.log)
wandb: ERROR Dropped streaming file chunk (see wandb/debug-internal.log)
wandb: ERROR Dropped streaming file chunk (see wandb/debug-internal.log)
wandb: ERROR Dropped streaming file chunk (see wandb/debug-internal.log)
wandb: ERROR Dropped streaming file chunk (see wandb/debug-internal.log)


In [1]:
import pytorch_lightning as pl
from pytorch_lightning.loggers.wandb import WandbLogger
from torch.utils.data import DataLoader, Subset
from disent.dataset import DisentDataset
from disent.dataset.data import XYObjectData, DSpritesData
from disent.dataset.sampling import GroundTruthPairOrigSampler
from disent.frameworks.ae import Ae
from disent.frameworks.vae import BetaVae, AdaVae
from disent.model import AutoEncoder
from disent.model.ae import DecoderConv64, EncoderConv64
from disent.dataset.transform import ToImgTensorF32
from disent.util import is_test_run  # you can ignore and remove this
from disent.metrics import metric_dci, metric_mig, metric_sap 
import torch
import matplotlib.pyplot as plt
import numpy as np
import wandb

wandb.init()
# prepare the data
data = DSpritesData()
dataset = DisentDataset(data, transform=ToImgTensorF32())
dataset1 = Subset(DisentDataset(data, transform=ToImgTensorF32()),range(0,40000))
dataloader = DataLoader(dataset=dataset1, batch_size=128, shuffle=True)

# create the pytorch lightning system
# create the pytorch lightning system
module: pl.LightningModule = BetaVae(
    model=AutoEncoder(
        encoder=EncoderConv64(x_shape=data.x_shape, z_size=6, z_multiplier=2),
        decoder=DecoderConv64(x_shape=data.x_shape, z_size=6),
    ),
    cfg=BetaVae.cfg(optimizer='adam', optimizer_kwargs=dict(lr=1e-3), loss_reduction='mean_sum', beta=4)
)
wandb_logger = WandbLogger(project="validation.ipynb")
# train the model
trainer = pl.Trainer(logger=wandb_logger, max_epochs = 10, fast_dev_run=is_test_run())
trainer.fit(module, dataloader)

module.eval()

get_repr = lambda x: module.encode(x.to(module.device))

a_results = {
        **metric_dci(dataset, get_repr, num_train=10 if is_test_run() else 1000, num_test=5 if is_test_run() else 500, boost_mode='sklearn'),
        **metric_mig(dataset, get_repr, num_train=20 if is_test_run() else 2000),
        **metric_sap(dataset, get_repr),
    }
print(a_results)
# BetaVAE beta = 4

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: iceburg. Use `wandb login --relogin` to force relogin


/opt/conda/lib/python3.9/site-packages/pytorch_lightning/loggers/wandb.py:351: UserWarning: There is a wandb run already in progress and newly created instances of `WandbLogger` will reuse this run. If this is not desired, call `wandb.finish()` before instantiating `WandbLogger`.
  rank_zero_warn(
GPU available: True (cuda), used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/opt/conda/lib/python3.9/site-packages/pytorch_lightning/trainer/trainer.py:1763: PossibleUserWarning: GPU available but not used. Set `accelerator` and `devices` using `Trainer(accelerator='gpu', devices=1)`.
  rank_zero_warn(
/opt/conda/lib/python3.9/site-packages/pytorch_lightning/trainer/configuration_validator.py:117: PossibleUserWarning: You defined a `validation_step` but have no `val_dataloader`. Skipping val loop.
  rank_zero_warn(

  | Name                          | Type                | Params
-------------------------------------

Training: 0it [00:00, ?it/s]

/opt/conda/lib/python3.9/site-packages/pytorch_lightning/callbacks/progress/base.py:247: UserWarning: The progress bar already tracks a metric with the name(s) 'loss' and `self.log('loss', ..., prog_bar=True)` will overwrite this value.  If this is undesired, change the name or override `get_metrics()` in the progress bar callback.
  rank_zero_warn(
`Trainer.fit` stopped: `max_epochs=10` reached.


{'dci.informativeness_train': 0.9448000000000001, 'dci.informativeness_test': 0.27, 'dci.disentanglement': 0.010743319132933409, 'dci.completeness': 0.015594829752244773, 'mig.discrete_score': 0.012013397436693617, 'sap.score': 0.004200000000000001}


In [2]:
import pytorch_lightning as pl
from pytorch_lightning.loggers.wandb import WandbLogger
from torch.utils.data import DataLoader, Subset
from disent.dataset import DisentDataset
from disent.dataset.data import XYObjectData, DSpritesData
from disent.dataset.sampling import GroundTruthPairOrigSampler
from disent.frameworks.ae import Ae
from disent.frameworks.vae import BetaVae, AdaVae
from disent.model import AutoEncoder
from disent.model.ae import DecoderConv64, EncoderConv64
from disent.dataset.transform import ToImgTensorF32
from disent.util import is_test_run  # you can ignore and remove this
from disent.metrics import metric_dci, metric_mig, metric_sap 
import torch
import matplotlib.pyplot as plt
import numpy as np
import wandb

wandb.init()
# prepare the data
data = DSpritesData()
dataset = DisentDataset(data, transform=ToImgTensorF32())
dataset1 = Subset(DisentDataset(data, transform=ToImgTensorF32()),range(0,40000))
dataloader = DataLoader(dataset=dataset1, batch_size=128, shuffle=True)

# create the pytorch lightning system
# create the pytorch lightning system
module: pl.LightningModule = BetaVae(
    model=AutoEncoder(
        encoder=EncoderConv64(x_shape=data.x_shape, z_size=6, z_multiplier=2),
        decoder=DecoderConv64(x_shape=data.x_shape, z_size=6),
    ),
    cfg=BetaVae.cfg(optimizer='adam', optimizer_kwargs=dict(lr=1e-3), loss_reduction='mean_sum', beta=3)
)
wandb_logger = WandbLogger(project="validation.ipynb")
# train the model
trainer = pl.Trainer(logger=wandb_logger, max_epochs = 10, fast_dev_run=is_test_run())
trainer.fit(module, dataloader)

module.eval()

get_repr = lambda x: module.encode(x.to(module.device))

a_results = {
        **metric_dci(dataset, get_repr, num_train=10 if is_test_run() else 1000, num_test=5 if is_test_run() else 500, boost_mode='sklearn'),
        **metric_mig(dataset, get_repr, num_train=20 if is_test_run() else 2000),
        **metric_sap(dataset, get_repr),
    }
print(a_results)
# BetaVAE beta = 3

aug_loss,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch,▁▁▁▁▂▂▂▂▃▃▃▃▃▃▃▃▄▄▄▄▅▅▅▅▆▆▆▆▆▆▆▆▇▇▇▇████
kl_loss,▁▁██▇▇█▇███████▇███▇██▇█████▇▇▇█▇███▇██▇
kl_reg_loss,▁▁██▇▇█▇███████▇███▇██▇█████▇▇▇█▇███▇██▇
loss,██▂▂▂▂▂▁▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
pixel_loss,██▂▂▂▂▁▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
recon_loss,██▂▂▂▂▁▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
reg_loss,▁▁██▇▇█▇███████▇███▇██▇█████▇▇▇█▇███▇██▇
trainer/global_step,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
aug_loss,0.0
epoch,9


/opt/conda/lib/python3.9/site-packages/pytorch_lightning/loggers/wandb.py:351: UserWarning: There is a wandb run already in progress and newly created instances of `WandbLogger` will reuse this run. If this is not desired, call `wandb.finish()` before instantiating `WandbLogger`.
  rank_zero_warn(
GPU available: True (cuda), used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/opt/conda/lib/python3.9/site-packages/pytorch_lightning/trainer/trainer.py:1763: PossibleUserWarning: GPU available but not used. Set `accelerator` and `devices` using `Trainer(accelerator='gpu', devices=1)`.
  rank_zero_warn(
/opt/conda/lib/python3.9/site-packages/pytorch_lightning/trainer/configuration_validator.py:117: PossibleUserWarning: You defined a `validation_step` but have no `val_dataloader`. Skipping val loop.
  rank_zero_warn(

  | Name                          | Type                | Params
-------------------------------------

Training: 0it [00:00, ?it/s]

/opt/conda/lib/python3.9/site-packages/pytorch_lightning/callbacks/progress/base.py:247: UserWarning: The progress bar already tracks a metric with the name(s) 'loss' and `self.log('loss', ..., prog_bar=True)` will overwrite this value.  If this is undesired, change the name or override `get_metrics()` in the progress bar callback.
  rank_zero_warn(
`Trainer.fit` stopped: `max_epochs=10` reached.


{'dci.informativeness_train': 0.9414, 'dci.informativeness_test': 0.2956, 'dci.disentanglement': 0.01580334537603302, 'dci.completeness': 0.021859942864444044, 'mig.discrete_score': 0.026773211423763828, 'sap.score': 0.024800000000000006}


In [1]:
import pytorch_lightning as pl
from pytorch_lightning.loggers.wandb import WandbLogger
from torch.utils.data import DataLoader, Subset
from disent.dataset import DisentDataset
from disent.dataset.data import XYObjectData, DSpritesData
from disent.dataset.sampling import GroundTruthPairOrigSampler
from disent.frameworks.ae import Ae
from disent.frameworks.vae import BetaVae, AdaVae
from disent.model import AutoEncoder
from disent.model.ae import DecoderConv64, EncoderConv64
from disent.dataset.transform import ToImgTensorF32
from disent.util import is_test_run  # you can ignore and remove this
from disent.metrics import metric_dci, metric_mig, metric_sap 
import torch
import matplotlib.pyplot as plt
import numpy as np
import wandb

wandb.init()
# prepare the data
data = DSpritesData()
dataset = DisentDataset(data, transform=ToImgTensorF32())
dataset1 = Subset(DisentDataset(data, transform=ToImgTensorF32()),range(0,40000))
dataloader = DataLoader(dataset=dataset1, batch_size=128, shuffle=True)

# create the pytorch lightning system
# create the pytorch lightning system
module: pl.LightningModule = BetaVae(
    model=AutoEncoder(
        encoder=EncoderConv64(x_shape=data.x_shape, z_size=6, z_multiplier=2),
        decoder=DecoderConv64(x_shape=data.x_shape, z_size=6),
    ),
    cfg=BetaVae.cfg(optimizer='adam', optimizer_kwargs=dict(lr=1e-3), loss_reduction='mean_sum', beta=2)
)
wandb_logger = WandbLogger(project="validation.ipynb")
# train the model
trainer = pl.Trainer(logger=wandb_logger, max_epochs = 10, fast_dev_run=is_test_run())
trainer.fit(module, dataloader)

module.eval()

get_repr = lambda x: module.encode(x.to(module.device))

a_results = {
        **metric_dci(dataset, get_repr, num_train=10 if is_test_run() else 1000, num_test=5 if is_test_run() else 500, boost_mode='sklearn'),
        **metric_mig(dataset, get_repr, num_train=20 if is_test_run() else 2000),
        **metric_sap(dataset, get_repr),
    }
print(a_results)
# BetaVAE beta = 2

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: iceburg. Use `wandb login --relogin` to force relogin


/opt/conda/lib/python3.9/site-packages/pytorch_lightning/loggers/wandb.py:351: UserWarning: There is a wandb run already in progress and newly created instances of `WandbLogger` will reuse this run. If this is not desired, call `wandb.finish()` before instantiating `WandbLogger`.
  rank_zero_warn(
GPU available: True (cuda), used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/opt/conda/lib/python3.9/site-packages/pytorch_lightning/trainer/trainer.py:1763: PossibleUserWarning: GPU available but not used. Set `accelerator` and `devices` using `Trainer(accelerator='gpu', devices=1)`.
  rank_zero_warn(
/opt/conda/lib/python3.9/site-packages/pytorch_lightning/trainer/configuration_validator.py:117: PossibleUserWarning: You defined a `validation_step` but have no `val_dataloader`. Skipping val loop.
  rank_zero_warn(

  | Name                          | Type                | Params
-------------------------------------

Training: 0it [00:00, ?it/s]

/opt/conda/lib/python3.9/site-packages/pytorch_lightning/callbacks/progress/base.py:247: UserWarning: The progress bar already tracks a metric with the name(s) 'loss' and `self.log('loss', ..., prog_bar=True)` will overwrite this value.  If this is undesired, change the name or override `get_metrics()` in the progress bar callback.
  rank_zero_warn(
`Trainer.fit` stopped: `max_epochs=10` reached.


{'dci.informativeness_train': 0.9398, 'dci.informativeness_test': 0.2964, 'dci.disentanglement': 0.007453266502894405, 'dci.completeness': 0.008843422304309788, 'mig.discrete_score': 0.010728792550681498, 'sap.score': 0.027800000000000002}


In [1]:
import pytorch_lightning as pl
from pytorch_lightning.loggers.wandb import WandbLogger
from torch.utils.data import DataLoader, Subset
from disent.dataset import DisentDataset
from disent.dataset.data import XYObjectData, DSpritesData
from disent.dataset.sampling import GroundTruthPairOrigSampler
from disent.frameworks.ae import Ae
from disent.frameworks.vae import BetaVae, AdaVae
from disent.model import AutoEncoder
from disent.model.ae import DecoderConv64, EncoderConv64
from disent.dataset.transform import ToImgTensorF32
from disent.util import is_test_run  # you can ignore and remove this
from disent.metrics import metric_dci, metric_mig, metric_sap 
import torch
import matplotlib.pyplot as plt
import numpy as np
import wandb

wandb.init()
# prepare the data
data = DSpritesData()
dataset = DisentDataset(data, transform=ToImgTensorF32())
dataset1 = Subset(DisentDataset(data, transform=ToImgTensorF32()),range(0,40000))
dataloader = DataLoader(dataset=dataset1, batch_size=128, shuffle=True)

# create the pytorch lightning system
# create the pytorch lightning system
module: pl.LightningModule = BetaVae(
    model=AutoEncoder(
        encoder=EncoderConv64(x_shape=data.x_shape, z_size=6, z_multiplier=2),
        decoder=DecoderConv64(x_shape=data.x_shape, z_size=6),
    ),
    cfg=BetaVae.cfg(optimizer='adam', optimizer_kwargs=dict(lr=1e-3), loss_reduction='mean_sum', beta=1)
)
wandb_logger = WandbLogger(project="validation.ipynb")
# train the model
trainer = pl.Trainer(logger=wandb_logger, max_epochs = 10, fast_dev_run=is_test_run())
trainer.fit(module, dataloader)

module.eval()

get_repr = lambda x: module.encode(x.to(module.device))

a_results = {
        **metric_dci(dataset, get_repr, num_train=10 if is_test_run() else 1000, num_test=5 if is_test_run() else 500, boost_mode='sklearn'),
        **metric_mig(dataset, get_repr, num_train=20 if is_test_run() else 2000),
        **metric_sap(dataset, get_repr),
    }
print(a_results)
# BetaVAE beta = 1

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: iceburg. Use `wandb login --relogin` to force relogin


/opt/conda/lib/python3.9/site-packages/pytorch_lightning/loggers/wandb.py:351: UserWarning: There is a wandb run already in progress and newly created instances of `WandbLogger` will reuse this run. If this is not desired, call `wandb.finish()` before instantiating `WandbLogger`.
  rank_zero_warn(
GPU available: True (cuda), used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/opt/conda/lib/python3.9/site-packages/pytorch_lightning/trainer/trainer.py:1763: PossibleUserWarning: GPU available but not used. Set `accelerator` and `devices` using `Trainer(accelerator='gpu', devices=1)`.
  rank_zero_warn(
/opt/conda/lib/python3.9/site-packages/pytorch_lightning/trainer/configuration_validator.py:117: PossibleUserWarning: You defined a `validation_step` but have no `val_dataloader`. Skipping val loop.
  rank_zero_warn(

  | Name                          | Type                | Params
-------------------------------------

Training: 0it [00:00, ?it/s]

/opt/conda/lib/python3.9/site-packages/pytorch_lightning/callbacks/progress/base.py:247: UserWarning: The progress bar already tracks a metric with the name(s) 'loss' and `self.log('loss', ..., prog_bar=True)` will overwrite this value.  If this is undesired, change the name or override `get_metrics()` in the progress bar callback.
  rank_zero_warn(
`Trainer.fit` stopped: `max_epochs=10` reached.


{'dci.informativeness_train': 0.9443999999999999, 'dci.informativeness_test': 0.27840000000000004, 'dci.disentanglement': 0.007064775821406678, 'dci.completeness': 0.011557220653987432, 'mig.discrete_score': 0.008483469396094914, 'sap.score': 0.013999999999999995}


In [2]:
import pytorch_lightning as pl
from pytorch_lightning.loggers.wandb import WandbLogger
from torch.utils.data import DataLoader, Subset
from disent.dataset import DisentDataset
from disent.dataset.data import XYObjectData, DSpritesData
from disent.dataset.sampling import GroundTruthPairOrigSampler
from disent.frameworks.ae import Ae
from disent.frameworks.vae import BetaVae, AdaVae
from disent.model import AutoEncoder
from disent.model.ae import DecoderConv64, EncoderConv64
from disent.dataset.transform import ToImgTensorF32
from disent.util import is_test_run  # you can ignore and remove this
from disent.metrics import metric_dci, metric_mig, metric_sap 
import torch
import matplotlib.pyplot as plt
import numpy as np
import wandb
import callback as tc

#wandb.init()
# prepare the data
data = DSpritesData()
dataset = DisentDataset(data, transform=ToImgTensorF32())
dataset1 = Subset(DisentDataset(data, transform=ToImgTensorF32()),range(0,40000))
dataloader = DataLoader(dataset=dataset1, batch_size=128, shuffle=True)

# create the pytorch lightning system
# create the pytorch lightning system
module: pl.LightningModule = BetaVae(
    model=AutoEncoder(
        encoder=EncoderConv64(x_shape=data.x_shape, z_size=6, z_multiplier=2),
        decoder=DecoderConv64(x_shape=data.x_shape, z_size=6),
    ),
    cfg=BetaVae.cfg(optimizer='adam', optimizer_kwargs=dict(lr=1e-3), loss_reduction='mean_sum', beta=1)
)
wandb_logger = WandbLogger(project="validation.ipynb")
# train the model
trainer = pl.Trainer(logger=wandb_logger,callbacks=[tc.betaControlCallback()], max_epochs = 10, fast_dev_run=is_test_run())
trainer.fit(module, dataloader)

module.eval()

get_repr = lambda x: module.encode(x.to(module.device))

a_results = {
        **metric_dci(dataset, get_repr, num_train=10 if is_test_run() else 1000, num_test=5 if is_test_run() else 500, boost_mode='sklearn'),
        **metric_mig(dataset, get_repr, num_train=20 if is_test_run() else 2000),
        **metric_sap(dataset, get_repr),
    }
print(a_results)
# BetaVAE beta = 1 at e=1 beta = 2

GPU available: True (cuda), used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/opt/conda/lib/python3.9/site-packages/pytorch_lightning/trainer/trainer.py:1763: PossibleUserWarning: GPU available but not used. Set `accelerator` and `devices` using `Trainer(accelerator='gpu', devices=1)`.
  rank_zero_warn(
/opt/conda/lib/python3.9/site-packages/pytorch_lightning/trainer/configuration_validator.py:117: PossibleUserWarning: You defined a `validation_step` but have no `val_dataloader`. Skipping val loop.
  rank_zero_warn(

  | Name                          | Type                | Params
----------------------------------------------------------------------
0 | _model                        | AutoEncoder         | 908 K 
1 | _AeAndVaeMixin__recon_handler | ReconLossHandlerMse | 0     
----------------------------------------------------------------------
908 K     Trainable params
0         Non-trainable params
908 K 

Training: 0it [00:00, ?it/s]

/opt/conda/lib/python3.9/site-packages/pytorch_lightning/callbacks/progress/base.py:247: UserWarning: The progress bar already tracks a metric with the name(s) 'loss' and `self.log('loss', ..., prog_bar=True)` will overwrite this value.  If this is undesired, change the name or override `get_metrics()` in the progress bar callback.
  rank_zero_warn(


trainer.current_epoch is now 1
beta was, 1
beta is now, 2


`Trainer.fit` stopped: `max_epochs=10` reached.



 Train end and beta NOW =  2
{'dci.informativeness_train': 0.952, 'dci.informativeness_test': 0.29840000000000005, 'dci.disentanglement': 0.01067420503011551, 'dci.completeness': 0.013610852883915234, 'mig.discrete_score': 0.014443691467473782, 'sap.score': 0.020240000000000005}


In [1]:
import pytorch_lightning as pl
from pytorch_lightning.loggers.wandb import WandbLogger
from torch.utils.data import DataLoader, Subset
from disent.dataset import DisentDataset
from disent.dataset.data import XYObjectData, DSpritesData
from disent.dataset.sampling import GroundTruthPairOrigSampler
from disent.frameworks.ae import Ae
from disent.frameworks.vae import BetaVae, AdaVae
from disent.model import AutoEncoder
from disent.model.ae import DecoderConv64, EncoderConv64
from disent.dataset.transform import ToImgTensorF32
from disent.util import is_test_run  # you can ignore and remove this
from disent.metrics import metric_dci, metric_mig, metric_sap 
import torch
import matplotlib.pyplot as plt
import numpy as np
import wandb
import callback as tc

wandb.init()
# prepare the data
data = DSpritesData()
dataset = DisentDataset(data, transform=ToImgTensorF32())
dataset1 = Subset(DisentDataset(data, transform=ToImgTensorF32()),range(0,40000))
dataloader = DataLoader(dataset=dataset1, batch_size=128, shuffle=True)

# create the pytorch lightning system
# create the pytorch lightning system
module: pl.LightningModule = BetaVae(
    model=AutoEncoder(
        encoder=EncoderConv64(x_shape=data.x_shape, z_size=6, z_multiplier=2),
        decoder=DecoderConv64(x_shape=data.x_shape, z_size=6),
    ),
    cfg=BetaVae.cfg(optimizer='adam', optimizer_kwargs=dict(lr=1e-3), loss_reduction='mean_sum', beta=1)
)
wandb_logger = WandbLogger(project="validation.ipynb")
# train the model
trainer = pl.Trainer(logger=wandb_logger,callbacks=[tc.betaControlCallback()], max_epochs = 10, fast_dev_run=is_test_run())
trainer.fit(module, dataloader)

module.eval()

get_repr = lambda x: module.encode(x.to(module.device))

a_results = {
        **metric_dci(dataset, get_repr, num_train=10 if is_test_run() else 1000, num_test=5 if is_test_run() else 500, boost_mode='sklearn'),
        **metric_mig(dataset, get_repr, num_train=20 if is_test_run() else 2000),
        **metric_sap(dataset, get_repr),
    }
print(a_results)
# BetaVAE beta = 1 at e=3 beta = 2

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: iceburg. Use `wandb login --relogin` to force relogin


/opt/conda/lib/python3.9/site-packages/pytorch_lightning/loggers/wandb.py:351: UserWarning: There is a wandb run already in progress and newly created instances of `WandbLogger` will reuse this run. If this is not desired, call `wandb.finish()` before instantiating `WandbLogger`.
  rank_zero_warn(
GPU available: True (cuda), used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/opt/conda/lib/python3.9/site-packages/pytorch_lightning/trainer/trainer.py:1763: PossibleUserWarning: GPU available but not used. Set `accelerator` and `devices` using `Trainer(accelerator='gpu', devices=1)`.
  rank_zero_warn(
/opt/conda/lib/python3.9/site-packages/pytorch_lightning/trainer/configuration_validator.py:117: PossibleUserWarning: You defined a `validation_step` but have no `val_dataloader`. Skipping val loop.
  rank_zero_warn(

  | Name                          | Type                | Params
-------------------------------------

Training: 0it [00:00, ?it/s]

/opt/conda/lib/python3.9/site-packages/pytorch_lightning/callbacks/progress/base.py:247: UserWarning: The progress bar already tracks a metric with the name(s) 'loss' and `self.log('loss', ..., prog_bar=True)` will overwrite this value.  If this is undesired, change the name or override `get_metrics()` in the progress bar callback.
  rank_zero_warn(


trainer.current_epoch is now 3
beta was, 1
beta is now, 2


`Trainer.fit` stopped: `max_epochs=10` reached.



 Train end and beta NOW =  2
{'dci.informativeness_train': 0.9432, 'dci.informativeness_test': 0.3228, 'dci.disentanglement': 0.06487938849052095, 'dci.completeness': 0.06673431709145436, 'mig.discrete_score': 0.04730935350770561, 'sap.score': 0.03764000000000001}


In [1]:
import pytorch_lightning as pl
from pytorch_lightning.loggers.wandb import WandbLogger
from torch.utils.data import DataLoader
from disent.dataset import DisentDataset
from disent.dataset.data import XYObjectData
from disent.dataset.sampling import GroundTruthPairOrigSampler
from disent.frameworks.ae import Ae
from disent.frameworks.vae import BetaVae, AdaVae
from disent.model import AutoEncoder
from disent.model.ae import DecoderConv64, EncoderConv64
from disent.dataset.transform import ToImgTensorF32
from disent.util import is_test_run  # you can ignore and remove this
from disent.metrics import metric_dci, metric_mig, metric_sap 
import torch
import matplotlib.pyplot as plt
import numpy as np
import wandb

wandb.init()
# prepare the data
data = XYObjectData()
dataset = DisentDataset(data, GroundTruthPairOrigSampler(), transform=ToImgTensorF32())
dataloader = DataLoader(dataset=dataset, batch_size=128, shuffle=True)

# create the pytorch lightning system
module: pl.LightningModule = AdaVae(
    model=AutoEncoder(
        encoder=EncoderConv64(x_shape=data.x_shape, z_size=6, z_multiplier=2),
        decoder=DecoderConv64(x_shape=data.x_shape, z_size=6),
    ),
    cfg=AdaVae.cfg(
        optimizer='adam', optimizer_kwargs=dict(lr=1e-3),
        loss_reduction='mean_sum', beta=0, ada_average_mode='gvae', ada_thresh_mode='kl',
    )
)
wandb_logger = WandbLogger(project="validation.ipynb")
# train the model
trainer = pl.Trainer(logger=wandb_logger, max_epochs = 10, fast_dev_run=is_test_run())
trainer.fit(module, dataloader)

module.eval()

get_repr = lambda x: module.encode(x.to(module.device))

a_results = {
        **metric_dci(dataset, get_repr, num_train=10 if is_test_run() else 1000, num_test=5 if is_test_run() else 500, boost_mode='sklearn'),
        **metric_mig(dataset, get_repr, num_train=20 if is_test_run() else 2000),
        **metric_sap(dataset, get_repr),
    }
print(a_results)
# AdaGVAE beta = 0

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: iceburg. Use `wandb login --relogin` to force relogin


/opt/conda/lib/python3.9/site-packages/pytorch_lightning/loggers/wandb.py:351: UserWarning: There is a wandb run already in progress and newly created instances of `WandbLogger` will reuse this run. If this is not desired, call `wandb.finish()` before instantiating `WandbLogger`.
  rank_zero_warn(
GPU available: True (cuda), used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/opt/conda/lib/python3.9/site-packages/pytorch_lightning/trainer/trainer.py:1763: PossibleUserWarning: GPU available but not used. Set `accelerator` and `devices` using `Trainer(accelerator='gpu', devices=1)`.
  rank_zero_warn(
/opt/conda/lib/python3.9/site-packages/pytorch_lightning/trainer/configuration_validator.py:117: PossibleUserWarning: You defined a `validation_step` but have no `val_dataloader`. Skipping val loop.
  rank_zero_warn(

  | Name                          | Type                | Params
-------------------------------------

Training: 0it [00:00, ?it/s]

/opt/conda/lib/python3.9/site-packages/pytorch_lightning/callbacks/progress/base.py:247: UserWarning: The progress bar already tracks a metric with the name(s) 'loss' and `self.log('loss', ..., prog_bar=True)` will overwrite this value.  If this is undesired, change the name or override `get_metrics()` in the progress bar callback.
  rank_zero_warn(
`Trainer.fit` stopped: `max_epochs=10` reached.


{'dci.informativeness_train': 0.9875, 'dci.informativeness_test': 0.6655, 'dci.disentanglement': 0.2374377488697766, 'dci.completeness': 0.19478966337021464, 'mig.discrete_score': 0.21664921208308005, 'sap.score': 0.01485}


In [1]:
import pytorch_lightning as pl
from pytorch_lightning.loggers.wandb import WandbLogger
from torch.utils.data import DataLoader
from disent.dataset import DisentDataset
from disent.dataset.data import XYObjectData
from disent.dataset.sampling import GroundTruthPairOrigSampler
from disent.frameworks.ae import Ae
from disent.frameworks.vae import BetaVae, AdaVae
from disent.model import AutoEncoder
from disent.model.ae import DecoderConv64, EncoderConv64
from disent.dataset.transform import ToImgTensorF32
from disent.util import is_test_run  # you can ignore and remove this
from disent.metrics import metric_dci, metric_mig, metric_sap 
import torch
import matplotlib.pyplot as plt
import numpy as np
import wandb
import callback as tc

wandb.init()
# prepare the data
data = XYObjectData()
dataset = DisentDataset(data, GroundTruthPairOrigSampler(), transform=ToImgTensorF32())
dataloader = DataLoader(dataset=dataset, batch_size=128, shuffle=True)

# create the pytorch lightning system
module: pl.LightningModule = AdaVae(
    model=AutoEncoder(
        encoder=EncoderConv64(x_shape=data.x_shape, z_size=6, z_multiplier=2),
        decoder=DecoderConv64(x_shape=data.x_shape, z_size=6),
    ),
    cfg=AdaVae.cfg(
        optimizer='adam', optimizer_kwargs=dict(lr=1e-3),
        loss_reduction='mean_sum', beta=1, ada_average_mode='gvae', ada_thresh_mode='kl',
    )
)
wandb_logger = WandbLogger(project="beta-VAE")
# train the model
trainer = pl.Trainer(logger=wandb_logger, callbacks=[tc.betaControlCallback()], max_epochs = 10, fast_dev_run=is_test_run())
trainer.fit(module, dataloader)

module.eval()

get_repr = lambda x: module.encode(x.to(module.device))

a_results = {
        **metric_dci(dataset, get_repr, num_train=10 if is_test_run() else 1000, num_test=5 if is_test_run() else 500, boost_mode='sklearn'),
        **metric_mig(dataset, get_repr, num_train=20 if is_test_run() else 2000),
        **metric_sap(dataset, get_repr),
    }
print(a_results)
# AdaGVAE beta = 1 at e=1, beta = 2

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: iceburg. Use `wandb login --relogin` to force relogin


/opt/conda/lib/python3.9/site-packages/pytorch_lightning/loggers/wandb.py:351: UserWarning: There is a wandb run already in progress and newly created instances of `WandbLogger` will reuse this run. If this is not desired, call `wandb.finish()` before instantiating `WandbLogger`.
  rank_zero_warn(
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/opt/conda/lib/python3.9/site-packages/pytorch_lightning/trainer/configuration_validator.py:117: PossibleUserWarning: You defined a `validation_step` but have no `val_dataloader`. Skipping val loop.
  rank_zero_warn(

  | Name                          | Type                | Params
----------------------------------------------------------------------
0 | _model                        | AutoEncoder         | 910 K 
1 | _AeAndVaeMixin__recon_handler | ReconLossHandlerMse | 0     
----------------------------------------------------------------------

Training: 0it [00:00, ?it/s]

/opt/conda/lib/python3.9/site-packages/pytorch_lightning/callbacks/progress/base.py:247: UserWarning: The progress bar already tracks a metric with the name(s) 'loss' and `self.log('loss', ..., prog_bar=True)` will overwrite this value.  If this is undesired, change the name or override `get_metrics()` in the progress bar callback.
  rank_zero_warn(


trainer.current_epoch is now 1
beta was, 1
beta is now, 2


`Trainer.fit` stopped: `max_epochs=10` reached.



 Train end and beta NOW =  2
{'dci.informativeness_train': 0.913, 'dci.informativeness_test': 0.20199999999999999, 'dci.disentanglement': 0.008324600184776315, 'dci.completeness': 0.010495640459921322, 'mig.discrete_score': 0.008899098587702162, 'sap.score': 0.0}


In [1]:
import pytorch_lightning as pl
from pytorch_lightning.loggers.wandb import WandbLogger
from torch.utils.data import DataLoader
from disent.dataset import DisentDataset
from disent.dataset.data import XYObjectData
from disent.dataset.sampling import GroundTruthPairOrigSampler
from disent.frameworks.ae import Ae
from disent.frameworks.vae import BetaVae, AdaVae
from disent.model import AutoEncoder
from disent.model.ae import DecoderConv64, EncoderConv64
from disent.dataset.transform import ToImgTensorF32
from disent.util import is_test_run  # you can ignore and remove this
from disent.metrics import metric_dci, metric_mig, metric_sap 
import torch
import matplotlib.pyplot as plt
import numpy as np
import wandb
import callback as tc

wandb.init()
# prepare the data
data = XYObjectData()
dataset = DisentDataset(data, GroundTruthPairOrigSampler(), transform=ToImgTensorF32())
dataloader = DataLoader(dataset=dataset, batch_size=128, shuffle=True)

# create the pytorch lightning system
module: pl.LightningModule = AdaVae(
    model=AutoEncoder(
        encoder=EncoderConv64(x_shape=data.x_shape, z_size=6, z_multiplier=2),
        decoder=DecoderConv64(x_shape=data.x_shape, z_size=6),
    ),
    cfg=AdaVae.cfg(
        optimizer='adam', optimizer_kwargs=dict(lr=1e-3),
        loss_reduction='mean_sum', beta=1, ada_average_mode='gvae', ada_thresh_mode='kl',
    )
)
wandb_logger = WandbLogger(project="beta-VAE")
# train the model
trainer = pl.Trainer(logger=wandb_logger, callbacks=[tc.betaControlCallback()], max_epochs = 10, fast_dev_run=is_test_run())
trainer.fit(module, dataloader)

module.eval()

get_repr = lambda x: module.encode(x.to(module.device))

a_results = {
        **metric_dci(dataset, get_repr, num_train=10 if is_test_run() else 1000, num_test=5 if is_test_run() else 500, boost_mode='sklearn'),
        **metric_mig(dataset, get_repr, num_train=20 if is_test_run() else 2000),
        **metric_sap(dataset, get_repr),
    }
print(a_results)
# AdaGVAE beta = 1 at e=3, beta = 2

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: iceburg. Use `wandb login --relogin` to force relogin


/opt/conda/lib/python3.9/site-packages/pytorch_lightning/loggers/wandb.py:351: UserWarning: There is a wandb run already in progress and newly created instances of `WandbLogger` will reuse this run. If this is not desired, call `wandb.finish()` before instantiating `WandbLogger`.
  rank_zero_warn(
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/opt/conda/lib/python3.9/site-packages/pytorch_lightning/trainer/configuration_validator.py:117: PossibleUserWarning: You defined a `validation_step` but have no `val_dataloader`. Skipping val loop.
  rank_zero_warn(

  | Name                          | Type                | Params
----------------------------------------------------------------------
0 | _model                        | AutoEncoder         | 910 K 
1 | _AeAndVaeMixin__recon_handler | ReconLossHandlerMse | 0     
----------------------------------------------------------------------

Training: 0it [00:00, ?it/s]

/opt/conda/lib/python3.9/site-packages/pytorch_lightning/callbacks/progress/base.py:247: UserWarning: The progress bar already tracks a metric with the name(s) 'loss' and `self.log('loss', ..., prog_bar=True)` will overwrite this value.  If this is undesired, change the name or override `get_metrics()` in the progress bar callback.
  rank_zero_warn(


trainer.current_epoch is now 3
beta was, 1
beta is now, 2


`Trainer.fit` stopped: `max_epochs=10` reached.



 Train end and beta NOW =  2
{'dci.informativeness_train': 0.97675, 'dci.informativeness_test': 0.673, 'dci.disentanglement': 0.5025637093809852, 'dci.completeness': 0.5769417514799695, 'mig.discrete_score': 0.4873825306087149, 'sap.score': 0.015100000000000008}


In [1]:
import pytorch_lightning as pl
from pytorch_lightning.loggers.wandb import WandbLogger
from torch.utils.data import DataLoader
from disent.dataset import DisentDataset
from disent.dataset.data import XYObjectData
from disent.dataset.sampling import GroundTruthPairOrigSampler
from disent.frameworks.ae import Ae
from disent.frameworks.vae import BetaVae, AdaVae
from disent.model import AutoEncoder
from disent.model.ae import DecoderConv64, EncoderConv64
from disent.dataset.transform import ToImgTensorF32
from disent.util import is_test_run  # you can ignore and remove this
from disent.metrics import metric_dci, metric_mig, metric_sap 
import torch
import matplotlib.pyplot as plt
import numpy as np
import wandb
import callback as tc

wandb.init()
# prepare the data
data = XYObjectData()
dataset = DisentDataset(data, GroundTruthPairOrigSampler(), transform=ToImgTensorF32())
dataloader = DataLoader(dataset=dataset, batch_size=128, shuffle=True)

# create the pytorch lightning system
module: pl.LightningModule = AdaVae(
    model=AutoEncoder(
        encoder=EncoderConv64(x_shape=data.x_shape, z_size=6, z_multiplier=2),
        decoder=DecoderConv64(x_shape=data.x_shape, z_size=6),
    ),
    cfg=AdaVae.cfg(
        optimizer='adam', optimizer_kwargs=dict(lr=1e-3),
        loss_reduction='mean_sum', beta=1, ada_average_mode='gvae', ada_thresh_mode='kl',
    )
)
wandb_logger = WandbLogger(project="beta-VAE")
# train the model
trainer = pl.Trainer(logger=wandb_logger, callbacks=[tc.betaControlCallback()], max_epochs = 10, fast_dev_run=is_test_run())
trainer.fit(module, dataloader)

module.eval()

get_repr = lambda x: module.encode(x.to(module.device))

a_results = {
        **metric_dci(dataset, get_repr, num_train=10 if is_test_run() else 1000, num_test=5 if is_test_run() else 500, boost_mode='sklearn'),
        **metric_mig(dataset, get_repr, num_train=20 if is_test_run() else 2000),
        **metric_sap(dataset, get_repr),
    }
print(a_results)
# AdaGVAE beta = 1 at e=5, beta = 2

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: iceburg. Use `wandb login --relogin` to force relogin


/opt/conda/lib/python3.9/site-packages/pytorch_lightning/loggers/wandb.py:351: UserWarning: There is a wandb run already in progress and newly created instances of `WandbLogger` will reuse this run. If this is not desired, call `wandb.finish()` before instantiating `WandbLogger`.
  rank_zero_warn(
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/opt/conda/lib/python3.9/site-packages/pytorch_lightning/trainer/configuration_validator.py:117: PossibleUserWarning: You defined a `validation_step` but have no `val_dataloader`. Skipping val loop.
  rank_zero_warn(

  | Name                          | Type                | Params
----------------------------------------------------------------------
0 | _model                        | AutoEncoder         | 910 K 
1 | _AeAndVaeMixin__recon_handler | ReconLossHandlerMse | 0     
----------------------------------------------------------------------

Training: 0it [00:00, ?it/s]

/opt/conda/lib/python3.9/site-packages/pytorch_lightning/callbacks/progress/base.py:247: UserWarning: The progress bar already tracks a metric with the name(s) 'loss' and `self.log('loss', ..., prog_bar=True)` will overwrite this value.  If this is undesired, change the name or override `get_metrics()` in the progress bar callback.
  rank_zero_warn(


trainer.current_epoch is now 5
beta was, 1
beta is now, 2


`Trainer.fit` stopped: `max_epochs=10` reached.



 Train end and beta NOW =  2
{'dci.informativeness_train': 0.9765, 'dci.informativeness_test': 0.6855, 'dci.disentanglement': 0.5314044110791448, 'dci.completeness': 0.557302093311387, 'mig.discrete_score': 0.4585289565787226, 'sap.score': 0.0232}


In [1]:
import pytorch_lightning as pl
from pytorch_lightning.loggers.wandb import WandbLogger
from torch.utils.data import DataLoader
from disent.dataset import DisentDataset
from disent.dataset.data import XYObjectData
from disent.dataset.sampling import GroundTruthPairOrigSampler
from disent.frameworks.ae import Ae
from disent.frameworks.vae import BetaVae, AdaVae
from disent.model import AutoEncoder
from disent.model.ae import DecoderConv64, EncoderConv64
from disent.dataset.transform import ToImgTensorF32
from disent.util import is_test_run  # you can ignore and remove this
from disent.metrics import metric_dci, metric_mig, metric_sap 
import torch
import matplotlib.pyplot as plt
import numpy as np
import wandb
import callback as tc

wandb.init()
# prepare the data
data = XYObjectData()
dataset = DisentDataset(data, GroundTruthPairOrigSampler(), transform=ToImgTensorF32())
dataloader = DataLoader(dataset=dataset, batch_size=128, shuffle=True)

# create the pytorch lightning system
module: pl.LightningModule = AdaVae(
    model=AutoEncoder(
        encoder=EncoderConv64(x_shape=data.x_shape, z_size=6, z_multiplier=2),
        decoder=DecoderConv64(x_shape=data.x_shape, z_size=6),
    ),
    cfg=AdaVae.cfg(
        optimizer='adam', optimizer_kwargs=dict(lr=1e-3),
        loss_reduction='mean_sum', beta=1, ada_average_mode='gvae', ada_thresh_mode='kl',
    )
)
wandb_logger = WandbLogger(project="beta-VAE")
# train the model
trainer = pl.Trainer(logger=wandb_logger, callbacks=[tc.betaControlCallback()], max_epochs = 10, fast_dev_run=is_test_run())
trainer.fit(module, dataloader)

module.eval()

get_repr = lambda x: module.encode(x.to(module.device))

a_results = {
        **metric_dci(dataset, get_repr, num_train=10 if is_test_run() else 1000, num_test=5 if is_test_run() else 500, boost_mode='sklearn'),
        **metric_mig(dataset, get_repr, num_train=20 if is_test_run() else 2000),
        **metric_sap(dataset, get_repr),
    }
print(a_results)
# AdaGVAE beta = 1 at e=7, beta = 2

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: iceburg. Use `wandb login --relogin` to force relogin


/opt/conda/lib/python3.9/site-packages/pytorch_lightning/loggers/wandb.py:351: UserWarning: There is a wandb run already in progress and newly created instances of `WandbLogger` will reuse this run. If this is not desired, call `wandb.finish()` before instantiating `WandbLogger`.
  rank_zero_warn(
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/opt/conda/lib/python3.9/site-packages/pytorch_lightning/trainer/configuration_validator.py:117: PossibleUserWarning: You defined a `validation_step` but have no `val_dataloader`. Skipping val loop.
  rank_zero_warn(

  | Name                          | Type                | Params
----------------------------------------------------------------------
0 | _model                        | AutoEncoder         | 910 K 
1 | _AeAndVaeMixin__recon_handler | ReconLossHandlerMse | 0     
----------------------------------------------------------------------

Training: 0it [00:00, ?it/s]

/opt/conda/lib/python3.9/site-packages/pytorch_lightning/callbacks/progress/base.py:247: UserWarning: The progress bar already tracks a metric with the name(s) 'loss' and `self.log('loss', ..., prog_bar=True)` will overwrite this value.  If this is undesired, change the name or override `get_metrics()` in the progress bar callback.
  rank_zero_warn(


trainer.current_epoch is now 7
beta was, 1
beta is now, 2


`Trainer.fit` stopped: `max_epochs=10` reached.



 Train end and beta NOW =  2
{'dci.informativeness_train': 0.97875, 'dci.informativeness_test': 0.7124999999999999, 'dci.disentanglement': 0.5682242378857836, 'dci.completeness': 0.6198013990956245, 'mig.discrete_score': 0.4851480340785487, 'sap.score': 0.03545}


In [1]:
import pytorch_lightning as pl
from pytorch_lightning.loggers.wandb import WandbLogger
from torch.utils.data import DataLoader
from disent.dataset import DisentDataset
from disent.dataset.data import XYObjectData
from disent.dataset.sampling import GroundTruthPairOrigSampler
from disent.frameworks.ae import Ae
from disent.frameworks.vae import BetaVae, AdaVae
from disent.model import AutoEncoder
from disent.model.ae import DecoderConv64, EncoderConv64
from disent.dataset.transform import ToImgTensorF32
from disent.util import is_test_run  # you can ignore and remove this
from disent.metrics import metric_dci, metric_mig, metric_sap 
import torch
import matplotlib.pyplot as plt
import numpy as np
import wandb
import callback as tc

wandb.init()
# prepare the data
data = XYObjectData()
dataset = DisentDataset(data, GroundTruthPairOrigSampler(), transform=ToImgTensorF32())
dataloader = DataLoader(dataset=dataset, batch_size=128, shuffle=True)

# create the pytorch lightning system
module: pl.LightningModule = AdaVae(
    model=AutoEncoder(
        encoder=EncoderConv64(x_shape=data.x_shape, z_size=6, z_multiplier=2),
        decoder=DecoderConv64(x_shape=data.x_shape, z_size=6),
    ),
    cfg=AdaVae.cfg(
        optimizer='adam', optimizer_kwargs=dict(lr=1e-3),
        loss_reduction='mean_sum', beta=1, ada_average_mode='gvae', ada_thresh_mode='kl',
    )
)
wandb_logger = WandbLogger(project="beta-VAE")
# train the model
trainer = pl.Trainer(logger=wandb_logger, callbacks=[tc.betaControlCallback()], max_epochs = 10, fast_dev_run=is_test_run())
trainer.fit(module, dataloader)

module.eval()

get_repr = lambda x: module.encode(x.to(module.device))

a_results = {
        **metric_dci(dataset, get_repr, num_train=10 if is_test_run() else 1000, num_test=5 if is_test_run() else 500, boost_mode='sklearn'),
        **metric_mig(dataset, get_repr, num_train=20 if is_test_run() else 2000),
        **metric_sap(dataset, get_repr),
    }
print(a_results)
# AdaGVAE beta = 1 at e=9, beta = 2

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: iceburg. Use `wandb login --relogin` to force relogin


/opt/conda/lib/python3.9/site-packages/pytorch_lightning/loggers/wandb.py:351: UserWarning: There is a wandb run already in progress and newly created instances of `WandbLogger` will reuse this run. If this is not desired, call `wandb.finish()` before instantiating `WandbLogger`.
  rank_zero_warn(
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/opt/conda/lib/python3.9/site-packages/pytorch_lightning/trainer/configuration_validator.py:117: PossibleUserWarning: You defined a `validation_step` but have no `val_dataloader`. Skipping val loop.
  rank_zero_warn(

  | Name                          | Type                | Params
----------------------------------------------------------------------
0 | _model                        | AutoEncoder         | 910 K 
1 | _AeAndVaeMixin__recon_handler | ReconLossHandlerMse | 0     
----------------------------------------------------------------------

Training: 0it [00:00, ?it/s]

/opt/conda/lib/python3.9/site-packages/pytorch_lightning/callbacks/progress/base.py:247: UserWarning: The progress bar already tracks a metric with the name(s) 'loss' and `self.log('loss', ..., prog_bar=True)` will overwrite this value.  If this is undesired, change the name or override `get_metrics()` in the progress bar callback.
  rank_zero_warn(


trainer.current_epoch is now 9
beta was, 1
beta is now, 2


`Trainer.fit` stopped: `max_epochs=10` reached.



 Train end and beta NOW =  2
{'dci.informativeness_train': 0.977, 'dci.informativeness_test': 0.6739999999999999, 'dci.disentanglement': 0.5273395192425567, 'dci.completeness': 0.5735073687737746, 'mig.discrete_score': 0.4557159689149634, 'sap.score': 0.030800000000000008}


In [1]:
import pytorch_lightning as pl
from pytorch_lightning.loggers.wandb import WandbLogger
from torch.utils.data import DataLoader
from disent.dataset import DisentDataset
from disent.dataset.data import XYObjectData
from disent.dataset.sampling import GroundTruthPairOrigSampler
from disent.frameworks.ae import Ae
from disent.frameworks.vae import BetaVae, AdaVae
from disent.model import AutoEncoder
from disent.model.ae import DecoderConv64, EncoderConv64
from disent.dataset.transform import ToImgTensorF32
from disent.util import is_test_run  # you can ignore and remove this
from disent.metrics import metric_dci, metric_mig, metric_sap 
import torch
import matplotlib.pyplot as plt
import numpy as np
import wandb
import callback as tc

wandb.init()
# prepare the data
data = XYObjectData()
dataset = DisentDataset(data, GroundTruthPairOrigSampler(), transform=ToImgTensorF32())
dataloader = DataLoader(dataset=dataset, batch_size=128, shuffle=True)

# create the pytorch lightning system
module: pl.LightningModule = AdaVae(
    model=AutoEncoder(
        encoder=EncoderConv64(x_shape=data.x_shape, z_size=6, z_multiplier=2),
        decoder=DecoderConv64(x_shape=data.x_shape, z_size=6),
    ),
    cfg=AdaVae.cfg(
        optimizer='adam', optimizer_kwargs=dict(lr=1e-3),
        loss_reduction='mean_sum', beta=2, ada_average_mode='gvae', ada_thresh_mode='kl',
    )
)
wandb_logger = WandbLogger(project="beta-VAE")
# train the model
trainer = pl.Trainer(logger=wandb_logger, callbacks=[tc.betaControlCallback()], max_epochs = 10, fast_dev_run=is_test_run())
trainer.fit(module, dataloader)

module.eval()

get_repr = lambda x: module.encode(x.to(module.device))

a_results = {
        **metric_dci(dataset, get_repr, num_train=10 if is_test_run() else 1000, num_test=5 if is_test_run() else 500, boost_mode='sklearn'),
        **metric_mig(dataset, get_repr, num_train=20 if is_test_run() else 2000),
        **metric_sap(dataset, get_repr),
    }
print(a_results)
# AdaGVAE beta = 2 at e=1, beta = 1

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: iceburg. Use `wandb login --relogin` to force relogin


/opt/conda/lib/python3.9/site-packages/pytorch_lightning/loggers/wandb.py:351: UserWarning: There is a wandb run already in progress and newly created instances of `WandbLogger` will reuse this run. If this is not desired, call `wandb.finish()` before instantiating `WandbLogger`.
  rank_zero_warn(
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/opt/conda/lib/python3.9/site-packages/pytorch_lightning/trainer/configuration_validator.py:117: PossibleUserWarning: You defined a `validation_step` but have no `val_dataloader`. Skipping val loop.
  rank_zero_warn(

  | Name                          | Type                | Params
----------------------------------------------------------------------
0 | _model                        | AutoEncoder         | 910 K 
1 | _AeAndVaeMixin__recon_handler | ReconLossHandlerMse | 0     
----------------------------------------------------------------------

Training: 0it [00:00, ?it/s]

/opt/conda/lib/python3.9/site-packages/pytorch_lightning/callbacks/progress/base.py:247: UserWarning: The progress bar already tracks a metric with the name(s) 'loss' and `self.log('loss', ..., prog_bar=True)` will overwrite this value.  If this is undesired, change the name or override `get_metrics()` in the progress bar callback.
  rank_zero_warn(


trainer.current_epoch is now 1
beta was, 2
beta is now, 1


`Trainer.fit` stopped: `max_epochs=10` reached.



 Train end and beta NOW =  1
{'dci.informativeness_train': 0.96975, 'dci.informativeness_test': 0.6440000000000001, 'dci.disentanglement': 0.4710847500972034, 'dci.completeness': 0.5294901956193412, 'mig.discrete_score': 0.4406668568409308, 'sap.score': 0.01465}


In [1]:
import pytorch_lightning as pl
from pytorch_lightning.loggers.wandb import WandbLogger
from torch.utils.data import DataLoader
from disent.dataset import DisentDataset
from disent.dataset.data import XYObjectData
from disent.dataset.sampling import GroundTruthPairOrigSampler
from disent.frameworks.ae import Ae
from disent.frameworks.vae import BetaVae, AdaVae
from disent.model import AutoEncoder
from disent.model.ae import DecoderConv64, EncoderConv64
from disent.dataset.transform import ToImgTensorF32
from disent.util import is_test_run  # you can ignore and remove this
from disent.metrics import metric_dci, metric_mig, metric_sap 
import torch
import matplotlib.pyplot as plt
import numpy as np
import wandb
import callback as tc

wandb.init()
# prepare the data
data = XYObjectData()
dataset = DisentDataset(data, GroundTruthPairOrigSampler(), transform=ToImgTensorF32())
dataloader = DataLoader(dataset=dataset, batch_size=128, shuffle=True)

# create the pytorch lightning system
module: pl.LightningModule = AdaVae(
    model=AutoEncoder(
        encoder=EncoderConv64(x_shape=data.x_shape, z_size=6, z_multiplier=2),
        decoder=DecoderConv64(x_shape=data.x_shape, z_size=6),
    ),
    cfg=AdaVae.cfg(
        optimizer='adam', optimizer_kwargs=dict(lr=1e-3),
        loss_reduction='mean_sum', beta=0, ada_average_mode='gvae', ada_thresh_mode='kl',
    )
)
wandb_logger = WandbLogger(project="beta-VAE")
# train the model
trainer = pl.Trainer(logger=wandb_logger, callbacks=[tc.betaControlCallback()], max_epochs = 10, fast_dev_run=is_test_run())
trainer.fit(module, dataloader)

module.eval()

get_repr = lambda x: module.encode(x.to(module.device))

a_results = {
        **metric_dci(dataset, get_repr, num_train=10 if is_test_run() else 1000, num_test=5 if is_test_run() else 500, boost_mode='sklearn'),
        **metric_mig(dataset, get_repr, num_train=20 if is_test_run() else 2000),
        **metric_sap(dataset, get_repr),
    }
print(a_results)
# AdaGVAE beta = 0 at e=5, beta = 1

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: iceburg. Use `wandb login --relogin` to force relogin


/opt/conda/lib/python3.9/site-packages/pytorch_lightning/loggers/wandb.py:351: UserWarning: There is a wandb run already in progress and newly created instances of `WandbLogger` will reuse this run. If this is not desired, call `wandb.finish()` before instantiating `WandbLogger`.
  rank_zero_warn(
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/opt/conda/lib/python3.9/site-packages/pytorch_lightning/trainer/configuration_validator.py:117: PossibleUserWarning: You defined a `validation_step` but have no `val_dataloader`. Skipping val loop.
  rank_zero_warn(

  | Name                          | Type                | Params
----------------------------------------------------------------------
0 | _model                        | AutoEncoder         | 910 K 
1 | _AeAndVaeMixin__recon_handler | ReconLossHandlerMse | 0     
----------------------------------------------------------------------

Training: 0it [00:00, ?it/s]

/opt/conda/lib/python3.9/site-packages/pytorch_lightning/callbacks/progress/base.py:247: UserWarning: The progress bar already tracks a metric with the name(s) 'loss' and `self.log('loss', ..., prog_bar=True)` will overwrite this value.  If this is undesired, change the name or override `get_metrics()` in the progress bar callback.
  rank_zero_warn(


trainer.current_epoch is now 5
beta was, 0
beta is now, 1


`Trainer.fit` stopped: `max_epochs=10` reached.



 Train end and beta NOW =  1
{'dci.informativeness_train': 0.99825, 'dci.informativeness_test': 0.8450000000000001, 'dci.disentanglement': 0.4133796563133506, 'dci.completeness': 0.34639792008111236, 'mig.discrete_score': 0.14527725061784266, 'sap.score': 0.006149999999999999}


In [1]:
import pytorch_lightning as pl
from pytorch_lightning.loggers.wandb import WandbLogger
from torch.utils.data import DataLoader
from disent.dataset import DisentDataset
from disent.dataset.data import XYObjectData
from disent.dataset.sampling import GroundTruthPairOrigSampler
from disent.frameworks.ae import Ae
from disent.frameworks.vae import BetaVae, AdaVae
from disent.model import AutoEncoder
from disent.model.ae import DecoderConv64, EncoderConv64
from disent.dataset.transform import ToImgTensorF32
from disent.util import is_test_run  # you can ignore and remove this
from disent.metrics import metric_dci, metric_mig, metric_sap 
import torch
import matplotlib.pyplot as plt
import numpy as np
import wandb
import callback as tc

wandb.init()
# prepare the data
data = XYObjectData()
dataset = DisentDataset(data, GroundTruthPairOrigSampler(), transform=ToImgTensorF32())
dataloader = DataLoader(dataset=dataset, batch_size=128, shuffle=True)

# create the pytorch lightning system
module: pl.LightningModule = AdaVae(
    model=AutoEncoder(
        encoder=EncoderConv64(x_shape=data.x_shape, z_size=6, z_multiplier=2),
        decoder=DecoderConv64(x_shape=data.x_shape, z_size=6),
    ),
    cfg=AdaVae.cfg(
        optimizer='adam', optimizer_kwargs=dict(lr=1e-3),
        loss_reduction='mean_sum', beta=1, ada_average_mode='gvae', ada_thresh_mode='kl',
    )
)
wandb_logger = WandbLogger(project="beta-VAE")
# train the model
trainer = pl.Trainer(logger=wandb_logger, callbacks=[tc.betaControlCallback()], max_epochs = 10, fast_dev_run=is_test_run())
trainer.fit(module, dataloader)

module.eval()

get_repr = lambda x: module.encode(x.to(module.device))

a_results = {
        **metric_dci(dataset, get_repr, num_train=10 if is_test_run() else 1000, num_test=5 if is_test_run() else 500, boost_mode='sklearn'),
        **metric_mig(dataset, get_repr, num_train=20 if is_test_run() else 2000),
        **metric_sap(dataset, get_repr),
    }
print(a_results)
# AdaGVAE beta = 1 at e=5, beta = 0

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: iceburg. Use `wandb login --relogin` to force relogin


/opt/conda/lib/python3.9/site-packages/pytorch_lightning/loggers/wandb.py:351: UserWarning: There is a wandb run already in progress and newly created instances of `WandbLogger` will reuse this run. If this is not desired, call `wandb.finish()` before instantiating `WandbLogger`.
  rank_zero_warn(
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/opt/conda/lib/python3.9/site-packages/pytorch_lightning/trainer/configuration_validator.py:117: PossibleUserWarning: You defined a `validation_step` but have no `val_dataloader`. Skipping val loop.
  rank_zero_warn(

  | Name                          | Type                | Params
----------------------------------------------------------------------
0 | _model                        | AutoEncoder         | 910 K 
1 | _AeAndVaeMixin__recon_handler | ReconLossHandlerMse | 0     
----------------------------------------------------------------------

Training: 0it [00:00, ?it/s]

/opt/conda/lib/python3.9/site-packages/pytorch_lightning/callbacks/progress/base.py:247: UserWarning: The progress bar already tracks a metric with the name(s) 'loss' and `self.log('loss', ..., prog_bar=True)` will overwrite this value.  If this is undesired, change the name or override `get_metrics()` in the progress bar callback.
  rank_zero_warn(


trainer.current_epoch is now 5
beta was, 1
beta is now, 0


`Trainer.fit` stopped: `max_epochs=10` reached.



 Train end and beta NOW =  0
{'dci.informativeness_train': 0.98575, 'dci.informativeness_test': 0.777, 'dci.disentanglement': 0.5336665616375376, 'dci.completeness': 0.48562076827643785, 'mig.discrete_score': 0.36161075332609516, 'sap.score': 0.016950000000000007}


In [1]:
import pytorch_lightning as pl
from pytorch_lightning.loggers.wandb import WandbLogger
from torch.utils.data import DataLoader
from disent.dataset import DisentDataset
from disent.dataset.data import XYObjectData
from disent.dataset.sampling import GroundTruthPairOrigSampler
from disent.frameworks.ae import Ae
from disent.frameworks.vae import BetaVae, AdaVae
from disent.model import AutoEncoder
from disent.model.ae import DecoderConv64, EncoderConv64
from disent.dataset.transform import ToImgTensorF32
from disent.util import is_test_run  # you can ignore and remove this
from disent.metrics import metric_dci, metric_mig, metric_sap 
import torch
import matplotlib.pyplot as plt
import numpy as np
import wandb
import callback as tc

wandb.init()
# prepare the data
data = XYObjectData()
dataset = DisentDataset(data, GroundTruthPairOrigSampler(), transform=ToImgTensorF32())
dataloader = DataLoader(dataset=dataset, batch_size=128, shuffle=True)

# create the pytorch lightning system
module: pl.LightningModule = AdaVae(
    model=AutoEncoder(
        encoder=EncoderConv64(x_shape=data.x_shape, z_size=6, z_multiplier=2),
        decoder=DecoderConv64(x_shape=data.x_shape, z_size=6),
    ),
    cfg=AdaVae.cfg(
        optimizer='adam', optimizer_kwargs=dict(lr=1e-3),
        loss_reduction='mean_sum', beta=2, ada_average_mode='gvae', ada_thresh_mode='kl',
    )
)
wandb_logger = WandbLogger(project="beta-VAE")
# train the model
trainer = pl.Trainer(logger=wandb_logger, callbacks=[tc.betaControlCallback()], max_epochs = 10, fast_dev_run=is_test_run())
trainer.fit(module, dataloader)

module.eval()

get_repr = lambda x: module.encode(x.to(module.device))

a_results = {
        **metric_dci(dataset, get_repr, num_train=10 if is_test_run() else 1000, num_test=5 if is_test_run() else 500, boost_mode='sklearn'),
        **metric_mig(dataset, get_repr, num_train=20 if is_test_run() else 2000),
        **metric_sap(dataset, get_repr),
    }
print(a_results)
# AdaGVAE beta = 2 at e=3, beta = 1

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: iceburg. Use `wandb login --relogin` to force relogin


/opt/conda/lib/python3.9/site-packages/pytorch_lightning/loggers/wandb.py:351: UserWarning: There is a wandb run already in progress and newly created instances of `WandbLogger` will reuse this run. If this is not desired, call `wandb.finish()` before instantiating `WandbLogger`.
  rank_zero_warn(
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/opt/conda/lib/python3.9/site-packages/pytorch_lightning/trainer/configuration_validator.py:117: PossibleUserWarning: You defined a `validation_step` but have no `val_dataloader`. Skipping val loop.
  rank_zero_warn(

  | Name                          | Type                | Params
----------------------------------------------------------------------
0 | _model                        | AutoEncoder         | 910 K 
1 | _AeAndVaeMixin__recon_handler | ReconLossHandlerMse | 0     
----------------------------------------------------------------------

Training: 0it [00:00, ?it/s]

/opt/conda/lib/python3.9/site-packages/pytorch_lightning/callbacks/progress/base.py:247: UserWarning: The progress bar already tracks a metric with the name(s) 'loss' and `self.log('loss', ..., prog_bar=True)` will overwrite this value.  If this is undesired, change the name or override `get_metrics()` in the progress bar callback.
  rank_zero_warn(


trainer.current_epoch is now 3
beta was, 2
beta is now, 1


`Trainer.fit` stopped: `max_epochs=10` reached.



 Train end and beta NOW =  1
{'dci.informativeness_train': 0.9695, 'dci.informativeness_test': 0.6155, 'dci.disentanglement': 0.3455165704779406, 'dci.completeness': 0.3418883665189123, 'mig.discrete_score': 0.1818126425060103, 'sap.score': 0.016600000000000007}


In [1]:
import pytorch_lightning as pl
from pytorch_lightning.loggers.wandb import WandbLogger
from torch.utils.data import DataLoader
from disent.dataset import DisentDataset
from disent.dataset.data import XYObjectData
from disent.dataset.sampling import GroundTruthPairOrigSampler
from disent.frameworks.ae import Ae
from disent.frameworks.vae import BetaVae, AdaVae
from disent.model import AutoEncoder
from disent.model.ae import DecoderConv64, EncoderConv64
from disent.dataset.transform import ToImgTensorF32
from disent.util import is_test_run  # you can ignore and remove this
from disent.metrics import metric_dci, metric_mig, metric_sap 
import torch
import matplotlib.pyplot as plt
import numpy as np
import wandb
import callback as tc

wandb.init()
# prepare the data
data = XYObjectData()
dataset = DisentDataset(data, GroundTruthPairOrigSampler(), transform=ToImgTensorF32())
dataloader = DataLoader(dataset=dataset, batch_size=128, shuffle=True)

# create the pytorch lightning system
module: pl.LightningModule = AdaVae(
    model=AutoEncoder(
        encoder=EncoderConv64(x_shape=data.x_shape, z_size=6, z_multiplier=2),
        decoder=DecoderConv64(x_shape=data.x_shape, z_size=6),
    ),
    cfg=AdaVae.cfg(
        optimizer='adam', optimizer_kwargs=dict(lr=1e-3),
        loss_reduction='mean_sum', beta=2, ada_average_mode='gvae', ada_thresh_mode='kl',
    )
)
wandb_logger = WandbLogger(project="beta-VAE")
# train the model
trainer = pl.Trainer(logger=wandb_logger, callbacks=[tc.betaControlCallback()], max_epochs = 10, fast_dev_run=is_test_run())
trainer.fit(module, dataloader)

module.eval()

get_repr = lambda x: module.encode(x.to(module.device))

a_results = {
        **metric_dci(dataset, get_repr, num_train=10 if is_test_run() else 1000, num_test=5 if is_test_run() else 500, boost_mode='sklearn'),
        **metric_mig(dataset, get_repr, num_train=20 if is_test_run() else 2000),
        **metric_sap(dataset, get_repr),
    }
print(a_results)
# AdaGVAE beta = 2 at e=5, beta = 1

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: iceburg. Use `wandb login --relogin` to force relogin


/opt/conda/lib/python3.9/site-packages/pytorch_lightning/loggers/wandb.py:351: UserWarning: There is a wandb run already in progress and newly created instances of `WandbLogger` will reuse this run. If this is not desired, call `wandb.finish()` before instantiating `WandbLogger`.
  rank_zero_warn(
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/opt/conda/lib/python3.9/site-packages/pytorch_lightning/trainer/configuration_validator.py:117: PossibleUserWarning: You defined a `validation_step` but have no `val_dataloader`. Skipping val loop.
  rank_zero_warn(

  | Name                          | Type                | Params
----------------------------------------------------------------------
0 | _model                        | AutoEncoder         | 910 K 
1 | _AeAndVaeMixin__recon_handler | ReconLossHandlerMse | 0     
----------------------------------------------------------------------

Training: 0it [00:00, ?it/s]

/opt/conda/lib/python3.9/site-packages/pytorch_lightning/callbacks/progress/base.py:247: UserWarning: The progress bar already tracks a metric with the name(s) 'loss' and `self.log('loss', ..., prog_bar=True)` will overwrite this value.  If this is undesired, change the name or override `get_metrics()` in the progress bar callback.
  rank_zero_warn(


trainer.current_epoch is now 5
beta was, 2
beta is now, 1


`Trainer.fit` stopped: `max_epochs=10` reached.



 Train end and beta NOW =  1
{'dci.informativeness_train': 0.968, 'dci.informativeness_test': 0.462, 'dci.disentanglement': 0.18318570226026765, 'dci.completeness': 0.17067280835321372, 'mig.discrete_score': 0.10233032107411798, 'sap.score': 0.02085000000000001}


wandb: Network error (ReadTimeout), entering retry loop.
wandb: Network error (ReadTimeout), entering retry loop.
wandb: Network error (ReadTimeout), entering retry loop.
wandb: Network error (ReadTimeout), entering retry loop.
wandb: Network error (ReadTimeout), entering retry loop.
wandb: Network error (ReadTimeout), entering retry loop.
wandb: Network error (ReadTimeout), entering retry loop.
wandb: Network error (ReadTimeout), entering retry loop.


In [1]:
import pytorch_lightning as pl
from pytorch_lightning.loggers.wandb import WandbLogger
from torch.utils.data import DataLoader
from disent.dataset import DisentDataset
from disent.dataset.data import XYObjectData
from disent.dataset.sampling import GroundTruthPairOrigSampler
from disent.frameworks.ae import Ae
from disent.frameworks.vae import BetaVae, AdaVae
from disent.model import AutoEncoder
from disent.model.ae import DecoderConv64, EncoderConv64
from disent.dataset.transform import ToImgTensorF32
from disent.util import is_test_run  # you can ignore and remove this
from disent.metrics import metric_dci, metric_mig, metric_sap 
import torch
import matplotlib.pyplot as plt
import numpy as np
import wandb
import callback as tc

wandb.init()
# prepare the data
data = XYObjectData()
dataset = DisentDataset(data, GroundTruthPairOrigSampler(), transform=ToImgTensorF32())
dataloader = DataLoader(dataset=dataset, batch_size=128, shuffle=True)

# create the pytorch lightning system
module: pl.LightningModule = AdaVae(
    model=AutoEncoder(
        encoder=EncoderConv64(x_shape=data.x_shape, z_size=6, z_multiplier=2),
        decoder=DecoderConv64(x_shape=data.x_shape, z_size=6),
    ),
    cfg=AdaVae.cfg(
        optimizer='adam', optimizer_kwargs=dict(lr=1e-3),
        loss_reduction='mean_sum', beta=2, ada_average_mode='gvae', ada_thresh_mode='kl',
    )
)
wandb_logger = WandbLogger(project="beta-VAE")
# train the model
trainer = pl.Trainer(logger=wandb_logger, callbacks=[tc.betaControlCallback()], max_epochs = 10, fast_dev_run=is_test_run())
trainer.fit(module, dataloader)

module.eval()

get_repr = lambda x: module.encode(x.to(module.device))

a_results = {
        **metric_dci(dataset, get_repr, num_train=10 if is_test_run() else 1000, num_test=5 if is_test_run() else 500, boost_mode='sklearn'),
        **metric_mig(dataset, get_repr, num_train=20 if is_test_run() else 2000),
        **metric_sap(dataset, get_repr),
    }
print(a_results)
# AdaGVAE beta = 2 at e=7, beta = 1

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: iceburg. Use `wandb login --relogin` to force relogin


/opt/conda/lib/python3.9/site-packages/pytorch_lightning/loggers/wandb.py:351: UserWarning: There is a wandb run already in progress and newly created instances of `WandbLogger` will reuse this run. If this is not desired, call `wandb.finish()` before instantiating `WandbLogger`.
  rank_zero_warn(
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/opt/conda/lib/python3.9/site-packages/pytorch_lightning/trainer/configuration_validator.py:117: PossibleUserWarning: You defined a `validation_step` but have no `val_dataloader`. Skipping val loop.
  rank_zero_warn(

  | Name                          | Type                | Params
----------------------------------------------------------------------
0 | _model                        | AutoEncoder         | 910 K 
1 | _AeAndVaeMixin__recon_handler | ReconLossHandlerMse | 0     
----------------------------------------------------------------------

Training: 0it [00:00, ?it/s]

/opt/conda/lib/python3.9/site-packages/pytorch_lightning/callbacks/progress/base.py:247: UserWarning: The progress bar already tracks a metric with the name(s) 'loss' and `self.log('loss', ..., prog_bar=True)` will overwrite this value.  If this is undesired, change the name or override `get_metrics()` in the progress bar callback.
  rank_zero_warn(


trainer.current_epoch is now 7
beta was, 2
beta is now, 1


`Trainer.fit` stopped: `max_epochs=10` reached.



 Train end and beta NOW =  1
{'dci.informativeness_train': 0.95625, 'dci.informativeness_test': 0.2275, 'dci.disentanglement': 0.011082880679038436, 'dci.completeness': 0.009629510037555272, 'mig.discrete_score': 0.009393038458709943, 'sap.score': 0.0}


In [1]:
import pytorch_lightning as pl
from pytorch_lightning.loggers.wandb import WandbLogger
from torch.utils.data import DataLoader
from disent.dataset import DisentDataset
from disent.dataset.data import XYObjectData
from disent.dataset.sampling import GroundTruthPairOrigSampler
from disent.frameworks.ae import Ae
from disent.frameworks.vae import BetaVae, AdaVae
from disent.model import AutoEncoder
from disent.model.ae import DecoderConv64, EncoderConv64
from disent.dataset.transform import ToImgTensorF32
from disent.util import is_test_run  # you can ignore and remove this
from disent.metrics import metric_dci, metric_mig, metric_sap 
import torch
import matplotlib.pyplot as plt
import numpy as np
import wandb
import callback as tc

wandb.init()
# prepare the data
data = XYObjectData()
dataset = DisentDataset(data, GroundTruthPairOrigSampler(), transform=ToImgTensorF32())
dataloader = DataLoader(dataset=dataset, batch_size=128, shuffle=True)

# create the pytorch lightning system
module: pl.LightningModule = AdaVae(
    model=AutoEncoder(
        encoder=EncoderConv64(x_shape=data.x_shape, z_size=6, z_multiplier=2),
        decoder=DecoderConv64(x_shape=data.x_shape, z_size=6),
    ),
    cfg=AdaVae.cfg(
        optimizer='adam', optimizer_kwargs=dict(lr=1e-3),
        loss_reduction='mean_sum', beta=2, ada_average_mode='gvae', ada_thresh_mode='kl',
    )
)
wandb_logger = WandbLogger(project="beta-VAE")
# train the model
trainer = pl.Trainer(logger=wandb_logger, callbacks=[tc.betaControlCallback()], max_epochs = 10, fast_dev_run=is_test_run())
trainer.fit(module, dataloader)

module.eval()

get_repr = lambda x: module.encode(x.to(module.device))

a_results = {
        **metric_dci(dataset, get_repr, num_train=10 if is_test_run() else 1000, num_test=5 if is_test_run() else 500, boost_mode='sklearn'),
        **metric_mig(dataset, get_repr, num_train=20 if is_test_run() else 2000),
        **metric_sap(dataset, get_repr),
    }
print(a_results)
# AdaGVAE beta = 2 at e=9, beta = 1

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: iceburg. Use `wandb login --relogin` to force relogin


/opt/conda/lib/python3.9/site-packages/pytorch_lightning/loggers/wandb.py:351: UserWarning: There is a wandb run already in progress and newly created instances of `WandbLogger` will reuse this run. If this is not desired, call `wandb.finish()` before instantiating `WandbLogger`.
  rank_zero_warn(
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/opt/conda/lib/python3.9/site-packages/pytorch_lightning/trainer/configuration_validator.py:117: PossibleUserWarning: You defined a `validation_step` but have no `val_dataloader`. Skipping val loop.
  rank_zero_warn(

  | Name                          | Type                | Params
----------------------------------------------------------------------
0 | _model                        | AutoEncoder         | 910 K 
1 | _AeAndVaeMixin__recon_handler | ReconLossHandlerMse | 0     
----------------------------------------------------------------------

Training: 0it [00:00, ?it/s]

/opt/conda/lib/python3.9/site-packages/pytorch_lightning/callbacks/progress/base.py:247: UserWarning: The progress bar already tracks a metric with the name(s) 'loss' and `self.log('loss', ..., prog_bar=True)` will overwrite this value.  If this is undesired, change the name or override `get_metrics()` in the progress bar callback.
  rank_zero_warn(


trainer.current_epoch is now 9
beta was, 2
beta is now, 1


`Trainer.fit` stopped: `max_epochs=10` reached.



 Train end and beta NOW =  1
{'dci.informativeness_train': 0.96275, 'dci.informativeness_test': 0.5760000000000001, 'dci.disentanglement': 0.32345104416905773, 'dci.completeness': 0.3451440374172538, 'mig.discrete_score': 0.1983131710741087, 'sap.score': 0.0049500000000000065}


In [2]:
import pytorch_lightning as pl
from pytorch_lightning.loggers.wandb import WandbLogger
from torch.utils.data import DataLoader
from disent.dataset import DisentDataset
from disent.dataset.data import XYObjectData
from disent.dataset.sampling import GroundTruthPairOrigSampler
from disent.frameworks.ae import Ae
from disent.frameworks.vae import BetaVae, AdaVae
from disent.model import AutoEncoder
from disent.model.ae import DecoderConv64, EncoderConv64
from disent.dataset.transform import ToImgTensorF32
from disent.util import is_test_run  # you can ignore and remove this
from disent.metrics import metric_dci, metric_mig, metric_sap 
import torch
import matplotlib.pyplot as plt
import numpy as np
import wandb
import callback as tc

wandb.init()
# prepare the data
data = XYObjectData()
dataset = DisentDataset(data, GroundTruthPairOrigSampler(), transform=ToImgTensorF32())
dataloader = DataLoader(dataset=dataset, batch_size=128, shuffle=True)

# create the pytorch lightning system
module: pl.LightningModule = AdaVae(
    model=AutoEncoder(
        encoder=EncoderConv64(x_shape=data.x_shape, z_size=6, z_multiplier=2),
        decoder=DecoderConv64(x_shape=data.x_shape, z_size=6),
    ),
    cfg=AdaVae.cfg(
        optimizer='adam', optimizer_kwargs=dict(lr=1e-3),
        loss_reduction='mean_sum', beta=1, ada_average_mode='gvae', ada_thresh_mode='kl',
    )
)
wandb_logger = WandbLogger(project="beta-VAE")
# train the model
trainer = pl.Trainer(logger=wandb_logger, callbacks=[tc.betaControlCallback()], max_epochs = 10, fast_dev_run=is_test_run())
trainer.fit(module, dataloader)

module.eval()

get_repr = lambda x: module.encode(x.to(module.device))

a_results = {
        **metric_dci(dataset, get_repr, num_train=10 if is_test_run() else 1000, num_test=5 if is_test_run() else 500, boost_mode='sklearn'),
        **metric_mig(dataset, get_repr, num_train=20 if is_test_run() else 2000),
        **metric_sap(dataset, get_repr),
    }
print(a_results)
# AdaGVAE beta = 1 at e=1, beta = 0 5有了

aug_loss,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch,▁▁▁▁▁▁▁▁▁▃▃▃▃▃▃▃▃▃▃▅▅▅▅▅▅▅▅▅▆▆▆▆▆▆▆▆▆███
kl_loss,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁█
kl_reg_loss,█▇▆▆▄▄▃▃▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
loss,▄▃▄▄▃▅▂▅▄▄▅▂▅▅▃▃▄▆▃▃▄▃▃▁▃█▁▇▁▂▄▅▇▃▁▂▃▄▁▁
pixel_loss,▄▃▄▄▃▅▂▅▄▄▅▂▅▅▃▃▄▆▃▃▄▃▃▁▃█▁▇▁▂▄▅▇▃▁▂▃▄▁▁
recon_loss,▄▃▄▄▃▅▂▅▄▄▅▂▅▅▃▃▄▆▃▃▄▃▃▁▃█▁▇▁▂▄▅▇▃▁▂▃▄▁▁
reg_loss,█▇▆▆▄▄▃▃▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
shared,▄█▃▄▅▅▅▃▃▃▃▅▅▆▄▆▁▂▁▅▄▅▄▃▃▂▅▄▃▄▄▄▇▂▅▄▇▃▄▅
trainer/global_step,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
aug_loss,0.0


GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs

  | Name                          | Type                | Params
----------------------------------------------------------------------
0 | _model                        | AutoEncoder         | 910 K 
1 | _AeAndVaeMixin__recon_handler | ReconLossHandlerMse | 0     
----------------------------------------------------------------------
910 K     Trainable params
0         Non-trainable params
910 K     Total params
3.643     Total estimated model params size (MB)


Training: 0it [00:00, ?it/s]

trainer.current_epoch is now 1
beta was, 1
beta is now, 0


`Trainer.fit` stopped: `max_epochs=10` reached.



 Train end and beta NOW =  0
{'dci.informativeness_train': 0.98125, 'dci.informativeness_test': 0.672, 'dci.disentanglement': 0.24214961621462538, 'dci.completeness': 0.1996359458164787, 'mig.discrete_score': 0.23532494574945279, 'sap.score': 0.029500000000000005}


In [2]:
import pytorch_lightning as pl
from pytorch_lightning.loggers.wandb import WandbLogger
from torch.utils.data import DataLoader
from disent.dataset import DisentDataset
from disent.dataset.data import XYObjectData
from disent.dataset.sampling import GroundTruthPairOrigSampler
from disent.frameworks.ae import Ae
from disent.frameworks.vae import BetaVae, AdaVae
from disent.model import AutoEncoder
from disent.model.ae import DecoderConv64, EncoderConv64
from disent.dataset.transform import ToImgTensorF32
from disent.util import is_test_run  # you can ignore and remove this
from disent.metrics import metric_dci, metric_mig, metric_sap 
import torch
import matplotlib.pyplot as plt
import numpy as np
import wandb
import callback as tc

wandb.init()
# prepare the data
data = XYObjectData()
dataset = DisentDataset(data, GroundTruthPairOrigSampler(), transform=ToImgTensorF32())
dataloader = DataLoader(dataset=dataset, batch_size=128, shuffle=True)

# create the pytorch lightning system
module: pl.LightningModule = AdaVae(
    model=AutoEncoder(
        encoder=EncoderConv64(x_shape=data.x_shape, z_size=6, z_multiplier=2),
        decoder=DecoderConv64(x_shape=data.x_shape, z_size=6),
    ),
    cfg=AdaVae.cfg(
        optimizer='adam', optimizer_kwargs=dict(lr=1e-3),
        loss_reduction='mean_sum', beta=1, ada_average_mode='gvae', ada_thresh_mode='kl',
    )
)
wandb_logger = WandbLogger(project="beta-VAE")
# train the model
trainer = pl.Trainer(logger=wandb_logger, callbacks=[tc.betaControlCallback()], max_epochs = 10, fast_dev_run=is_test_run())
trainer.fit(module, dataloader)

module.eval()

get_repr = lambda x: module.encode(x.to(module.device))

a_results = {
        **metric_dci(dataset, get_repr, num_train=10 if is_test_run() else 1000, num_test=5 if is_test_run() else 500, boost_mode='sklearn'),
        **metric_mig(dataset, get_repr, num_train=20 if is_test_run() else 2000),
        **metric_sap(dataset, get_repr),
    }
print(a_results)
# AdaGVAE beta = 1 at e=3, beta = 0 5有了

aug_loss,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch,▁▁▁▁▁▁▁▁▁▃▃▃▃▃▃▃▃▃▅▅▅▅▅▅▅▅▅▆▆▆▆▆▆▆▆▆████
kl_loss,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁█
kl_reg_loss,█▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
loss,█▅▃▃▃▅▅▆▂▂▂▁▆▆▃▃▅▂▄▂▅▆▄▁▃▄▇▃▅▆▂▃▃▄▂▃▄▄▁▆
pixel_loss,█▅▃▃▃▅▅▆▂▂▂▁▆▆▃▃▅▂▄▂▅▆▄▁▃▄▇▃▅▆▂▃▃▄▂▃▄▄▁▆
recon_loss,█▅▃▃▃▅▅▆▂▂▂▁▆▆▃▃▅▂▄▂▅▆▄▁▃▄▇▃▅▆▂▃▃▄▂▃▄▄▁▆
reg_loss,█▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
shared,▇▇▄▃▃▃▁▂▂▃▂▂▂▄▁▃▁▂▂▂▁▂▃▁▃▃▁▅▆▆▅▆██▇▇▇▆▆▁
trainer/global_step,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
aug_loss,0.0


GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs

  | Name                          | Type                | Params
----------------------------------------------------------------------
0 | _model                        | AutoEncoder         | 910 K 
1 | _AeAndVaeMixin__recon_handler | ReconLossHandlerMse | 0     
----------------------------------------------------------------------
910 K     Trainable params
0         Non-trainable params
910 K     Total params
3.643     Total estimated model params size (MB)


Training: 0it [00:00, ?it/s]

trainer.current_epoch is now 3
beta was, 1
beta is now, 0


`Trainer.fit` stopped: `max_epochs=10` reached.



 Train end and beta NOW =  0
{'dci.informativeness_train': 0.979, 'dci.informativeness_test': 0.749, 'dci.disentanglement': 0.3881731932500562, 'dci.completeness': 0.32677987681490056, 'mig.discrete_score': 0.25746161816279073, 'sap.score': 0.03065}


In [1]:
import pytorch_lightning as pl
from pytorch_lightning.loggers.wandb import WandbLogger
from torch.utils.data import DataLoader
from disent.dataset import DisentDataset
from disent.dataset.data import XYObjectData
from disent.dataset.sampling import GroundTruthPairOrigSampler
from disent.frameworks.ae import Ae
from disent.frameworks.vae import BetaVae, AdaVae
from disent.model import AutoEncoder
from disent.model.ae import DecoderConv64, EncoderConv64
from disent.dataset.transform import ToImgTensorF32
from disent.util import is_test_run  # you can ignore and remove this
from disent.metrics import metric_dci, metric_mig, metric_sap 
import torch
import matplotlib.pyplot as plt
import numpy as np
import wandb
import callback as tc

wandb.init()
# prepare the data
data = XYObjectData()
dataset = DisentDataset(data, GroundTruthPairOrigSampler(), transform=ToImgTensorF32())
dataloader = DataLoader(dataset=dataset, batch_size=128, shuffle=True)

# create the pytorch lightning system
module: pl.LightningModule = AdaVae(
    model=AutoEncoder(
        encoder=EncoderConv64(x_shape=data.x_shape, z_size=6, z_multiplier=2),
        decoder=DecoderConv64(x_shape=data.x_shape, z_size=6),
    ),
    cfg=AdaVae.cfg(
        optimizer='adam', optimizer_kwargs=dict(lr=1e-3),
        loss_reduction='mean_sum', beta=1, ada_average_mode='gvae', ada_thresh_mode='kl',
    )
)
wandb_logger = WandbLogger(project="beta-VAE")
# train the model
trainer = pl.Trainer(logger=wandb_logger, callbacks=[tc.betaControlCallback()], max_epochs = 10, fast_dev_run=is_test_run())
trainer.fit(module, dataloader)

module.eval()

get_repr = lambda x: module.encode(x.to(module.device))

a_results = {
        **metric_dci(dataset, get_repr, num_train=10 if is_test_run() else 1000, num_test=5 if is_test_run() else 500, boost_mode='sklearn'),
        **metric_mig(dataset, get_repr, num_train=20 if is_test_run() else 2000),
        **metric_sap(dataset, get_repr),
    }
print(a_results)
# AdaGVAE beta = 1 at e=7, beta = 0 5有了

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: iceburg. Use `wandb login --relogin` to force relogin


/opt/conda/lib/python3.9/site-packages/pytorch_lightning/loggers/wandb.py:351: UserWarning: There is a wandb run already in progress and newly created instances of `WandbLogger` will reuse this run. If this is not desired, call `wandb.finish()` before instantiating `WandbLogger`.
  rank_zero_warn(
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/opt/conda/lib/python3.9/site-packages/pytorch_lightning/trainer/configuration_validator.py:117: PossibleUserWarning: You defined a `validation_step` but have no `val_dataloader`. Skipping val loop.
  rank_zero_warn(

  | Name                          | Type                | Params
----------------------------------------------------------------------
0 | _model                        | AutoEncoder         | 910 K 
1 | _AeAndVaeMixin__recon_handler | ReconLossHandlerMse | 0     
----------------------------------------------------------------------

Training: 0it [00:00, ?it/s]

/opt/conda/lib/python3.9/site-packages/pytorch_lightning/callbacks/progress/base.py:247: UserWarning: The progress bar already tracks a metric with the name(s) 'loss' and `self.log('loss', ..., prog_bar=True)` will overwrite this value.  If this is undesired, change the name or override `get_metrics()` in the progress bar callback.
  rank_zero_warn(


trainer.current_epoch is now 7
beta was, 1
beta is now, 0


`Trainer.fit` stopped: `max_epochs=10` reached.



 Train end and beta NOW =  0
{'dci.informativeness_train': 0.9715, 'dci.informativeness_test': 0.6345000000000001, 'dci.disentanglement': 0.38012774579468395, 'dci.completeness': 0.4008640526950967, 'mig.discrete_score': 0.3217035148044986, 'sap.score': 0.02229999999999999}


In [1]:
import pytorch_lightning as pl
from pytorch_lightning.loggers.wandb import WandbLogger
from torch.utils.data import DataLoader
from disent.dataset import DisentDataset
from disent.dataset.data import XYObjectData
from disent.dataset.sampling import GroundTruthPairOrigSampler
from disent.frameworks.ae import Ae
from disent.frameworks.vae import BetaVae, AdaVae
from disent.model import AutoEncoder
from disent.model.ae import DecoderConv64, EncoderConv64
from disent.dataset.transform import ToImgTensorF32
from disent.util import is_test_run  # you can ignore and remove this
from disent.metrics import metric_dci, metric_mig, metric_sap 
import torch
import matplotlib.pyplot as plt
import numpy as np
import wandb
import callback as tc

wandb.init()
# prepare the data
data = XYObjectData()
dataset = DisentDataset(data, GroundTruthPairOrigSampler(), transform=ToImgTensorF32())
dataloader = DataLoader(dataset=dataset, batch_size=128, shuffle=True)

# create the pytorch lightning system
module: pl.LightningModule = AdaVae(
    model=AutoEncoder(
        encoder=EncoderConv64(x_shape=data.x_shape, z_size=6, z_multiplier=2),
        decoder=DecoderConv64(x_shape=data.x_shape, z_size=6),
    ),
    cfg=AdaVae.cfg(
        optimizer='adam', optimizer_kwargs=dict(lr=1e-3),
        loss_reduction='mean_sum', beta=1, ada_average_mode='gvae', ada_thresh_mode='kl',
    )
)
wandb_logger = WandbLogger(project="beta-VAE")
# train the model
trainer = pl.Trainer(logger=wandb_logger, callbacks=[tc.betaControlCallback()], max_epochs = 10, fast_dev_run=is_test_run())
trainer.fit(module, dataloader)

module.eval()

get_repr = lambda x: module.encode(x.to(module.device))

a_results = {
        **metric_dci(dataset, get_repr, num_train=10 if is_test_run() else 1000, num_test=5 if is_test_run() else 500, boost_mode='sklearn'),
        **metric_mig(dataset, get_repr, num_train=20 if is_test_run() else 2000),
        **metric_sap(dataset, get_repr),
    }
print(a_results)
# AdaGVAE beta = 1 at e=9, beta = 0 5有了

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: iceburg. Use `wandb login --relogin` to force relogin


/opt/conda/lib/python3.9/site-packages/pytorch_lightning/loggers/wandb.py:351: UserWarning: There is a wandb run already in progress and newly created instances of `WandbLogger` will reuse this run. If this is not desired, call `wandb.finish()` before instantiating `WandbLogger`.
  rank_zero_warn(
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/opt/conda/lib/python3.9/site-packages/pytorch_lightning/trainer/configuration_validator.py:117: PossibleUserWarning: You defined a `validation_step` but have no `val_dataloader`. Skipping val loop.
  rank_zero_warn(

  | Name                          | Type                | Params
----------------------------------------------------------------------
0 | _model                        | AutoEncoder         | 910 K 
1 | _AeAndVaeMixin__recon_handler | ReconLossHandlerMse | 0     
----------------------------------------------------------------------

Training: 0it [00:00, ?it/s]

/opt/conda/lib/python3.9/site-packages/pytorch_lightning/callbacks/progress/base.py:247: UserWarning: The progress bar already tracks a metric with the name(s) 'loss' and `self.log('loss', ..., prog_bar=True)` will overwrite this value.  If this is undesired, change the name or override `get_metrics()` in the progress bar callback.
  rank_zero_warn(


trainer.current_epoch is now 9
beta was, 1
beta is now, 0


`Trainer.fit` stopped: `max_epochs=10` reached.



 Train end and beta NOW =  0
{'dci.informativeness_train': 0.992, 'dci.informativeness_test': 0.7765000000000001, 'dci.disentanglement': 0.5177135363332566, 'dci.completeness': 0.5043791851253021, 'mig.discrete_score': 0.2952711240717714, 'sap.score': 0.02780000000000001}


In [1]:
import pytorch_lightning as pl
from pytorch_lightning.loggers.wandb import WandbLogger
from torch.utils.data import DataLoader
from disent.dataset import DisentDataset
from disent.dataset.data import XYObjectData
from disent.dataset.sampling import GroundTruthPairOrigSampler
from disent.frameworks.ae import Ae
from disent.frameworks.vae import BetaVae, AdaVae
from disent.model import AutoEncoder
from disent.model.ae import DecoderConv64, EncoderConv64
from disent.dataset.transform import ToImgTensorF32
from disent.util import is_test_run  # you can ignore and remove this
from disent.metrics import metric_dci, metric_mig, metric_sap 
import torch
import matplotlib.pyplot as plt
import numpy as np
import wandb
import callback as tc

wandb.init()
# prepare the data
data = XYObjectData()
dataset = DisentDataset(data, GroundTruthPairOrigSampler(), transform=ToImgTensorF32())
dataloader = DataLoader(dataset=dataset, batch_size=128, shuffle=True)

# create the pytorch lightning system
module: pl.LightningModule = AdaVae(
    model=AutoEncoder(
        encoder=EncoderConv64(x_shape=data.x_shape, z_size=6, z_multiplier=2),
        decoder=DecoderConv64(x_shape=data.x_shape, z_size=6),
    ),
    cfg=AdaVae.cfg(
        optimizer='adam', optimizer_kwargs=dict(lr=1e-3),
        loss_reduction='mean_sum', beta=0, ada_average_mode='gvae', ada_thresh_mode='kl',
    )
)
wandb_logger = WandbLogger(project="beta-VAE")
# train the model
trainer = pl.Trainer(logger=wandb_logger, callbacks=[tc.betaControlCallback()], max_epochs = 10, fast_dev_run=is_test_run())
trainer.fit(module, dataloader)

module.eval()

get_repr = lambda x: module.encode(x.to(module.device))

a_results = {
        **metric_dci(dataset, get_repr, num_train=10 if is_test_run() else 1000, num_test=5 if is_test_run() else 500, boost_mode='sklearn'),
        **metric_mig(dataset, get_repr, num_train=20 if is_test_run() else 2000),
        **metric_sap(dataset, get_repr),
    }
print(a_results)
# AdaGVAE beta = 0 at e=1, beta = 1 5有了

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: iceburg. Use `wandb login --relogin` to force relogin


/opt/conda/lib/python3.9/site-packages/pytorch_lightning/loggers/wandb.py:351: UserWarning: There is a wandb run already in progress and newly created instances of `WandbLogger` will reuse this run. If this is not desired, call `wandb.finish()` before instantiating `WandbLogger`.
  rank_zero_warn(
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/opt/conda/lib/python3.9/site-packages/pytorch_lightning/trainer/configuration_validator.py:117: PossibleUserWarning: You defined a `validation_step` but have no `val_dataloader`. Skipping val loop.
  rank_zero_warn(

  | Name                          | Type                | Params
----------------------------------------------------------------------
0 | _model                        | AutoEncoder         | 910 K 
1 | _AeAndVaeMixin__recon_handler | ReconLossHandlerMse | 0     
----------------------------------------------------------------------

Training: 0it [00:00, ?it/s]

/opt/conda/lib/python3.9/site-packages/pytorch_lightning/callbacks/progress/base.py:247: UserWarning: The progress bar already tracks a metric with the name(s) 'loss' and `self.log('loss', ..., prog_bar=True)` will overwrite this value.  If this is undesired, change the name or override `get_metrics()` in the progress bar callback.
  rank_zero_warn(


trainer.current_epoch is now 1
beta was, 0
beta is now, 1


`Trainer.fit` stopped: `max_epochs=10` reached.



 Train end and beta NOW =  1
{'dci.informativeness_train': 0.97975, 'dci.informativeness_test': 0.7475, 'dci.disentanglement': 0.44892538107491864, 'dci.completeness': 0.47912099331444696, 'mig.discrete_score': 0.2883159063105663, 'sap.score': 0.043000000000000003}


In [1]:
import pytorch_lightning as pl
from pytorch_lightning.loggers.wandb import WandbLogger
from torch.utils.data import DataLoader
from disent.dataset import DisentDataset
from disent.dataset.data import XYObjectData
from disent.dataset.sampling import GroundTruthPairOrigSampler
from disent.frameworks.ae import Ae
from disent.frameworks.vae import BetaVae, AdaVae
from disent.model import AutoEncoder
from disent.model.ae import DecoderConv64, EncoderConv64
from disent.dataset.transform import ToImgTensorF32
from disent.util import is_test_run  # you can ignore and remove this
from disent.metrics import metric_dci, metric_mig, metric_sap 
import torch
import matplotlib.pyplot as plt
import numpy as np
import wandb
import callback as tc

wandb.init()
# prepare the data
data = XYObjectData()
dataset = DisentDataset(data, GroundTruthPairOrigSampler(), transform=ToImgTensorF32())
dataloader = DataLoader(dataset=dataset, batch_size=128, shuffle=True)

# create the pytorch lightning system
module: pl.LightningModule = AdaVae(
    model=AutoEncoder(
        encoder=EncoderConv64(x_shape=data.x_shape, z_size=6, z_multiplier=2),
        decoder=DecoderConv64(x_shape=data.x_shape, z_size=6),
    ),
    cfg=AdaVae.cfg(
        optimizer='adam', optimizer_kwargs=dict(lr=1e-3),
        loss_reduction='mean_sum', beta=0, ada_average_mode='gvae', ada_thresh_mode='kl',
    )
)
wandb_logger = WandbLogger(project="beta-VAE")
# train the model
trainer = pl.Trainer(logger=wandb_logger, callbacks=[tc.betaControlCallback()], max_epochs = 10, fast_dev_run=is_test_run())
trainer.fit(module, dataloader)

module.eval()

get_repr = lambda x: module.encode(x.to(module.device))

a_results = {
        **metric_dci(dataset, get_repr, num_train=10 if is_test_run() else 1000, num_test=5 if is_test_run() else 500, boost_mode='sklearn'),
        **metric_mig(dataset, get_repr, num_train=20 if is_test_run() else 2000),
        **metric_sap(dataset, get_repr),
    }
print(a_results)
# AdaGVAE beta = 0 at e=3, beta = 1 5有了

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: iceburg. Use `wandb login --relogin` to force relogin


/opt/conda/lib/python3.9/site-packages/pytorch_lightning/loggers/wandb.py:351: UserWarning: There is a wandb run already in progress and newly created instances of `WandbLogger` will reuse this run. If this is not desired, call `wandb.finish()` before instantiating `WandbLogger`.
  rank_zero_warn(
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/opt/conda/lib/python3.9/site-packages/pytorch_lightning/trainer/configuration_validator.py:117: PossibleUserWarning: You defined a `validation_step` but have no `val_dataloader`. Skipping val loop.
  rank_zero_warn(

  | Name                          | Type                | Params
----------------------------------------------------------------------
0 | _model                        | AutoEncoder         | 910 K 
1 | _AeAndVaeMixin__recon_handler | ReconLossHandlerMse | 0     
----------------------------------------------------------------------

Training: 0it [00:00, ?it/s]

/opt/conda/lib/python3.9/site-packages/pytorch_lightning/callbacks/progress/base.py:247: UserWarning: The progress bar already tracks a metric with the name(s) 'loss' and `self.log('loss', ..., prog_bar=True)` will overwrite this value.  If this is undesired, change the name or override `get_metrics()` in the progress bar callback.
  rank_zero_warn(


trainer.current_epoch is now 3
beta was, 0
beta is now, 1


`Trainer.fit` stopped: `max_epochs=10` reached.



 Train end and beta NOW =  1
{'dci.informativeness_train': 0.9652499999999999, 'dci.informativeness_test': 0.662, 'dci.disentanglement': 0.27075096628862394, 'dci.completeness': 0.22816474307507986, 'mig.discrete_score': 0.0631221718553924, 'sap.score': 0.013850000000000001}


In [1]:
import pytorch_lightning as pl
from pytorch_lightning.loggers.wandb import WandbLogger
from torch.utils.data import DataLoader
from disent.dataset import DisentDataset
from disent.dataset.data import XYObjectData
from disent.dataset.sampling import GroundTruthPairOrigSampler
from disent.frameworks.ae import Ae
from disent.frameworks.vae import BetaVae, AdaVae
from disent.model import AutoEncoder
from disent.model.ae import DecoderConv64, EncoderConv64
from disent.dataset.transform import ToImgTensorF32
from disent.util import is_test_run  # you can ignore and remove this
from disent.metrics import metric_dci, metric_mig, metric_sap 
import torch
import matplotlib.pyplot as plt
import numpy as np
import wandb
import callback as tc

wandb.init()
# prepare the data
data = XYObjectData()
dataset = DisentDataset(data, GroundTruthPairOrigSampler(), transform=ToImgTensorF32())
dataloader = DataLoader(dataset=dataset, batch_size=128, shuffle=True)

# create the pytorch lightning system
module: pl.LightningModule = AdaVae(
    model=AutoEncoder(
        encoder=EncoderConv64(x_shape=data.x_shape, z_size=6, z_multiplier=2),
        decoder=DecoderConv64(x_shape=data.x_shape, z_size=6),
    ),
    cfg=AdaVae.cfg(
        optimizer='adam', optimizer_kwargs=dict(lr=1e-3),
        loss_reduction='mean_sum', beta=0, ada_average_mode='gvae', ada_thresh_mode='kl',
    )
)
wandb_logger = WandbLogger(project="beta-VAE")
# train the model
trainer = pl.Trainer(logger=wandb_logger, callbacks=[tc.betaControlCallback()], max_epochs = 10, fast_dev_run=is_test_run())
trainer.fit(module, dataloader)

module.eval()

get_repr = lambda x: module.encode(x.to(module.device))

a_results = {
        **metric_dci(dataset, get_repr, num_train=10 if is_test_run() else 1000, num_test=5 if is_test_run() else 500, boost_mode='sklearn'),
        **metric_mig(dataset, get_repr, num_train=20 if is_test_run() else 2000),
        **metric_sap(dataset, get_repr),
    }
print(a_results)
# AdaGVAE beta = 0 at e=7, beta = 1 5有了

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: iceburg. Use `wandb login --relogin` to force relogin


/opt/conda/lib/python3.9/site-packages/pytorch_lightning/loggers/wandb.py:351: UserWarning: There is a wandb run already in progress and newly created instances of `WandbLogger` will reuse this run. If this is not desired, call `wandb.finish()` before instantiating `WandbLogger`.
  rank_zero_warn(
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/opt/conda/lib/python3.9/site-packages/pytorch_lightning/trainer/configuration_validator.py:117: PossibleUserWarning: You defined a `validation_step` but have no `val_dataloader`. Skipping val loop.
  rank_zero_warn(

  | Name                          | Type                | Params
----------------------------------------------------------------------
0 | _model                        | AutoEncoder         | 910 K 
1 | _AeAndVaeMixin__recon_handler | ReconLossHandlerMse | 0     
----------------------------------------------------------------------

Training: 0it [00:00, ?it/s]

/opt/conda/lib/python3.9/site-packages/pytorch_lightning/callbacks/progress/base.py:247: UserWarning: The progress bar already tracks a metric with the name(s) 'loss' and `self.log('loss', ..., prog_bar=True)` will overwrite this value.  If this is undesired, change the name or override `get_metrics()` in the progress bar callback.
  rank_zero_warn(


trainer.current_epoch is now 7
beta was, 0
beta is now, 1


`Trainer.fit` stopped: `max_epochs=10` reached.



 Train end and beta NOW =  1
{'dci.informativeness_train': 0.99475, 'dci.informativeness_test': 0.8865, 'dci.disentanglement': 0.6819841219157023, 'dci.completeness': 0.6519969000755033, 'mig.discrete_score': 0.4758582514190737, 'sap.score': 0.011800000000000003}


In [1]:
import pytorch_lightning as pl
from pytorch_lightning.loggers.wandb import WandbLogger
from torch.utils.data import DataLoader
from disent.dataset import DisentDataset
from disent.dataset.data import XYObjectData
from disent.dataset.sampling import GroundTruthPairOrigSampler
from disent.frameworks.ae import Ae
from disent.frameworks.vae import BetaVae, AdaVae
from disent.model import AutoEncoder
from disent.model.ae import DecoderConv64, EncoderConv64
from disent.dataset.transform import ToImgTensorF32
from disent.util import is_test_run  # you can ignore and remove this
from disent.metrics import metric_dci, metric_mig, metric_sap 
import torch
import matplotlib.pyplot as plt
import numpy as np
import wandb
import callback as tc

wandb.init()
# prepare the data
data = XYObjectData()
dataset = DisentDataset(data, GroundTruthPairOrigSampler(), transform=ToImgTensorF32())
dataloader = DataLoader(dataset=dataset, batch_size=128, shuffle=True)

# create the pytorch lightning system
module: pl.LightningModule = AdaVae(
    model=AutoEncoder(
        encoder=EncoderConv64(x_shape=data.x_shape, z_size=6, z_multiplier=2),
        decoder=DecoderConv64(x_shape=data.x_shape, z_size=6),
    ),
    cfg=AdaVae.cfg(
        optimizer='adam', optimizer_kwargs=dict(lr=1e-3),
        loss_reduction='mean_sum', beta=0, ada_average_mode='gvae', ada_thresh_mode='kl',
    )
)
wandb_logger = WandbLogger(project="beta-VAE")
# train the model
trainer = pl.Trainer(logger=wandb_logger, callbacks=[tc.betaControlCallback()], max_epochs = 10, fast_dev_run=is_test_run())
trainer.fit(module, dataloader)

module.eval()

get_repr = lambda x: module.encode(x.to(module.device))

a_results = {
        **metric_dci(dataset, get_repr, num_train=10 if is_test_run() else 1000, num_test=5 if is_test_run() else 500, boost_mode='sklearn'),
        **metric_mig(dataset, get_repr, num_train=20 if is_test_run() else 2000),
        **metric_sap(dataset, get_repr),
    }
print(a_results)
# AdaGVAE beta = 0 at e=9, beta = 1 5有了

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: iceburg. Use `wandb login --relogin` to force relogin


/opt/conda/lib/python3.9/site-packages/pytorch_lightning/loggers/wandb.py:351: UserWarning: There is a wandb run already in progress and newly created instances of `WandbLogger` will reuse this run. If this is not desired, call `wandb.finish()` before instantiating `WandbLogger`.
  rank_zero_warn(
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/opt/conda/lib/python3.9/site-packages/pytorch_lightning/trainer/configuration_validator.py:117: PossibleUserWarning: You defined a `validation_step` but have no `val_dataloader`. Skipping val loop.
  rank_zero_warn(

  | Name                          | Type                | Params
----------------------------------------------------------------------
0 | _model                        | AutoEncoder         | 910 K 
1 | _AeAndVaeMixin__recon_handler | ReconLossHandlerMse | 0     
----------------------------------------------------------------------

Training: 0it [00:00, ?it/s]

/opt/conda/lib/python3.9/site-packages/pytorch_lightning/callbacks/progress/base.py:247: UserWarning: The progress bar already tracks a metric with the name(s) 'loss' and `self.log('loss', ..., prog_bar=True)` will overwrite this value.  If this is undesired, change the name or override `get_metrics()` in the progress bar callback.
  rank_zero_warn(


trainer.current_epoch is now 9
beta was, 0
beta is now, 1


`Trainer.fit` stopped: `max_epochs=10` reached.



 Train end and beta NOW =  1
{'dci.informativeness_train': 0.9615, 'dci.informativeness_test': 0.6535, 'dci.disentanglement': 0.2579765505708988, 'dci.completeness': 0.26653818843745913, 'mig.discrete_score': 0.19755160396858237, 'sap.score': 0.010899999999999996}
